#
<h1><span style="color:blue">AI for Security Project: Clank Ops</span></h1>

<p><em>Dataset:</em> <a href="https://huggingface.co/datasets/qualifire/prompt-injections-benchmark/viewer/default/test?row=85&views%5B%5D=test" target="_blank">Jailbreak attacks on LLMs</a></p>

<h2>Here we are! Four beautiful minds at work:</h2>
<ul>
    <li><strong>Tiziano Rossi</strong></li>
    <li><strong>Benedetta Tesi</strong></li>
    <li><strong>Francesco Friolo</strong></li>
    <li><strong>Francesca Falco</strong></li>
</ul>

#Loading the dataset

Mounts Google Drive to enable file access and persistence.

In [ ]:

# Mount Google Drive to read your dataset and save outputs
from google.colab import drive
drive.mount('/content/drive')


Define the path to the dataset stored on Google Drive

In [ ]:
DATASET_PATH = "/content/drive/MyDrive/AI_for_security/dataset.csv"

Loads the dataset and prints a few samples to inspect labels and text content

In [ ]:
import pandas as pd
df = pd.read_csv(DATASET_PATH)

from textwrap import fill

pd.set_option('display.max_colwidth', None)  # niente "..."
for i, row in df.head(5).iterrows():        # cambia 10 come vuoi
    print(f"#{i}  [{row['label']}]")
    print(fill(str(row['text']), width=100)) # va a capo ogni ~100 caratteri
    print("-" * 80)


Displays the number of rows and columns in the dataset.

In [ ]:
rows, cols = df.shape
print(f"NUmber of rows: {rows} \nNumber of columns: {cols}")

Sets the random seed to ensure reproducibility of experiments.

In [ ]:
random_seed = 50

#training/test/validation split

Splits the dataset into training, validation, and test sets using stratification.

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop('label', axis=1)
y = df['label']

X_train_temp, X_test_val_temp, y_train, y_test_val = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=random_seed,
    stratify=y
)

X_val_temp, X_test_temp, y_val, y_test = train_test_split(
    X_test_val_temp,
    y_test_val,
    test_size=0.5,
    random_state=random_seed,
    stratify=y_test_val
)

idx_train = X_train_temp.index
idx_val = X_val_temp.index
idx_test = X_test_temp.index

print(f"--- Suddivisione 70% / 15% / 15% ---")
print(f"Training Set (Indici):   {len(idx_train)} samples")
print(f"Validation Set (Indici): {len(idx_val)} samples")
print(f"Test Set (Indici):       {len(idx_test)} samples")

#Data exploration through blacklist

In [ ]:
import numpy as np

## NLTK installation

We are going to use the **NLTK** (Nartural Language ToolKit) package, a library for a suite of libraries and programs for natural language processing (NLP) in Python.


In [ ]:
!pip install nltk

Now we can import the package, to get access to all its functions...

In [ ]:
import nltk
import string

and we can download the packages to use for text analysis (more on this later).

In [ ]:
nltk.download('words')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

## Continue

In [ ]:
# Tokenize text into individual words using NLTK
from nltk.tokenize import word_tokenize

In [ ]:
punctuations = list(string.punctuation)   # list of punctuation characters
punctuations

Define the set of stopwords

In [ ]:
# Load the set of English stopwords to remove common non-informative words
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords

Stopwords are common words (e.g., “the”, “and”, “is”) that usually carry little semantic information and are therefore often removed during text preprocessing.

Define a stemmer to be used for preprocessing text

In [ ]:
stemmer = nltk.PorterStemmer()

In [ ]:
import pandas as pd
from itertools import islice
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer

# Configuration parameters
TEXT_COL = "text"          # column containing raw text
STOP_LANG = "english"      # stopwords language
USE_SNOWBALL = False       # choose stemmer type
LOWERCASE = True           # lowercase tokens

# Safety check
assert TEXT_COL in df.columns, f"La colonna '{TEXT_COL}' non esiste in df!"

# Extract text data indexed by original dataframe indices
data = {idx: row for idx, row in df[TEXT_COL].items()}

# Tokenize each document
preprocessed_data = {k: word_tokenize(v if isinstance(v, str) else str(v)) for k, v in data.items()}

# Preview tokenization
for key, value in islice(preprocessed_data.items(), 3):
    print(f"{key}: {value}")

# Prepare punctuation handling
punctuations = list(string.punctuation)
punct_str = "".join(punctuations)

# Remove punctuation tokens
def strip_and_filter_punct(tokens):
    out = []
    for tok in tokens:
        t = tok.strip(punct_str)
        if t and t not in punctuations:
            out.append(t)
    return out

preprocessed_data = {
    key: strip_and_filter_punct(tokens)
    for key, tokens in preprocessed_data.items()
}

# Preview after punctuation removal
for key, value in islice(preprocessed_data.items(), 3):
    print(f"{key}: {value}")

# Load stopwords
stop_set = set(stopwords.words(STOP_LANG))

# Remove stopwords and optionally lowercase
def remove_stopwords(tokens):

    if LOWERCASE:
        tokens = [t.lower() for t in tokens]
    kept = [t for t in tokens if t not in stop_set]
    return kept if len(tokens) > 2 else []

preprocessed_data = {
    key: remove_stopwords(tokens)
    for key, tokens in preprocessed_data.items()
}

# Preview after stopword removal
for key, value in islice(preprocessed_data.items(), 3):
    print(f"{key}: {value}")

# Initialize stemmer
if USE_SNOWBALL:
    stemmer = SnowballStemmer(STOP_LANG)
else:
    stemmer = PorterStemmer()

# Apply stemming
preprocessed_data = {
    key: [stemmer.stem(token) for token in tokens]
    for key, tokens in preprocessed_data.items()
}

# Preview after stemming
for key, value in islice(preprocessed_data.items(), 3):
    print(f"{key}: {value}")

# Store processed tokens back into the dataframe
processed_series = pd.Series(preprocessed_data)
df["tokens_processed"] = df.index.map(processed_series)

# Final check
df[["text", "label", "tokens_processed"]].head()


Remove stopwords (additionally we drop lists of tokens that contain less than two elements).

In [ ]:
STOP_LANG = "english"  # language used for stopword removal
stop_set = set(stopwords.words(STOP_LANG))  # load stopwords as a set for fast lookup

# Remove stopwords from each tokenized document
# Documents with very few tokens are replaced with an empty list
preprocessed_data = {
    file_name: [token for token in tokens if token not in stop_set] if len(tokens) > 2 else []
    for file_name, tokens in preprocessed_data.items()
}

Finally we apply stemming.

In [ ]:
# Apply stemming to each token to reduce words to their root form
preprocessed_data = {
    file_name: [stemmer.stem(token) for token in tokens]
    for file_name, tokens in preprocessed_data.items()
}

for key, value in islice(preprocessed_data.items(), 3):
    print(f"{key}: {value}")

print()

number of unique stems found

In [ ]:
# Collect all unique stems appearing in the dataset
all_stems = set()
for tokens in preprocessed_data.values():
    all_stems.update(tokens)

# Print the total number of unique stems found
print(f"\nTotal number of unique stems found: {len(all_stems)}")


training/test split (temporary, just for the blacklist)

In [ ]:
X_train = X_train_temp
X_test = X_val_temp

In [ ]:
# Blacklist-based model (training + validation)
from itertools import islice
from tqdm.auto import tqdm
import time
from sklearn.metrics import classification_report

# 1) Binary label mapping
def to_binary_label(y):
    """
    Map labels to binary format:
    jailbreak -> 1, benign -> 0
    """
    if isinstance(y, str):
        y = y.strip().lower()
        if y == "jailbreak":
            return 1
        elif y == "benign":
            return 0
    return int(y)

# Create label dictionary indexed by dataframe index
labels = df["label"].map(to_binary_label).to_dict()

# 2) Build blacklist from training set
jailbreak_words = set()
benign_words = set()

# Collect stems from training samples only
for idx in tqdm(X_train_temp.index, desc="Building blacklist (train set)"):
    stems = preprocessed_data.get(idx, [])
    if stems:
        if labels[idx] == 1:
            jailbreak_words.update(stems)
        else:
            benign_words.update(stems)

# Words appearing only in jailbreak samples
blacklist = jailbreak_words - benign_words

# 3) Quick blacklist inspection
print("Sample blacklist stems:")
for token in islice(blacklist, 5):
    print(token)

print(f"\nBlacklist size: {len(blacklist)}")

# 4) Keep only real English words
from nltk.corpus import words as nltk_words

word_set = set(nltk_words.words())
word_blacklist = word_set.intersection(blacklist)

print("Real-word blacklist size:", len(word_blacklist))

# 5) Validation set evaluation
tp = fp = tn = fn = 0
y_true_val = []
y_pred_val = []

for idx in tqdm(X_val_temp.index, desc="Evaluating on validation set"):
    true_label = labels[idx]
    stems = preprocessed_data.get(idx, [])

    # Predict jailbreak if any blacklist word is present
    pred_label = 1 if stems and set(stems) & blacklist else 0

    y_true_val.append(true_label)
    y_pred_val.append(pred_label)

    if pred_label == 1 and true_label == 1:
        tp += 1
    elif pred_label == 1 and true_label == 0:
        fp += 1
    elif pred_label == 0 and true_label == 1:
        fn += 1
    else:
        tn += 1

# 6) Metrics and report
print("\nConfusion Matrix (validation):")
print(f"| TP {tp:5d} | FN {fn:5d} |")
print(f"| FP {fp:5d} | TN {tn:5d} |")
print("\nClassification Report (validation):")
print(classification_report(y_true_val, y_pred_val,
                            target_names=["benign", "jailbreak"]))

The blacklist-based classifier achieves moderate performance on the validation set (accuracy ≈ 0.68). It shows a reasonable recall for the jailbreak class (0.63), indicating that a significant portion of malicious prompts is detected. However, precision for jailbreak remains lower (0.59), reflecting a non-negligible number of false positives. Overall, the results confirm that a simple lexical blacklist captures part of the jailbreak signal, but lacks sufficient discriminative power compared to learning-based models.

comparison with random classifier (taking into account the jailbreak frequency):

In [ ]:
# BINARY DATA PREPARATION (0/1)

# 1. True labels for the Validation Set (0/1)
# We use y_val (string Series) obtained from the 70/15/15 split
# y_val is already aligned in size with X_val_temp
y_val_binary = y_val.map(to_binary_label).values

# 2. True labels for the Training Set (0/1)
# We use y_train (string Series) obtained from the 70/15/15 split
y_train_binary = y_train.map(to_binary_label).values

# RANDOM CLASSIFIER (STRATIFIED) - BASELINE

# Positive class prevalence (jailbreak) in the training split
p_pos_train = y_train_binary.mean()
print(f"\nPositive class prevalence in training: {p_pos_train:.5f}")

# RANDOM GENERATOR with fixed seed (for reproducibility)
rng = np.random.default_rng(random_seed)

# Generate STRATIFIED random predictions
# (predict y=1 with probability equal to the training prevalence)
# The prediction vector must match the Validation Set size
len_val_set = len(y_val_binary)
y_pred_rand_strat = (rng.random(len_val_set) < p_pos_train).astype(int)

# EVALUATION FUNCTION
def evaluate_predictions(y_true, y_pred, title):
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))

    acc = (tp + tn) / len(y_true)
    prec = tp / (tp + fp) if (tp + fp) else 0.0
    rec  = tp / (tp + fn) if (tp + fn) else 0.0

    # F1 SCORE
    f1 = 2 * prec * rec / (prec + rec) if (prec + rec) else 0.0

    # SPECIFICITY (TN / (TN + FP))
    spec = tn / (tn + fp) if (tn + fp) else 0.0

    print(f"\n=== {title} ===")
    print("Counts:")
    print(f"TP: {tp}")
    print(f"FN: {fn}")
    print(f"FP: {fp}")
    print(f"TN: {tn}")

    print("\nConfusion matrix (rows=true, cols=pred):")
    print(f"| TP {tp:5d} | FN {fn:5d} |")
    print(f"| FP {fp:5d} | TN {tn:5d} |")

    print("\nMetrics:")
    print(f"Accuracy:    {acc:.5f}")
    print(f"Precision:   {prec:.5f}")
    print(f"Recall:      {rec:.5f}")
    print(f"F1 score:    {f1:.5f}")
    print(f"Specificity: {spec:.5f}")

# EVALUATION ON THE VALIDATION SET
evaluate_predictions(
    y_val_binary,        #Binary labels for the Validation Set
    y_pred_rand_strat,
    "Random classifier (stratified, evaluated on Validation Set)"
)

This stratified random classifier reflects the class distribution observed in the training set and serves as a baseline.
As expected, performance is close to chance level: accuracy is around 51% and both precision and recall for the jailbreak class are below 0.4.
These results confirm that meaningful structure must be learned from the data to outperform a naive random strategy.

The Blacklist is noticebly better than the Random Classifier, which makes us think that there is some kind of correlation between the meaning of the words in prompts and the jailbreak/benign label

In [ ]:
# Initialize placeholders for train/test splits (will be assigned later)
X_train, X_test, y_train_temporary = None, None, None

#Data visualization

In this data visualization section we explore the dataset before training any models, to better understand its structure and potential patterns.
We first look at the class distribution (benign vs jailbreak) to check whether the dataset is balanced or imbalanced. Then we examine the prompt lengths across classes to see if jailbreak prompts tend to be longer or differently structured than benign ones.

Next, we analyze the most frequent words in each class (after tokenization, stopword removal, and stemming) to identify which terms are particularly characteristic of jailbreaks compared to benign prompts. Finally, we inspect the presence of blacklist words (e.g., ignore, previous, override, forget, write, create) and compare how often they appear in each class, to assess whether they can serve as useful signals for detecting jailbreak attempts.



In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
sns.countplot(x='label', data=df, palette='YlOrBr')
plt.title('Distribuzione classi: benign vs jailbreak')
plt.gca().set_facecolor('#fff8dc')
plt.show()

#This plot shows how many samples belong to each class,
#comparing the count of benign prompts versus jailbreak prompts.
#It lets you quickly see whether the dataset is balanced or if one class is more frequent than the other.

In [ ]:
df['length'] = df['text'].str.split().apply(len)
sns.boxplot(x='label', y='length', data=df, palette='YlOrBr')
plt.title('Distribution of length of prompts')
plt.gca().set_facecolor('#fff8dc')

#This boxplot compares the prompt length (in words) between the two classes.
#It shows the median, spread, and outliers of lengths for benign vs jailbreak prompts,
#letting you see whether jailbreaks tend to be longer or shorter than benign prompts.


Normalizing the data might be a good idea!

In [ ]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re

stemmer = PorterStemmer()
stop = set(stopwords.words('english'))

def preprocess(text):
    words = re.findall(r'\b\w+\b', text.lower())
    return [stemmer.stem(w) for w in words if w not in stop]

df['tokens'] = df['text'].apply(preprocess)

# parole più frequenti nei jailbreak
jail_tokens = [w for tokens in df[df.label=='jailbreak'].tokens for w in tokens]
benign_tokens = [w for tokens in df[df.label=='benign'].tokens for w in tokens]

jail_freq = Counter(jail_tokens).most_common(20)
benign_freq = Counter(benign_tokens).most_common(20)


In [ ]:
j_df = pd.DataFrame(jail_freq, columns=['word','count'])
b_df = pd.DataFrame(benign_freq, columns=['word','count'])

fig, axes = plt.subplots(1,2, figsize=(14,6))
sns.barplot(y='word', x='count', data=j_df, ax=axes[0], palette='YlOrBr')
sns.barplot(y='word', x='count', data=b_df, ax=axes[1], palette='YlOrBr')
axes[0].set_title('Top parole jailbreak')
axes[1].set_title('Top parole benign')
for ax in axes: ax.set_facecolor('#fff8dc')
plt.show()

#This figure shows two side-by-side barplots with the most frequent stems for each class:
#on the left, the top words in jailbreak prompts, and on the right, the top words in benign prompts.
#It highlights which terms are most common in each class and allows you to visually compare their lexical profiles.


In [ ]:
blacklist = ['ignore', 'ethical', 'data', 'information', 'write', 'create']
df['blacklisted'] = df['text'].str.lower().apply(lambda t: any(word in t for word in blacklist))

sns.countplot(x='label', hue='blacklisted', data=df, palette='YlOrBr')
plt.title('Blacklist words per class')
plt.gca().set_facecolor('#fff8dc')
plt.show()

#This plot shows, for each class (benign vs jailbreak), how many prompts contain at least one word from the blacklist and how many do not.
#It lets you see whether blacklist terms like “ignore”, “ethical”, “data”, “information”, “write”,
#or “create” appear more often in jailbreak prompts than in benign ones.


In [ ]:
blacklist = ['ignore', 'previous', 'override', 'forget', 'write', 'create']
df['blacklisted'] = df['text'].str.lower().apply(lambda t: any(word in t for word in blacklist))

sns.countplot(x='label', hue='blacklisted', data=df, palette='YlOrBr')
plt.title('Blacklist words per class')
plt.gca().set_facecolor('#fff8dc')
plt.show()

#This plot shows, for each class (benign vs jailbreak), how many prompts contain at least one of the updated blacklist
#words (“ignore”, “previous”, “override”, “forget”, “write”, “create”) and how many do not.
#It lets you check whether these jailbreak-like trigger words appear more frequently in jailbreak prompts than in benign ones.


# Word embeddings

In order to train Word2Vec on the data we first need to convert it to the right format.
- For training Word2Vec, it is usual to **separate data into individual sentences** and then tokenize those sentences separately
- So let's use regular expressions to remove the end-of-line characters from each document and then split the documents into sentences using a regular expression that looks for question marks, exclamation marks, and periods, followed by a space:

In [ ]:
pip install gensim pandas nltk


In [ ]:
import re
import pandas as pd
from gensim.models.word2vec import Word2Vec


# 1. Load dataset
texts = df["text"].astype(str).tolist()

# 2. Clean text: remove line breaks
texts = [re.sub(r"\s+", " ", t).strip() for t in texts]

# 3. Split into sentences
def split_into_sentences(text):
    return re.split(r"(?<=[\.\?\!])\s+", text)

sentences = []
for t in texts:
    sentences.extend(split_into_sentences(t))

# 4. Tokenize:
#    re.sub('\W', ' ', sentence).lower().split()
tokenized_sentences = [
    re.sub(r"\W", " ", sentence).lower().split()
    for sentence in sentences
]

# 5. Remove sentences with only 1 word
tokenized_sentences = [
    sentence for sentence in tokenized_sentences
    if len(sentence) > 1
]

# Visual check
for s in tokenized_sentences[:10]:
    print(s)

# 6. Train Word2Vec
model = Word2Vec(
    tokenized_sentences,
    vector_size=30,
    min_count=5,
    window=10
)

# 7. Vocabulary size
print("Vocabulary size:", len(model.wv))


## Visualising the embedding vectors using t-SNE

We'll now visualise some of the word vectors in a 3 dimensional space using t-SNE.

The vocabulary of word vectors is quite large (around 25,000). Giving them all to t-SNE will cause it to take far too long to converge.
- So let's first choose a random subset of 500 terms to show:

In [ ]:
# Randomly sample 500 words from the Word2Vec vocabulary to inspect learned tokens
import random

# random.seed(50)  # <-- fixed seed

sample = random.sample(list(model.wv.key_to_index), 500)
print(sample)


Now we'll get the word vectors for the sampled terms:

In [ ]:
word_vectors = model.wv[sample]
word_vectors

Displays the dimensionality of the Word2Vec embedding matrix (number of words x embedding size).

In [ ]:
word_vectors.shape

We provide the word vectors to t-SNE to fit the model and reduce the data to 3 dimensions

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(word_vectors)

Now transform the data into 3 columns (for x, y, and z):

In [ ]:
import numpy as np

x, y, z = np.transpose(tsne_embedding)

And generate the 3d plot:

In [ ]:
import plotly.express as px

fig = px.scatter_3d(x=x, y=y, z=z)
fig.update_traces(marker=dict(size=3,line=dict(width=2)))
fig.show()

Well that's a not a particularly interesting 3d plot!
- How about we label some of the points on the graph to see what words they correspond to:

In [ ]:
fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=sample[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

We perform a simple vector arithmetic operation in the embedding space to explore semantic relationships.
Starting from a word associated with jailbreak prompts, we add and subtract unrelated words to observe whether the resulting vector remains close to other jailbreak-related terms.

In [ ]:
v_answer = model.wv['answer']
v_use = model.wv['use']
v_character = model.wv['character']

vec = v_answer + (v_use - v_character)

model.wv.similar_by_vector(vec, topn=10)



The resulting nearest neighbors suggest that the vector arithmetic shifted the original word toward a more instruction- and prompt-related semantic space. Terms like prompt, questions, and violates indicate that the embedding captures contextual associations linked to structured or potentially sensitive user inputs, rather than a strictly semantic notion of “jailbreak.”

we got "questions", which was a jailbreak word accoridng to the other embbedding

Let's extend the random set of terms with a set of colours to see if they cluster:

## Loading Pre-trained Embedding


In [ ]:
#Load pre-trained GloVe embeddings trained on Twitter and Wikipedia corpora.
import gensim.downloader as api

model_twitter = api.load("glove-twitter-50")
model_wiki = api.load("glove-wiki-gigaword-50")

How big are their vocabularies?

In [ ]:
print(f"Vocabulary size twitter model:   {len(model_twitter)}")
print(f"Vocabulary size wikipedia model: {len(model_wiki)}")

Woah, they are big vocabularies!
- The twitter one has over a million tokens!


In [ ]:
term = 'jailbreak'    # target word
print(f'Twitter embedding, most similar words to: {term}')
model_twitter.most_similar(term)    # inspect semantic neighbors in GloVe-Twitter space

The Twitter embedding associates “jailbreak” mainly with mobile devices, operating systems, and firmware-related terms, reflecting its common usage in a technological and consumer context rather than in a security-oriented sense.

And in the Wikipedia embedding:

In [ ]:
print(f'Wikipedia embedding, most similar words to: {term}')
model_wiki.most_similar(term)

The Wikipedia embedding captures a more abstract and heterogeneous notion of “jailbreak”, associating it with violent, military, or disruptive events rather than with the technical meaning observed in the Twitter embedding. This highlights how the semantic representation of the same term strongly depends on the training corpus.

Which embedding is better?

Try some other words:

In [ ]:
term = 'benign'
print(f'Twitter embedding:   {model_twitter.most_similar(term)}')
print(f'Wikipedia embedding: {model_wiki.most_similar(term)}')

In both embeddings, “benign” is strongly linked to medical contexts, highlighting a semantic mismatch with its use as a class label in this task

Embedding spaces have interesting geometric properties, where translation between different word vectors caries semantic meaning.


In [ ]:
# Vector arithmetic: combine semantic relations (answer + use − character)
vec = model_wiki.get_vector('answer') + (model_wiki.get_vector('use') - model_wiki.get_vector('character'))
vec

- Then look for the most similar vectors to it:

In [ ]:
#The vector arithmetic moves the semantic meaning of “answer” toward prompt- and instruction-related concepts.
model_wiki.similar_by_vector(vec)

Try it again but with the Twitter based model

In [ ]:
model = model_twitter
vec = model.get_vector('answer') + (model.get_vector('use') - model.get_vector('character'))
model.similar_by_vector(vec)

The resulting vector is mainly associated with generic communication and interaction terms (e.g., ask, tell, respond), rather than jailbreak-related concepts. This suggests that, in the Twitter embedding space, the vector arithmetic captures semantic usage patterns of language rather than security- or policy-specific meanings.

# data structure to save all models for further use

This section defines a central model registry used to store and organize all trained classifiers and their variants. The `register_model` function saves each model in a nested dictionary structure keyed by model name, embedding type, and variant name, along with its trained instance and parameters. It also prints a message indicating whether a new model has been added or an existing one has been overwritten, making it easier to manage and reuse different models across experiments.


In [ ]:
models_registry = {}

an example of the data structure:

In [ ]:
# models_registry = {
#     "SVC": {
#         "w2v": {
#             "grid_C_0.1_5.0": {
#                 "model": <trained SVC>,
#                 "params": {...},
#                 "embedding": "w2v",
#             },
#             "default": {
#                 "model": <trained SVC>,
#                 "params": {...},
#                 "embedding": "w2v",
#             }
#         },
#         "glove_twitter": { ... },
#         ...
#     },
#     "LogisticRegression": { ... },
# }


In [ ]:
models_registry = {}

def register_model(
    registry,
    model_name: str,
    embedding_name: str,
    variant_name: str,
    model,
    params: dict | None = None,
):

    # Crea i livelli se non esistono
    if model_name not in registry:
        registry[model_name] = {}

    if embedding_name not in registry[model_name]:
        registry[model_name][embedding_name] = {}

    # Messaggio se il variant esiste già
    if variant_name in registry[model_name][embedding_name]:
        print(f"⚠️  ATTENZIONE: stai sovrascrivendo il modello "
              f"[{model_name}] embedding [{embedding_name}] variante [{variant_name}]")
    else:
        print(f"✅ Nuovo modello registrato: "
              f"[{model_name}] embedding [{embedding_name}] variante [{variant_name}]")

    # Salvataggio
    registry[model_name][embedding_name][variant_name] = {
        "model": model,
        "params": params,
        "embedding": embedding_name,
    }


#text vectorization

This section builds and manages all the text vectorizations used in the experiments. It first creates a bag-of-words representation from stemmed tokens with a CountVectorizer, then loads or computes several dense embeddings: Word2Vec, two pre-trained GloVe variants, FastText, and SBERT.  Finally, all embedding matrices are normalized using statistics from the training set, so that downstream models can compare and use these different representations in a consistent way.


To speed up the rest of the notebook, we also save the vectorizations on drive

In [ ]:
embedding_names = ["w2v", "glove_twitter", "glove_wiki", "fasttext", "sbert"]
all_representations = ["bow"] + embedding_names

##with the stems of the words

we use "preprocessed_data" from the Blacklist chapter

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

X_train_stems = [preprocessed_data.get(idx, []) for idx in idx_train]
X_val_stems = [preprocessed_data.get(idx, []) for idx in idx_val]
X_test_stems = [preprocessed_data.get(idx, []) for idx in idx_test]

X_train_joined = [" ".join(stems) for stems in X_train_stems]
X_val_joined = [" ".join(stems) for stems in X_val_stems]
X_test_joined = [" ".join(stems) for stems in X_test_stems]

vectorizer = CountVectorizer()

X_train_vector = vectorizer.fit_transform(X_train_joined)
X_val_vector = vectorizer.transform(X_val_joined)
X_test_vector = vectorizer.transform(X_test_joined)

print("\n--- Vettorizzazione Finale ---")
print(f"Training Matrix Shape:     {X_train_vector.shape}")
print(f"Validation Matrix Shape:   {X_val_vector.shape}")
print(f"Test Matrix Shape:         {X_test_vector.shape}")

In [ ]:
#load the other embeddings if already done

In [ ]:
from google.colab import drive
import os
import pickle

# Mount Google Drive
drive.mount('/content/drive')

DRIVE_BASE_PATH = "/content/drive/MyDrive/AI_for_security"
FILENAME = "other_embeddings_data.pkl"
LOAD_PATH = os.path.join(DRIVE_BASE_PATH, FILENAME)

# 1. Initialize control variable
loading_ok = False

try:
    with open(LOAD_PATH, 'rb') as f:
        loaded_embeddings = pickle.load(f)
    print("✅ Dizionario caricato con successo!")

    other_embeddings = loaded_embeddings

    # 2. finished loading
    loading_ok = True

except FileNotFoundError:
    # Initialize empty dictionary if there is no file
    other_embeddings = {
        "train": {},
        "val": {},
        "test": {}
    }
    print(f"❌ File non trovato in: {LOAD_PATH}")

except Exception as e:
    print(f"❌ Errore durante il caricamento: {e}")

print(f"Stato caricamento: {loading_ok}")

## w2v

In [ ]:
import re
from nltk.corpus import stopwords
stopwords_en = set(stopwords.words("english"))

def text_to_vector(text, w2v):
    text = re.sub(r"\s+", " ", str(text)).strip().lower()

    sentences = split_into_sentences(text)

    tokens = []
    for sent in sentences:
        # only keep characters
        sent = re.sub(r"[^a-zA-Z]", " ", sent)
        tokens.extend(sent.split())

    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords_en]


    # Generate the average vector
    vecs = [w2v[w] for w in tokens if w in w2v.key_to_index]

    if not vecs:
        return np.zeros(w2v.vector_size)

    return np.mean(vecs, axis=0)


In [ ]:
import numpy as np
from tqdm.auto import tqdm

if not loading_ok:
    print("⚠️ Dati non trovati. Avvio il calcolo di Word2Vec...")

    # Feature matrixes based on Word2Vec
    X_train_w2v = np.vstack([
        text_to_vector(df.loc[i, "text"], model)
        for i in idx_train
    ])

    X_val_w2v = np.vstack([
        text_to_vector(df.loc[i, "text"], model)
        for i in idx_val
    ])

    X_test_w2v = np.vstack([
        text_to_vector(df.loc[i, "text"], model)
        for i in idx_test
    ])

    other_embeddings["train"]["w2v"] = X_train_w2v
    other_embeddings["val"]["w2v"]   = X_val_w2v
    other_embeddings["test"]["w2v"]  = X_test_w2v

    print("Calcolo completato.")

else:
    print("✅ Dati già caricati dal Drive. Recupero le variabili dal dizionario...")
    X_train_w2v = other_embeddings["train"]["w2v"]
    X_val_w2v   = other_embeddings["val"]["w2v"]
    X_test_w2v  = other_embeddings["test"]["w2v"]

print(f"\nDimensioni finali W2V:")
print(f"Train: {X_train_w2v.shape}")
print(f"Val:   {X_val_w2v.shape}")
print(f"Test:  {X_test_w2v.shape}")

In [ ]:
if not loading_ok:

  other_embeddings["train"]["w2v"] = X_train_w2v
  other_embeddings["val"]["w2v"]   = X_val_w2v
  other_embeddings["test"]["w2v"]  = X_test_w2v


## Glove pre-trained

In [ ]:
## GloVe pre-trained

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# Function to compute the average embedding vector of a document/prompt
def get_avg_embedding(stems, embedding_model, embedding_dim=50):
    """
    Compute the average embedding of a document.

    Parameters
    ----------
    stems : list of pre-processed tokens/stems for a prompt.
    embedding_model : loaded GloVe model (e.g., model_twitter or model_wiki).
    embedding_dim : dimensionality of the embedding vectors (e.g., 50).

    Returns
    -------
    np.array of shape (embedding_dim,)
        The mean vector over all known word embeddings in the prompt.
        If no token is found in the vocabulary, a zero vector is returned.
    """

    # List to collect vectors of words that are found in the model vocabulary
    word_vectors = []

    for word in stems:
        # Check if the word is present in the embedding model vocabulary
        if word in embedding_model:
            word_vectors.append(embedding_model[word])

    if word_vectors:
        # Compute the arithmetic mean of all the word vectors
        # This is a dense representation of the whole prompt
        return np.mean(word_vectors, axis=0)
    else:
        # If no words are found (all tokens are OOV), return a zero vector
        return np.zeros(embedding_dim)

import numpy as np
from tqdm.auto import tqdm

In [ ]:
if not loading_ok:
    print("⚠️ Data not found. Computing GloVe Twitter embeddings...")

    # Choose the embedding model (e.g., GloVe Twitter 50D)
    embedding_model = model_twitter
    EMBEDDING_DIM = 50

    # 1. Training set embeddings
    X_train_glove_t = np.vstack([
        get_avg_embedding(stems, embedding_model, EMBEDDING_DIM)
        for stems in tqdm(X_train_stems, desc="GloVe Twitter: Training Set")
    ])

    # 2. Validation set embeddings
    X_val_glove_t = np.vstack([
        get_avg_embedding(stems, embedding_model, EMBEDDING_DIM)
        for stems in tqdm(X_val_stems, desc="GloVe Twitter: Validation Set")
    ])

    # 3. Test set embeddings
    X_test_glove_t = np.vstack([
        get_avg_embedding(stems, embedding_model, EMBEDDING_DIM)
        for stems in tqdm(X_test_stems, desc="GloVe Twitter: Test Set")
    ])

    # Store results in the dictionary (important for later saving to Drive)
    other_embeddings["train"]["glove_twitter"] = X_train_glove_t
    other_embeddings["val"]["glove_twitter"]   = X_val_glove_t
    other_embeddings["test"]["glove_twitter"]  = X_test_glove_t

    print("GloVe Twitter computation completed.")

else:
    print("✅ GloVe Twitter data recovered from dictionary.")

    # Recover variables from the loaded dictionary
    # (If you skip this, X_train_glove_t / X_val_glove_t / X_test_glove_t will not exist)
    X_train_glove_t = other_embeddings["train"]["glove_twitter"]
    X_val_glove_t   = other_embeddings["val"]["glove_twitter"]
    X_test_glove_t  = other_embeddings["test"]["glove_twitter"]

# Final shape check
print(f"\n--- GloVe Twitter 50D Embeddings ---")
print(f"Training Emb Shape:   {X_train_glove_t.shape}")
print(f"Validation Emb Shape: {X_val_glove_t.shape}")
print(f"Test Emb Shape:       {X_test_glove_t.shape}")

In [ ]:
if not loading_ok:

  other_embeddings["train"]["glove_twitter"] = X_train_glove_t
  other_embeddings["val"]["glove_twitter"]   = X_val_glove_t
  other_embeddings["test"]["glove_twitter"]  = X_test_glove_t

In [ ]:
if not loading_ok:
    print("⚠️ Data not found. Computing GloVe Wikipedia embeddings...")

    # Choose the embedding model (GloVe Wikipedia 50D)
    embedding_model = model_wiki
    EMBEDDING_DIM = 50

    # 1. Training set embeddings
    X_train_glove_w = np.vstack([
        get_avg_embedding(stems, embedding_model, EMBEDDING_DIM)
        for stems in tqdm(X_train_stems, desc="GloVe Wiki: Training Set")
    ])

    # 2. Validation set embeddings
    X_val_glove_w = np.vstack([
        get_avg_embedding(stems, embedding_model, EMBEDDING_DIM)
        for stems in tqdm(X_val_stems, desc="GloVe Wiki: Validation Set")
    ])

    # 3. Test set embeddings
    X_test_glove_w = np.vstack([
        get_avg_embedding(stems, embedding_model, EMBEDDING_DIM)
        for stems in tqdm(X_test_stems, desc="GloVe Wiki: Test Set")
    ])

    # Store results in the dictionary
    other_embeddings["train"]["glove_wiki"] = X_train_glove_w
    other_embeddings["val"]["glove_wiki"]   = X_val_glove_w
    other_embeddings["test"]["glove_wiki"]  = X_test_glove_w

    print("GloVe Wikipedia computation completed.")

else:
    print("✅ GloVe Wikipedia data recovered from dictionary.")

    # Recover variables from the loaded dictionary
    X_train_glove_w = other_embeddings["train"]["glove_wiki"]
    X_val_glove_w   = other_embeddings["val"]["glove_wiki"]
    X_test_glove_w  = other_embeddings["test"]["glove_wiki"]

# Final shape check
print(f"\n--- GloVe Wikipedia 50D Embeddings ---")
print(f"Training Emb Shape:   {X_train_glove_w.shape}")
print(f"Validation Emb Shape: {X_val_glove_w.shape}")
print(f"Test Emb Shape:       {X_test_glove_w.shape}")

In [ ]:
if not loading_ok:

  other_embeddings["train"]["glove_wiki"] = X_train_glove_w
  other_embeddings["val"]["glove_wiki"]   = X_val_glove_w
  other_embeddings["test"]["glove_wiki"]  = X_test_glove_w

## FastText

### pretrained

In [ ]:
!pip install fasttext

In [ ]:
# Download and overwrite if the file exists (-O specifies the output filename)
!wget -O cc.en.300.bin.gz http://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz

# Decompress forcing overwrite
!gzip -d -f cc.en.300.bin.gz

In [ ]:
import fasttext

if not loading_ok:
  ft_model = fasttext.load_model('cc.en.300.bin')

In [ ]:
# Simplified version for FastText (get_word_vector naturally handles OOV words)
def get_avg_embedding_fast(stems, embedding_model):
    """
    Compute the average FastText embedding for a list of tokens.

    This function uses FastText's get_word_vector, which does not raise
    OOV errors and can construct embeddings for unseen words via subword info.
    """
    if not stems:
        return np.zeros(embedding_model.get_dimension())

    # FastText provides a fast, direct method for getting word vectors
    vectors = [embedding_model.get_word_vector(word) for word in stems]
    return np.mean(vectors, axis=0)

In [ ]:
import numpy as np
from tqdm.auto import tqdm

# -------------------------------------------------------------------
# FastText embeddings (300D)
# -------------------------------------------------------------------
if not loading_ok:
    print("⚠️ Data not found. Computing FastText (300D) embeddings...")

    embedding_model = ft_model
    EMBEDDING_DIM = 300

    # 1. Training set embeddings
    X_train_ft = np.vstack([
        get_avg_embedding_fast(stems, embedding_model)
        for stems in tqdm(X_train_stems, desc="FastText: Training")
    ])

    # 2. Validation set embeddings
    X_val_ft = np.vstack([
        get_avg_embedding_fast(stems, embedding_model)
        for stems in tqdm(X_val_stems, desc="FastText: Validation")
    ])

    # 3. Test set embeddings
    X_test_ft = np.vstack([
        get_avg_embedding_fast(stems, embedding_model)
        for stems in tqdm(X_test_stems, desc="FastText: Test")
    ])

    # Store results in the dictionary
    other_embeddings["train"]["fasttext"] = X_train_ft
    other_embeddings["val"]["fasttext"]   = X_val_ft
    other_embeddings["test"]["fasttext"]  = X_test_ft

    print("FastText computation completed.")

else:
    print("✅ FastText data recovered from dictionary.")

    # Recover variables from the loaded dictionary
    X_train_ft = other_embeddings["train"]["fasttext"]
    X_val_ft   = other_embeddings["val"]["fasttext"]
    X_test_ft  = other_embeddings["test"]["fasttext"]

# Final shape check
print(f"\n--- FastText 300D Embeddings ---")
print(f"Training Emb Shape:   {X_train_ft.shape}")
print(f"Validation Emb Shape: {X_val_ft.shape}")
print(f"Test Emb Shape:       {X_test_ft.shape}")

In [ ]:
if not loading_ok:
  other_embeddings["train"]["fasttext"] = X_train_ft
  other_embeddings["val"]["fasttext"]   = X_val_ft
  other_embeddings["test"]["fasttext"]  = X_test_ft


## Sentence Bert

In [ ]:
from sentence_transformers import SentenceTransformer
sbert_model = SentenceTransformer('all-MiniLM-L12-v2')


In [ ]:
if not loading_ok:

    # Recover texts for the three splits from the main dataframe
    texts_train = df.loc[idx_train, "text"].astype(str).tolist()
    texts_val   = df.loc[idx_val,   "text"].astype(str).tolist()
    texts_test  = df.loc[idx_test,  "text"].astype(str).tolist()

    print("Encoding TRAIN with SBERT...")
    X_train_sbert = sbert_model.encode(
        texts_train,
        convert_to_numpy=True,
        show_progress_bar=True
    )

    print("Encoding VAL with SBERT...")
    X_val_sbert = sbert_model.encode(
        texts_val,
        convert_to_numpy=True,
        show_progress_bar=True
    )

    print("Encoding TEST with SBERT...")
    X_test_sbert = sbert_model.encode(
        texts_test,
        convert_to_numpy=True,
        show_progress_bar=True
    )


In [ ]:
if not loading_ok:
  other_embeddings["train"]["sbert"] = X_train_sbert
  other_embeddings["val"]["sbert"]   = X_val_sbert
  other_embeddings["test"]["sbert"]  = X_test_sbert


In [ ]:
if not loading_ok:
  print("Train SBERT:", X_train_sbert.shape)
  print("Val SBERT:",   X_val_sbert.shape)
  print("Test SBERT:",  X_test_sbert.shape)


## Normalize data

In [ ]:
from sklearn.preprocessing import StandardScaler

# -------------------------------------------------------------------
# Normalization: Standardize each embedding set separately
# -------------------------------------------------------------------
if not loading_ok:
    print("\n--- Normalizing all embeddings ---")

    # Iterate over all embedding names stored in the dictionary
    # (e.g., w2v, glove_twitter, glove_wiki, fasttext, sbert, etc.)
    for emb_name in other_embeddings["train"].keys():
        print(f"🔹 Normalizing embedding: {emb_name}")

        scaler = StandardScaler()

        # Fit ONLY on the training set (compute mean and std)
        X_train_norm = scaler.fit_transform(other_embeddings["train"][emb_name])

        # Transform validation and test sets using train statistics
        X_val_norm  = scaler.transform(other_embeddings["val"][emb_name])
        X_test_norm = scaler.transform(other_embeddings["test"][emb_name])

        # Overwrite the dictionary with the normalized versions
        other_embeddings["train"][emb_name] = X_train_norm
        other_embeddings["val"][emb_name]   = X_val_norm
        other_embeddings["test"][emb_name]  = X_test_norm

    print("\n✅ Normalization completed and dictionary updated!")

else:
    print("✅ Data loaded from Drive (assumed already normalized before saving).")


## Save on drive (delete the file on drive if you want to overwrite save)

In [ ]:
# Base path on Drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/AI_for_security"

# Ensure the directory exists
if not os.path.exists(DRIVE_BASE_PATH):
    print(f"Creating directory: {DRIVE_BASE_PATH}")
    os.makedirs(DRIVE_BASE_PATH)

# Output filename (we use .pkl, standard for pickle)
FILENAME = "other_embeddings_data.pkl"
SAVE_PATH = os.path.join(DRIVE_BASE_PATH, FILENAME)

print(f"The dictionary will be saved to: {SAVE_PATH}")

In [ ]:
import pickle
import os  # Import os to check whether the file already exists

# 1. Check if the file already exists before saving
if os.path.exists(SAVE_PATH):
    # If the file exists, print a warning and skip saving
    print(f"\n⚠️ WARNING: File already exists at: {SAVE_PATH}")
    print("Save skipped to avoid accidental overwrite.")
else:
    # 2. Save only if the file does NOT exist
    try:
        with open(SAVE_PATH, 'wb') as f:
            pickle.dump(other_embeddings, f)

        print("\n✅ Save completed successfully!")

    except Exception as e:
        print(f"\n❌ Critical error while saving: {e}")

# other data visualization

In this “other data visualization” section we explore how different embedding methods (Word2Vec, FastText and SBERT) represent our prompts and how those representations behave. We look at predicted class distributions from simple classifiers built on top of each embedding, compare the norms of the embedding vectors across classes, inspect which words are closest to class centroids in embedding space, and visualize how “semantic blacklist” scores differ between benign and jailbreak prompts. This helps us understand not just the raw text, but how each representation captures jailbreak-like patterns.


##**other data visualization for Sentence Bert**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

# 0. Encode with SBERT if embeddings are missing
try:
    X_train_sbert
    X_val_sbert
except NameError:
    print("SBERT embeddings not found in memory: encoding from scratch...")

    # load SBERT model
    sbert_model = SentenceTransformer("all-MiniLM-L12-v2")

    # texts for train/val (assumes df, idx_train, idx_val already defined)
    texts_train = df.loc[idx_train, "text"].astype(str).tolist()
    texts_val   = df.loc[idx_val,   "text"].astype(str).tolist()

    X_train_sbert = sbert_model.encode(
        texts_train,
        convert_to_numpy=True,
        show_progress_bar=True
    )
    X_val_sbert = sbert_model.encode(
        texts_val,
        convert_to_numpy=True,
        show_progress_bar=True
    )

    # optional: save back into other_embeddings if you use it elsewhere
    if "train" in other_embeddings and "val" in other_embeddings:
        other_embeddings["train"]["sbert"] = X_train_sbert
        other_embeddings["val"]["sbert"]   = X_val_sbert

print("X_train_sbert shape:", X_train_sbert.shape)
print("X_val_sbert shape:",   X_val_sbert.shape)
print("len(y_train):", len(y_train))

# 1. Use SBERT embeddings
X_train = X_train_sbert
X_val   = X_val_sbert

# 2. Train classifier on SBERT embeddings
clf_sbert = LogisticRegression(max_iter=1000, n_jobs=-1)
clf_sbert.fit(X_train, y_train)

# 3. Predictions on validation set
y_val_pred_sbert = clf_sbert.predict(X_val)

# 4. Build a DataFrame just for plotting
df_sbert_pred = pd.DataFrame({
    "label": y_val_pred_sbert
})

# 5. SAME TYPE OF PLOT: class distribution (predicted)
plt.figure(figsize=(6, 4))
sns.countplot(x="label", data=df_sbert_pred, palette="YlOrBr")
plt.title("Predicted class distribution (SBERT + Logistic Regression)")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Predicted label")
plt.ylabel("Count")
plt.show()

#This countplot shows how often the SBERT + Logistic Regression classifier predicts each class on the validation set.
#If the bars are roughly the same height, SBERT is predicting jailbreak and benign with similar frequency;
#if one bar is much taller, the model is biased towards that class. Comparing this to the true class distribution (your earlier plot on df["label"])
#tells you whether SBERT is overpredicting the majority class or actually correcting some class imbalance.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 0) Robust retrieval of SBERT embeddings
try:
    X_train_sbert
except NameError:
    X_train_sbert = other_embeddings["train"]["sbert"]

try:
    X_val_sbert
except NameError:
    X_val_sbert = other_embeddings["val"]["sbert"]

# The test split may not exist: handle it safely
try:
    X_test_sbert
except NameError:
    X_test_sbert = None  # treat as optional

print("X_train_sbert shape:", X_train_sbert.shape)
print("X_val_sbert shape:",   X_val_sbert.shape)
if X_test_sbert is not None:
    print("X_test_sbert shape:",  X_test_sbert.shape)

# 1) Column for SBERT norm
df["sbert_norm"] = np.nan

# Fill train and val (which we definitely have)
df.loc[idx_train, "sbert_norm"] = np.linalg.norm(X_train_sbert, axis=1)
df.loc[idx_val,   "sbert_norm"] = np.linalg.norm(X_val_sbert,   axis=1)

# If the test split exists, add it as well
if X_test_sbert is not None:
    df.loc[idx_test, "sbert_norm"] = np.linalg.norm(X_test_sbert, axis=1)

# 2) Same type of plot: boxplot by class
plt.figure(figsize=(6, 4))
sns.boxplot(x="label", y="sbert_norm", data=df, palette="YlOrBr")
plt.title("Distribution of SBERT norms for prompts")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Label")
plt.ylabel("‖SBERT embedding‖ (L2)")
plt.show()

# Here we are comparing the L2 norm of the SBERT embeddings between jailbreak and benign prompts.
# If one class has consistently higher norms (higher median and upper quartiles), it suggests SBERT represents those prompts as
# “larger” vectors in embedding space, which can indicate differences in semantic richness or how the model encodes that type of content.
# If the distributions overlap a lot, SBERT norm alone is not a strong separator; if they diverge, this scalar feature already
# carries some class signal.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

# 0. Ensure we have the SBERT model loaded
try:
    sbert_model
except NameError:
    sbert_model = SentenceTransformer("all-MiniLM-L12-v2")

# 1. Assumption: X_train_sbert and y_train already exist.
#    If they are not in RAM, recover them from the dictionary:
try:
    X_train_sbert
    y_train
except NameError:
    X_train_sbert = other_embeddings["train"]["sbert"]
    # y_train must already exist in the notebook (e.g., created by the train/val/test split)

y_train_arr = np.array(y_train)

mask_jail = (y_train_arr == "jailbreak")
mask_ben  = (y_train_arr == "benign")

jail_centroid   = X_train_sbert[mask_jail].mean(axis=0)
benign_centroid = X_train_sbert[mask_ben].mean(axis=0)

# 2. "Light" version: use only words already present in jail_freq / benign_freq,
#    and limit the number of candidates to avoid heavy RAM usage.
MAX_CANDIDATES = 300  # you can increase/decrease this

jail_words   = [w for w, _ in jail_freq][:MAX_CANDIDATES]
benign_words = [w for w, _ in benign_freq][:MAX_CANDIDATES]

def top_words_for_centroid_sbert(centroid, candidate_words, topn=15):
    """
    Compute the words closest to a SBERT centroid among a list
    of candidate words (light version, no huge vocabularies).
    """
    if len(candidate_words) == 0:
        return pd.DataFrame(columns=["word", "count"])

    # Encode candidate words with SBERT
    word_embs = sbert_model.encode(
        candidate_words,
        convert_to_numpy=True,
        show_progress_bar=False
    )

    # Normalize for cosine similarity
    c = centroid / (np.linalg.norm(centroid) + 1e-12)
    w_norm = word_embs / (np.linalg.norm(word_embs, axis=1, keepdims=True) + 1e-12)

    sims = w_norm @ c  # dot product = cosine similarity
    sims = np.asarray(sims)

    idx = np.argsort(sims)[::-1][:topn]
    top_words = [candidate_words[i] for i in idx]
    top_sims  = sims[idx]

    return pd.DataFrame({
        "word": top_words,
        "count": top_sims  # use 'count' to reuse your plotting code
    })

# 3. DataFrames for jailbreak / benign (light SBERT version)
j_df = top_words_for_centroid_sbert(jail_centroid,   jail_words,   topn=15)
b_df = top_words_for_centroid_sbert(benign_centroid, benign_words, topn=15)

# 4. Same type of plot: two horizontal barplots side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.barplot(y='word', x='count', data=j_df, ax=axes[0], palette='YlOrBr')
sns.barplot(y='word', x='count', data=b_df, ax=axes[1], palette='YlOrBr')

axes[0].set_title('Top SBERT words near jailbreak centroid (light)')
axes[1].set_title('Top SBERT words near benign centroid (light)')

axes[0].set_xlabel('Cosine similarity')
axes[1].set_xlabel('Cosine similarity')

for ax in axes:
    ax.set_facecolor('#fff8dc')

plt.tight_layout()
plt.show()

# These two horizontal barplots show which words (from your dataset vocabulary) are closest in SBERT space
# to the jailbreak centroid and to the benign centroid.
# The jailbreak side highlights what SBERT considers “typical” jailbreak-like terms; the benign side shows words
# more characteristic of benign prompts. Comparing the two plots tells you whether SBERT is capturing intuitive differences:
# for example, jailbreak might lean towards instruction-breaking, system-manipulation, or sensitive-content words,
# while benign might show more neutral or task-oriented vocabulary.
# This gives a more interpretable view of what SBERT has learned about the two classes.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

# 0) Ensure we have the SBERT model
try:
    sbert_model
except NameError:
    sbert_model = SentenceTransformer("all-MiniLM-L12-v2")

# 1) First blacklist
blacklist = ['ignore', 'ethical', 'data', 'information', 'write', 'create']

# 2) SBERT centroid of the blacklist
bl_embs = sbert_model.encode(blacklist, convert_to_numpy=True, show_progress_bar=False)
blacklist_centroid = bl_embs.mean(axis=0)

# 3) Robust retrieval of SBERT embeddings for the three splits
try:
    X_train_sbert
except NameError:
    X_train_sbert = other_embeddings["train"]["sbert"]

try:
    X_val_sbert
except NameError:
    X_val_sbert   = other_embeddings["val"]["sbert"]

# Test split may not exist or may not have SBERT; handle with .get
X_test_sbert = None
if "test" in other_embeddings:
    X_test_sbert = other_embeddings["test"].get("sbert", None)

print("X_train_sbert:", X_train_sbert.shape)
print("X_val_sbert:",   X_val_sbert.shape)
print("X_test_sbert is None?", X_test_sbert is None)

# 4) Function for cosine similarity to centroid
def sim_to_centroid(matrix, centroid):
    c = centroid / (np.linalg.norm(centroid) + 1e-12)
    norms = np.linalg.norm(matrix, axis=1, keepdims=True) + 1e-12
    m_norm = matrix / norms
    return m_norm @ c  # dot product = cosine similarity

# 5) Column for SBERT similarity to the blacklist centroid
df["sbert_sim_blacklist"] = np.nan

df.loc[idx_train, "sbert_sim_blacklist"] = sim_to_centroid(X_train_sbert, blacklist_centroid)
df.loc[idx_val,   "sbert_sim_blacklist"] = sim_to_centroid(X_val_sbert,   blacklist_centroid)

# Use the test split only if it really exists
if (X_test_sbert is not None) and (len(idx_test) > 0):
    df.loc[idx_test, "sbert_sim_blacklist"] = sim_to_centroid(X_test_sbert, blacklist_centroid)

# 6) Data-driven threshold: e.g. top 20% most similar
sims = df["sbert_sim_blacklist"].dropna()
threshold = sims.quantile(0.8)

df["sbert_blacklisted"] = df["sbert_sim_blacklist"] > threshold
print(df["sbert_blacklisted"].value_counts(dropna=False))

# 7) Same type of plot: countplot per class with hue on SBERT flag
plt.figure(figsize=(6, 4))
sns.countplot(x="label", hue="sbert_blacklisted", data=df, palette="YlOrBr")
plt.title("Presence of 'semantic blacklist' (SBERT) per class")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

# In this plot, we take a first blacklist (ignore, ethical, data, information, write, create),
# compute a SBERT centroid for those words, and then measure how similar each prompt’s embedding is to that centroid.
# Prompts above the chosen similarity threshold are marked sbert_blacklisted = True.
# The countplot shows, for each class, how many prompts are flagged by this semantic blacklist.
# If jailbreak has a much higher proportion of True than benign, this blacklist and SBERT similarity are good at capturing jailbreak-like prompts.
# If both bars look similar, this blacklist does not discriminate well in SBERT space.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer

# 0) Ensure we have the SBERT model
try:
    sbert_model
except NameError:
    sbert_model = SentenceTransformer("all-MiniLM-L12-v2")

# 1) Updated blacklist
blacklist = ['ignore', 'previous', 'override', 'forget', 'write', 'create']

# 2) SBERT centroid of the blacklist
bl_embs = sbert_model.encode(blacklist, convert_to_numpy=True, show_progress_bar=False)
blacklist_centroid = bl_embs.mean(axis=0)

# 3) Robust retrieval of SBERT embeddings for the three splits
try:
    X_train_sbert
except NameError:
    X_train_sbert = other_embeddings["train"]["sbert"]

try:
    X_val_sbert
except NameError:
    X_val_sbert = other_embeddings["val"]["sbert"]

# Test split may not exist / may not have SBERT
X_test_sbert = None
if "test" in other_embeddings:
    X_test_sbert = other_embeddings["test"].get("sbert", None)

print("X_train_sbert:", X_train_sbert.shape)
print("X_val_sbert:",   X_val_sbert.shape)
print("X_test_sbert is None?", X_test_sbert is None)

# 4) Function for cosine similarity to centroid
def sim_to_centroid(matrix, centroid):
    c = centroid / (np.linalg.norm(centroid) + 1e-12)
    norms = np.linalg.norm(matrix, axis=1, keepdims=True) + 1e-12
    m_norm = matrix / norms
    return m_norm @ c  # dot product = cosine similarity

# 5) Column for SBERT similarity to the blacklist
df["sbert_sim_blacklist"] = np.nan

df.loc[idx_train, "sbert_sim_blacklist"] = sim_to_centroid(X_train_sbert, blacklist_centroid)
df.loc[idx_val,   "sbert_sim_blacklist"] = sim_to_centroid(X_val_sbert,   blacklist_centroid)

if (X_test_sbert is not None) and (len(idx_test) > 0):
    df.loc[idx_test, "sbert_sim_blacklist"] = sim_to_centroid(X_test_sbert, blacklist_centroid)

# 6) Data-driven threshold: e.g. top 20% most similar
sims = df["sbert_sim_blacklist"].dropna()
threshold = sims.quantile(0.8)   # you can change 0.8 to 0.7 / 0.9 if you want more/less "True"

df["sbert_blacklisted"] = df["sbert_sim_blacklist"] > threshold
print(df["sbert_blacklisted"].value_counts(dropna=False))

# 7) Same plot: countplot per class with hue on SBERT flag
plt.figure(figsize=(6, 4))
sns.countplot(x="label", hue="sbert_blacklisted", data=df, palette="YlOrBr")
plt.title("Presence of 'semantic blacklist' SBERT per class")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

# In this plot, we take an updated blacklist (ignore, previous, override, forget, write, create),
# compute a SBERT centroid for those words, and then measure how similar each prompt’s embedding is to that centroid.
# Prompts above the chosen similarity threshold are marked sbert_blacklisted = True.
# The countplot shows, for each class, how many prompts are flagged by this semantic blacklist.
# If jailbreak has a much higher proportion of True than benign,
# this blacklist and SBERT similarity are good at capturing jailbreak-like prompts.
# If both bars look similar, this blacklist does not discriminate well in SBERT space.


##**other data visualization for Fast Text**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 1. Train a classifier on FastText embeddings
X_train = X_train_ft
X_val   = X_val_ft

clf_ft = LogisticRegression(max_iter=1000, n_jobs=-1)
clf_ft.fit(X_train, y_train)

# 2. Predictions on validation (or test) set
y_val_pred_ft = clf_ft.predict(X_val)

# 3. Build a small DataFrame just for plotting
df_ft_pred = pd.DataFrame({
    "label": y_val_pred_ft
})

# 4. SAME TYPE OF PLOT: countplot of classes
plt.figure(figsize=(6, 4))
sns.countplot(x="label", data=df_ft_pred, palette="YlOrBr")
plt.title("Predicted class distribution (FastText + Logistic Regression)")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Predicted label")
plt.ylabel("Count")
plt.show()

#This countplot shows how often the FastText + Logistic Regression classifier predicts each class on the validation set,
#letting you see if the model tends to favor jailbreak or benign.

In [ ]:
import os
import subprocess
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import fasttext
import pandas as pd

# 0. Ensure FastText file is present (download if missing)
fasttext_path = "cc.en.300.bin"

if not os.path.exists(fasttext_path):
    # scarica e decomprimi il modello pre-addestrato
    print("FastText file not found, downloading...")
    subprocess.run(
        [
            "wget",
            "http://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz"
        ],
        check=True
    )
    subprocess.run(["gzip", "-d", "cc.en.300.bin.gz"], check=True)

    assert os.path.exists(fasttext_path), "Download/decompression failed, file still missing."

# 1. Load FastText model (if not already loaded)
try:
    ft_model
except NameError:
    ft_model = fasttext.load_model(fasttext_path)
    print("FastText model loaded from:", fasttext_path)

# 2. Function to get FastText sentence embedding (no newlines)
def text_to_vector_fast(text, embedding_model=None):
    if embedding_model is None:
        embedding_model = ft_model

    if pd.isna(text):
        return np.zeros(embedding_model.get_dimension())

    cleaned = str(text).replace("\n", " ").replace("\r", " ").strip()
    return embedding_model.get_sentence_vector(cleaned)

# 3. Compute L2 norm of FastText embedding for each text
df["ft_norm"] = df["text"].apply(
    lambda txt: np.linalg.norm(text_to_vector_fast(txt))
)

# 4. Same boxplot as before, but with FastText
plt.figure(figsize=(6, 4))
sns.boxplot(x="label", y="ft_norm", data=df, palette="YlOrBr")
plt.title("Distribuzione norme FastText dei prompt")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Label")
plt.ylabel("‖FastText embedding‖ (L2)")
plt.show()

#This boxplot compares the L2 norm of FastText sentence embeddings between benign and jailbreak prompts,
#showing whether one class tends to have systematically larger or smaller embedding magnitudes.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# OPTIONAL: stopword list to remove very generic words
stopwords = {
    "the","a","an","to","of","in","on","for","and","or","but",
    "it","is","are","was","were","be","been","being",
    "this","that","these","those",
    "there","here","any","some","many","much",
    "how","about","way","have","has","had","not","no",
    "do","does","did","can","could","would","should",
    "i","you","he","she","we","they","them","me","my","your",
}

# 1. Class labels array
y_train_arr = np.array(y_train)

mask_jail = (y_train_arr == "jailbreak")
mask_ben  = (y_train_arr == "benign")

# Centroids in FastText space
jail_centroid   = X_train_ft[mask_jail].mean(axis=0)
benign_centroid = X_train_ft[mask_ben].mean(axis=0)

# 2. Helper: rank ONLY given words by FastText similarity to centroid
def top_words_for_centroid_ft(centroid, ft_model, candidate_words, topn=15):
    c = centroid / (np.linalg.norm(centroid) + 1e-12)

    words = []
    sims  = []

    for w in candidate_words:
        # filter stopwords and very short tokens
        if len(w) < 3:
            continue
        if w.lower() in stopwords:
            continue

        # FastText handles OOV via subword, so this is safe
        v = ft_model.get_word_vector(w)
        v_norm = v / (np.linalg.norm(v) + 1e-12)
        sim = np.dot(c, v_norm)  # cosine similarity (both normalized)
        words.append(w)
        sims.append(sim)

    if not words:
        return pd.DataFrame(columns=["word", "count"])

    sims = np.array(sims)
    words = np.array(words)

    idx = np.argsort(sims)[::-1][:topn]
    top_words = words[idx]
    top_sims  = sims[idx]

    return pd.DataFrame({
        "word": top_words,
        "count": top_sims   # keep 'count' to reuse your plotting code
    })

# 3. Build candidate word lists from your freq structures
#    (assuming jail_freq / benign_freq = list of (word, count))
jail_words   = [w for w, _ in jail_freq]
benign_words = [w for w, _ in benign_freq]

j_df = top_words_for_centroid_ft(jail_centroid, ft_model, jail_words,   topn=15)
b_df = top_words_for_centroid_ft(benign_centroid, ft_model, benign_words, topn=15)

# 4. SAME TYPE OF PLOT: two horizontal barplots side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.barplot(y='word', x='count', data=j_df, ax=axes[0], palette='YlOrBr')
sns.barplot(y='word', x='count', data=b_df, ax=axes[1], palette='YlOrBr')

axes[0].set_title('Top parole FastText vicine a jailbreak (dataset vocab)')
axes[1].set_title('Top parole FastText vicine a benign (dataset vocab)')

axes[0].set_xlabel('Cosine similarity')
axes[1].set_xlabel('Cosine similarity')

for ax in axes:
    ax.set_facecolor('#fff8dc')

plt.tight_layout()
plt.show()

#This boxplot compares the L2 norm of FastText sentence embeddings between benign and jailbreak prompts,
#showing whether one class tends to have systematically larger or smaller embedding magnitudes

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# 1) Initial blacklist
blacklist = ['ignore', 'ethical', 'data', 'information', 'write', 'create']

# 2) Build the FastText centroid of the blacklist
#    (assuming ft_model is already loaded)
vocab = set(ft_model.get_words())
blacklist_vecs = [
    ft_model.get_word_vector(w) for w in blacklist
    if w in vocab
]

blacklist_centroid = np.mean(blacklist_vecs, axis=0)

# 3) Functions for sentence embedding and similarity
def ft_sentence_vector(text, model=ft_model):
    if pd.isna(text):
        return np.zeros(model.get_dimension())
    cleaned = str(text).replace("\n", " ").replace("\r", " ").strip()
    return model.get_sentence_vector(cleaned)

def ft_blacklist_similarity(text):
    v = ft_sentence_vector(text)
    if np.linalg.norm(v) == 0:
        return np.nan
    return np.dot(v, blacklist_centroid) / (
        np.linalg.norm(v) * np.linalg.norm(blacklist_centroid) + 1e-12
    )

# 4) Compute similarity for each prompt
df["ft_sim_blacklist"] = df["text"].apply(ft_blacklist_similarity)

# 5) Data-driven threshold: e.g. top 20% most similar
sims = df["ft_sim_blacklist"].dropna()
threshold = sims.quantile(0.8)   # you can change 0.8 → 0.7 / 0.9 etc.

df["ft_blacklisted"] = df["ft_sim_blacklist"] > threshold

print(df["ft_blacklisted"].value_counts(dropna=False))

# 6) Same plot: countplot per class with hue on FastText flag
plt.figure(figsize=(6, 4))
sns.countplot(x='label', hue='ft_blacklisted', data=df, palette='YlOrBr')
plt.title('Presence of "semantic blacklist" FastText per class')
plt.gca().set_facecolor('#fff8dc')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

# Here each prompt gets a similarity score to the centroid of the blacklist words and those above a threshold are flagged.
# The countplot shows, per class, how many prompts are semantically close to this blacklist,
# indicating how discriminative it is for jailbreak vs benign.


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# 1) Updated blacklist
blacklist = ['ignore', 'previous', 'override', 'forget', 'write', 'create']

# 2) Build the FastText centroid of the blacklist
#    (assuming ft_model is already loaded)
vocab = set(ft_model.get_words())
blacklist_vecs = [
    ft_model.get_word_vector(w) for w in blacklist
    if w in vocab
]

# Safety: if none of the blacklist words are in the vocabulary
if len(blacklist_vecs) == 0:
    raise ValueError("None of the blacklist words are present in the FastText vocabulary.")

blacklist_centroid = np.mean(blacklist_vecs, axis=0)

# 3) Helper functions: sentence embedding + similarity
def ft_sentence_vector(text, model=ft_model):
    if pd.isna(text):
        return np.zeros(model.get_dimension())
    cleaned = str(text).replace("\n", " ").replace("\r", " ").strip()
    return model.get_sentence_vector(cleaned)

def ft_blacklist_similarity(text):
    v = ft_sentence_vector(text)
    if np.linalg.norm(v) == 0:
        return np.nan
    return np.dot(v, blacklist_centroid) / (
        np.linalg.norm(v) * np.linalg.norm(blacklist_centroid) + 1e-12
    )

# 4) Similarity for each prompt
df["ft_sim_blacklist"] = df["text"].apply(ft_blacklist_similarity)

# 5) Data-driven threshold: top 20% most similar (you can change 0.8)
sims = df["ft_sim_blacklist"].dropna()
threshold = sims.quantile(0.8)

df["ft_blacklisted"] = df["ft_sim_blacklist"] > threshold
print(df["ft_blacklisted"].value_counts(dropna=False))

# 6) Same plot: countplot per class with hue on FastText flag
plt.figure(figsize=(6, 4))
sns.countplot(x='label', hue='ft_blacklisted', data=df, palette='YlOrBr')
plt.title("Presence of 'semantic blacklist' FastText per class")
plt.gca().set_facecolor('#fff8dc')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

# Here each prompt gets a similarity score to the centroid of the updated blacklist words,
# and those above a fixed percentile (e.g., 80%) are flagged as ft_blacklisted.
# The countplot shows, per class, how many prompts are semantically close to this blacklist,
# indicating how discriminative it is for jailbreak vs benign.


##**other data visualization for w2v:**

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression

# 1. Train a classifier on W2V embeddings
X_train = X_train_w2v
X_val   = X_val_w2v

clf_w2v = LogisticRegression(max_iter=1000, n_jobs=-1)
clf_w2v.fit(X_train, y_train)

# 2. Predictions on validation (or test) set
y_val_pred_w2v = clf_w2v.predict(X_val)

# 3. Build a small DataFrame just for plotting
df_w2v_pred = pd.DataFrame({
    "label": y_val_pred_w2v
})

# 4. SAME TYPE OF PLOT: countplot of classes
plt.figure(figsize=(6, 4))
sns.countplot(x="label", data=df_w2v_pred, palette="YlOrBr")
plt.title("Predicted class distribution (W2V + Logistic Regression)")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Predicted label")
plt.ylabel("Count")
plt.show()

#This countplot shows how often the W2V + Logistic Regression classifier predicts each class on the validation set,
#revealing whether the model tends to favor jailbreak or benign prompts.


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Compute a scalar W2V-based feature for each text: the L2 norm of its embedding
df["w2v_norm"] = df["text"].apply(
    lambda txt: np.linalg.norm(text_to_vector(txt, model))
)

# 2. Same type of plot: boxplot per class, but using w2v_norm instead of length
plt.figure(figsize=(6, 4))
sns.boxplot(x="label", y="w2v_norm", data=df, palette="YlOrBr")
plt.title("Distribuzione norme W2V dei prompt")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Label")
plt.ylabel("‖W2V embedding‖ (L2)")
plt.show()

#This boxplot compares the L2 norm of the Word2Vec-based embeddings between benign and jailbreak prompts,
#showing whether one class tends to have systematically larger or smaller W2V representations.


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# ---- stopword list (same as before, extend if you like) ----
stopwords = {
    "the","a","an","to","of","in","on","for","and","or","but",
    "it","is","are","was","were","be","been","being",
    "this","that","these","those",
    "there","here","any","some","many","much",
    "how","about","way","have","has","had","not","no",
    "do","does","did","can","could","would","should",
    "i","you","he","she","we","they","them","me","my","your",
}

# ---- 1. Class centroids ----
y_train_arr = np.array(y_train)

mask_jail = (y_train_arr == "jailbreak")
mask_ben  = (y_train_arr == "benign")

jail_centroid   = X_train_w2v[mask_jail].mean(axis=0)
benign_centroid = X_train_w2v[mask_ben].mean(axis=0)

# ---- 2. Helper: top-N words (here N=100) ----
def top_words_for_centroid(centroid, w2v_model, topn=100):
    c = centroid / (np.linalg.norm(centroid) + 1e-12)

    words = []
    sims = []

    for w in w2v_model.key_to_index.keys():
        if len(w) < 3:
            continue
        if w.lower() in stopwords:
            continue

        v = w2v_model[w]
        v_norm = v / (np.linalg.norm(v) + 1e-12)
        sim = np.dot(c, v_norm)
        words.append(w)
        sims.append(sim)

    sims = np.array(sims)
    words = np.array(words)

    idx = np.argsort(sims)[::-1][:topn]
    top_words = words[idx]
    top_sims  = sims[idx]

    return pd.DataFrame({
        "word": top_words,
        "similarity": top_sims
    })

# ---- 3. DataFrames for jailbreak / benign (top 100) ----
j_df = top_words_for_centroid(jail_centroid, model, topn=100)
b_df = top_words_for_centroid(benign_centroid, model, topn=100)

# ---- 4. Plot: two horizontal barplots side by side ----
# height increased to fit 100 labels
fig, axes = plt.subplots(1, 2, figsize=(18, 30))

sns.barplot(
    y="word",
    x="similarity",
    data=j_df,
    ax=axes[0],
    palette="YlOrBr"
)
sns.barplot(
    y="word",
    x="similarity",
    data=b_df,
    ax=axes[1],
    palette="YlOrBr"
)

axes[0].set_title("Top 100 parole W2V vicine a jailbreak (no stopwords)")
axes[1].set_title("Top 100 parole W2V vicine a benign (no stopwords)")

axes[0].set_xlabel("Cosine similarity")
axes[1].set_xlabel("Cosine similarity")

for ax in axes:
    ax.set_facecolor("#fff8dc")

plt.tight_layout()
plt.show()

#These two tall barplots list the 100 words whose W2V vectors are closest to the jailbreak and benign centroids, respectively.
#They highlight which terms Word2Vec associates most strongly with each class once generic stopwords are removed.


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1) Blacklist words
blacklist = ['ignore', 'previous', 'override', 'forget', 'write', 'create']

# 2) W2V centroid of the blacklist
blacklist_vecs = [model[w] for w in blacklist if w in model.key_to_index]
blacklist_centroid = np.mean(blacklist_vecs, axis=0)

def w2v_similarity(text, w2v_model, centroid):
    """Cosine similarity between text embedding and blacklist centroid."""
    v = text_to_vector(text, w2v_model)
    if np.linalg.norm(v) == 0:
        return np.nan
    return np.dot(v, centroid) / (np.linalg.norm(v) * np.linalg.norm(centroid) + 1e-12)

# 3) W2V similarity of each prompt to the blacklist centroid
df["w2v_sim"] = df["text"].apply(lambda t: w2v_similarity(t, model, blacklist_centroid))

# 4) Data-driven threshold: e.g. top 20% most similar
threshold = df["w2v_sim"].quantile(0.8)   # you can try 0.7, 0.9, etc.

df["w2v_blacklisted"] = df["w2v_sim"] > threshold

print(df["w2v_blacklisted"].value_counts(dropna=False))

# 5) Same plot as before
plt.figure(figsize=(6, 4))
sns.countplot(x="label", hue="w2v_blacklisted", data=df, palette="YlOrBr")
plt.title("Presence of 'semantic blacklist' W2V per class")
plt.gca().set_facecolor("#fff8dc")
plt.xlabel("Label")
plt.ylabel("Count")
plt.show()

# Here each prompt receives a cosine similarity score to the W2V centroid of the blacklist words,
# and those above a fixed percentile (e.g., 80%) are flagged as w2v_blacklisted.
# The countplot shows, per class, how many prompts are semantically close to this blacklist.


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1) Blacklist words
blacklist = ['ignore', 'previous', 'override', 'forget', 'write', 'create']

# 2) W2V centroid of the blacklist
blacklist_vecs = [model[w] for w in blacklist if w in model.key_to_index]
blacklist_centroid = np.mean(blacklist_vecs, axis=0)

def w2v_similarity(text, w2v_model, centroid):
    """Cosine similarity between text embedding and blacklist centroid."""
    v = text_to_vector(text, w2v_model)
    if np.linalg.norm(v) == 0:
        return np.nan
    return np.dot(v, centroid) / (np.linalg.norm(v) * np.linalg.norm(centroid) + 1e-12)

# 3) W2V similarity of each prompt with the blacklist
df["w2v_sim"] = df["text"].apply(lambda t: w2v_similarity(t, model, blacklist_centroid))

# 4) Automatic threshold choice: search for a percentile that yields both True and False
sims = df["w2v_sim"].dropna()

threshold = None
for q in [0.9, 0.8, 0.7, 0.6, 0.5]:
    t = sims.quantile(q)
    flags = sims > t
    if flags.nunique() == 2:   # we get both 0 and 1
        threshold = t
        break

# If for some reason we do not find anything, fall back to the mean
if threshold is None:
    threshold = sims.mean()

print("Chosen threshold:", threshold)

df["w2v_blacklisted"] = df["w2v_sim"] > threshold
print(df["w2v_blacklisted"].value_counts(dropna=False))

# 5) Same plot as before
plt.figure(figsize=(6, 4))
sns.countplot(x='label', hue='w2v_blacklisted', data=df, palette='YlOrBr')
plt.title("Presence of semantic blacklist (W2V) per class")
plt.gca().set_facecolor('#fff8dc')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

# This is similar to the previous graph but uses an adaptive threshold chosen to ensure both True and False values appear.
# The resulting countplot shows how this more robust thresholding affects the distribution of “blacklisted” prompts across
# benign and jailbreak classes.


# Logistic regression, SVC, Naive Bayes

In this chapter we build and compare classic machine-learning classifiers for the jailbreak vs benign task. Starting from the preprocessed texts, we vectorize them (bag-of-words and various embeddings), then train and evaluate Multinomial Naive Bayes, Logistic Regression, and Support Vector Machines. We also tune key hyperparameters (like C and alpha) and inspect metrics and confusion matrices to understand model performance and possible under- or overfitting.


### Classifiers fitting and evaluation

We can see how different classifiers behave on the same data set and compare their performances.

As for now we will work with simple linear classifiers:
- *Naive Bayes*
- *Logistic Regression*
- *Support Vector Machines*

In [ ]:
import time


In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
)
import matplotlib.pyplot as plt


def evaluate_classifier(
    clf,
    X_val,
    y_val,
    pos_label="jailbreak",
    neg_label="benign",
    show_plots=True,
):

    # 1) Predictions
    y_pred = clf.predict(X_val)

    # 2) Base Metrics
    acc = accuracy_score(y_val, y_pred)
    prec = precision_score(y_val, y_pred, pos_label=pos_label)
    rec = recall_score(y_val, y_pred, pos_label=pos_label)
    f1 = f1_score(y_val, y_pred, pos_label=pos_label)

    spec = recall_score(y_val, y_pred, pos_label=neg_label)

    print(f"Accuracy:     {acc:.5f}")
    print(f"Precision:    {prec:.5f}   (positive class: {pos_label})")
    print(f"Recall:       {rec:.5f}   (sensibility, {pos_label})")
    print(f"Specificity:  {spec:.5f}   (recall of {neg_label})")
    print(f"F1-score:     {f1:.5f}")

    # 3) Confusion matrix
    labels_order = [neg_label, pos_label]
    cm = confusion_matrix(y_val, y_pred, labels=labels_order)

    tn, fp, fn, tp = cm[0, 0], cm[0, 1], cm[1, 0], cm[1, 1]

    print("\nConfusion matrix (rows=true, cols=pred):")
    print(f"\t\t       Pred: {neg_label:>8} | {pos_label:>8}")
    print(f"\tTrue {neg_label:>8} | {tn:8d} | {fp:8d}")
    print(f"\tTrue {pos_label:>8} | {fn:8d} | {tp:8d}")

    # 4) Confusion matrix plot
    if show_plots:
        # Not normalized
        ConfusionMatrixDisplay.from_predictions(
            y_val,
            y_pred,
            display_labels=labels_order,
            cmap="Blues",
        )
        plt.title("Confusion Matrix (counts)")
        plt.show()

        # Normalized
        ConfusionMatrixDisplay.from_predictions(
            y_val,
            y_pred,
            display_labels=labels_order,
            cmap="Blues",
            normalize="true",
        )
        plt.title("Confusion Matrix (normalized)")
        plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np
import time
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt


def sweep_model_C(
    model_factory,
    X_train_vector,
    y_train,
    X_val_vector,
    y_val,
    pos_label="jailbreak",
    neg_label="benign",
    max_iter=200,
    C_values=None,
    model_name=None,
):
    """
    Sweep different values of the regularization parameter C for a given model.

    Parameters
    ----------
    model_factory : callable
        Function that takes a value of C and returns an untrained model
        instance (e.g., lambda C: LogisticRegression(C=C, ...)).
    X_train_vector : array-like
        Training feature matrix.
    y_train : array-like
        Training labels.
    X_val_vector : array-like
        Validation feature matrix.
    y_val : array-like
        Validation labels.
    pos_label : hashable, default="jailbreak"
        Label considered as the positive class.
    neg_label : hashable, default="benign"
        Label considered as the negative class (for specificity).
    max_iter : int, default=200
        Maximum number of iterations for the model (if applicable).
    C_values : iterable of float, optional
        Values of C (inverse of regularization strength) to sweep.
        If None, defaults to np.linspace(0.1, 1.0, 19).
    model_name : str, optional
        Name of the model for plot titles. If None, it is inferred from the
        first model created by model_factory.

    Returns
    -------
    dict
        Dictionary with lists of metrics for each C:
        {
            "C": list of C values,
            "accuracy": [...],
            "precision": [...],
            "recall": [...],
            "specificity": [...],
            "f1": [...],
            "training_time": [...]
        }
    """

    if C_values is None:
        C_values = np.linspace(0.1, 1.0, 19)

    list_c = list(C_values)

    list_accuracy = []
    list_precision = []
    list_recall = []
    list_specificity = []
    list_f1 = []
    list_training_time = []

    # Infer model name if not provided
    if model_name is None:
        tmp_model = model_factory(list_c[0])
        model_name = tmp_model.__class__.__name__

    for c in list_c:
        clf = model_factory(c)

        t_start = time.time()

        # Train on the training set
        clf.fit(X_train_vector, y_train)

        t_stop = time.time()

        # Predict on the validation set
        y_pred = clf.predict(X_val_vector)

        # Metrics computed on the validation set
        list_accuracy.append(accuracy_score(y_val, y_pred))
        list_precision.append(
            precision_score(y_val, y_pred, pos_label=pos_label)
        )
        list_recall.append(
            recall_score(y_val, y_pred, pos_label=pos_label)
        )
        list_f1.append(
            f1_score(y_val, y_pred, pos_label=pos_label)
        )
        # Specificity = recall on the negative class
        list_specificity.append(
            recall_score(y_val, y_pred, pos_label=neg_label)
        )
        list_training_time.append(t_stop - t_start)

    # Plot results
    fig, ax = plt.subplots(3, 2, figsize=(12, 8))

    ax[0][0].plot(list_c, list_accuracy, c="tab:blue")
    ax[0][0].set_xlim([min(list_c), max(list_c) * 1.05])
    ax[0][0].set_title(f"Accuracy ({model_name})")
    ax[0][0].set_ylabel("Accuracy (Validation)")
    ax[0][0].set_xlabel("C (inverse regularization)")

    ax[0][1].plot(list_c, list_precision, c="tab:orange")
    ax[0][1].set_xlim([min(list_c), max(list_c) * 1.05])
    ax[0][1].set_title(f"Precision ({model_name})")
    ax[0][1].set_ylabel("Precision (Validation)")
    ax[0][1].set_xlabel("C (inverse regularization)")

    ax[1][0].plot(list_c, list_recall, c="tab:green")
    ax[1][0].set_xlim([min(list_c), max(list_c) * 1.05])
    ax[1][0].set_title(f"Recall ({model_name})")
    ax[1][0].set_ylabel("Recall (Validation)")
    ax[1][0].set_xlabel("C (inverse regularization)")

    ax[1][1].plot(list_c, list_specificity, c="tab:olive")
    ax[1][1].set_xlim([min(list_c), max(list_c) * 1.05])
    ax[1][1].set_title(f"Specificity ({model_name})")
    ax[1][1].set_ylabel("Specificity (Validation)")
    ax[1][1].set_xlabel("C (inverse regularization)")

    ax[2][0].plot(list_c, list_f1, c="tab:cyan")
    ax[2][0].set_xlim([min(list_c), max(list_c) * 1.05])
    ax[2][0].set_title(f"F1 ({model_name})")
    ax[2][0].set_ylabel("F1 (Validation)")
    ax[2][0].set_xlabel("C (inverse regularization)")

    ax[2][1].plot(list_c, list_training_time, c="tab:red")
    ax[2][1].set_xlim([min(list_c), max(list_c) * 1.05])
    ax[2][1].set_title(f"Training time ({model_name})")
    ax[2][1].set_ylabel("Training time [s]")
    ax[2][1].set_xlabel("C (inverse regularization)")

    plt.tight_layout()
    plt.show()

    return {
        "C": list_c,
        "accuracy": list_accuracy,
        "precision": list_precision,
        "recall": list_recall,
        "specificity": list_specificity,
        "f1": list_f1,
        "training_time": list_training_time,
    }


we are gonna use the following plots:

**Confusion matrix (counts)**
Shows, in absolute numbers, how many benign and jailbreak prompts were correctly classified (diagonal cells)
and how many were misclassified (off-diagonal) by Multinomial Naive Bayes on the validation set.

**Confusion matrix (normalized)**
Same matrix but row-normalized, so each row sums to 1 and the diagonal values directly show the recall for benign and for jailbreak.

**Accuracy vs alpha**
Shows how overall validation accuracy changes as you vary the Naive Bayes smoothing parameter alpha.

**Precision vs alpha**
Shows how the precision for the positive class (jailbreak) changes with alpha, i.e. how many predicted jailbreaks are actually jailbreaks.

**Recall vs alpha**
Shows how often true jailbreaks are correctly detected as jailbreaks for different values of alpha.

**Specificity vs alpha**
Shows recall of the negative class (benign) as alpha varies, i.e. how often benign prompts are correctly classified as benign.

**F1 vs alpha**
Shows the F1-score (harmonic mean of precision and recall) for jailbreak across different alpha values, summarizing the trade-off between missing and falsely flagging jailbreaks.

**Training time vs alpha**
Shows the training time of Naive Bayes for each alpha; usually fairly flat, it just confirms that sweeping alpha is computationally cheap.

#### Naive Bayes

First step is to import the class of the desired classifier and create and object with an instance of that class.

[Here](https://scikit-learn.org/stable/modules/naive_bayes.html) you can take a look of the different verisions of Naive Bayes implemented in Scikit-Learn.
We will be working with *Multinomial Naive Bayes*, [here](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB) find the documentation of the classifier.

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()

We train the instance of the classifier on the training data using the `fit`method.

In [ ]:
clf.fit(X_train_vector, y_train)

We compute the predictions of the trained classifier on the validation data using the `fit`method.

In [ ]:
y_pred = clf.predict(X_val_vector)

In [ ]:
y_pred

Finally we can use Scikit-Learn built-in fucntions to evaluate the model starting from the target labels and the predictions.
We will compute the same metrics as before: accuracy, precision and recall.

In [ ]:
evaluate_classifier(clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")


In [ ]:
from sklearn.naive_bayes import MultinomialNB
import numpy as np

nb_factory = lambda alpha: MultinomialNB(alpha=alpha)

alpha_values = np.linspace(0.1, 1.0, 19)

results_nb = sweep_model_C(
    model_factory=nb_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
    C_values=alpha_values,          # C_values = alpha_values
    model_name="MultinomialNB",
)


Unfortunately the model doesn't work with negative numbers,so we can't test it on the other word embeddings

#### Logistic Regression

Let's try now with a different model, logistic regression.

Let's import the model and create an instance!

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

Now we can train the instance

In [ ]:
clf.fit(X_train_vector, y_train)

and check its predictions and performances

In [ ]:
clf = LogisticRegression()
clf.fit(X_train_vector, y_train)

evaluate_classifier(clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")

#The normalized matrix shows about 90% recall for benign and 81% recall for jailbreak, so the model is strong on both classes,
#slightly better at recognizing benign prompts while still catching most jailbreaks with relatively few false alarms.

You can play with the hyperparameters of the model, looking for the best configuration.

The complete list of parameters can be found on the sklearn [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', max_iter=500)
clf.fit(X_train_vector, y_train)

How do performances change with the new parameters?

In [ ]:


evaluate_classifier(clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")

#The normalized matrix shows about 89% recall for benign and 82% recall for jailbreak, very similar balance to before but with higher overall accuracy,
#precision, and F1. So tuning the hyperparameters gives a small but consistent performance gain without changing the error pattern much.


#####continue

In [ ]:
list_c = np.linspace(0.1, 1.0, 19)
list_c

At each iteration we are going to append the newest result in these lists, at the end we will plot all the results.

Iterate over the values of `C` and fit each time a new classifier on the training set, then register the results on the test set.

Plot the results.

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg_factory = lambda C: LogisticRegression(
    max_iter=200,
    penalty="l2",
    C=C,
    solver="liblinear",
)

results_logreg = sweep_model_C(
    model_factory=logreg_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
)

#Here C is the inverse regularization for logistic regression on the BoW features:

#Accuracy – Gently increases with C, peaking around 0.86–0.865, so a bit less regularization helps.
#Precision – Also rises with C, meaning higher C reduces false positives on jailbreak.
#Recall – Gradually improves as C grows, so the model catches slightly more jailbreaks too.
#Specificity – Already high and creeps up to ~0.90, so benign prompts are very well preserved across C.
#F1 – Follows precision/recall and steadily increases, with the best values at the largest C tried.
#Training time – Stays low overall, with some small bumps as C increases, but nothing problematic computationally.


#####different word embeddings:

In [ ]:
logreg_models_default = {}  # per salvare i modelli logreg (default) per ogni embedding

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 LogisticRegression (parametri default) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # 1) standard model
    clf = LogisticRegression(
        max_iter=1000,
        n_jobs=-1
    )
    clf.fit(X_train_vec, y_train)

    logreg_models_default[emb_name] = clf

    # 2) evaluation
    evaluate_classifier(
        clf,
        X_val_vec,
        y_val,
    )

#For w2v, logistic regression reaches about 0.72 accuracy with moderate precision and recall on jailbreak; it misses many jailbreaks (109) and also mislabels 100 benign prompts.

#With glove_twitter, accuracy and specificity improve slightly, but recall on jailbreak drops a bit, so the model is a bit more conservative and prefers to call things benign.

#With glove_wiki, performance is very close to glove_twitter, with a small gain in recall and F1; overall behaviour is almost the same as Twitter GloVe.

#With fasttext, there is a clear improvement: accuracy, precision, recall and F1 all increase, and missed jailbreaks fall to 76 while benign errors stay similar, giving a better balance.

#With sbert, results are the best: highest accuracy (0.83), higher precision and recall on jailbreak, and better specificity on benign, meaning it detects more jailbreaks while making fewer false alarms on benign prompts.

In [ ]:
C_values = list_c

results_logreg_sweep = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Sweep C per LogisticRegression su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # Factory for LogisticRegression
    logreg_factory = lambda C: LogisticRegression(
        C=C,
        max_iter=1000,
        n_jobs=-1
    )

    results_lr = sweep_model_C(
        model_factory=logreg_factory,
        X_train_vector=X_train_vec,
        y_train=y_train,
        X_val_vector=X_val_vec,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        C_values=C_values,
        model_name=f"LogisticRegression_{emb_name}"
    )

    results_logreg_sweep[emb_name] = results_lr

#For w2v, changing C only slightly moves the curves: metrics stay relatively low and fairly flat, so tuning regularization cannot fix the weaker embedding.

#For glove_twitter, the scores are a bit higher than w2v, but again the curves are smooth with only small bumps; a wide range of C values gives similar mid-level performance.

#For glove_wiki, behaviour is very similar to glove_twitter: modest improvements over w2v, no sharp optimum in C, and only mild variation in accuracy, recall and F1.

#For fasttext, all the metric curves sit clearly above the previous ones; there is a broad region of C where accuracy,
#recall and F1 are high and stable, so FastText plus logistic regression works well without needing very precise tuning.

#For sbert, the curves are highest and quite flat across C, showing that SBERT embeddings give the best performance and that logistic regression remains robust even if C is not perfectly optimized.

#### Support Vector Machine

Let's try now with a different type of classifier, a support vector machine.
As before, we import the corresponding class and we create an instance.

In [ ]:
from sklearn.svm import SVC

clf = SVC(C=0.5)

Once we have the instance of our SVM we can fit it on the training data.

**NOTE**: we must limit to a subset of the words otherwise the algorithm won't fit the model in a reasonable amount of time.

In [ ]:
clf.fit(X_train_vector, y_train)

And we can evaluate it on the validation data.



In [ ]:


evaluate_classifier(clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")

#This model shows solid, fairly balanced performance:
#It correctly classifies most benign prompts (377) and mislabels 73 as jailbreak.
#For jailbreak, it correctly detects 243 and misses 57.
#The normalized matrix shows about 84% recall for benign and 81% recall for jailbreak, so the classifier is slightly better
#at recognizing benign prompts but still identifies the majority of jailbreak attempts with relatively few false alarms.


In [ ]:
clf = SVC(C=0.1)
clf.fit(X_train_vector, y_train)

As usual we compute the predictions on the validation set and we use them to evaluate the model.


In [ ]:


evaluate_classifier(clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")

#Accuracy is about 0.74 with F1 around 0.68, so overall performance is moderate.
#For benign prompts, 348 are correctly classified while 102 are wrongly flagged as jailbreak (about 77% recall on benign).
#For jailbreak, 206 are correctly detected but 94 are missed (about 69% recall), so almost one third of jailbreaks are still predicted as benign.
#The normalized matrix highlights this imbalance: the model is noticeably less reliable on jailbreak than on benign,
#and generates more false negatives than in the better-performing setting.


In [ ]:
clf = SVC(C=0.9)
clf.fit(X_train_vector, y_train)

As usual we compute the predictions on the validation set and we use them to evaluate the model.



In [ ]:


evaluate_classifier(clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")

#Here the SVM with C = 0.9 is performing quite well:
#Accuracy is about 0.84 and F1 around 0.80, so overall performance is strong.
#For benign prompts, 388 are correctly classified and 62 are wrongly flagged as jailbreak (about 86% recall on benign).
#For jailbreak prompts, 241 are correctly detected and 59 are missed (about 80% recall), which is better balance than in the previous SVM runs.
#The normalized confusion matrix shows both classes are handled fairly symmetrically, with only a modest fraction of false positives and false negatives.

In [ ]:
from sklearn.svm import SVC

svc_factory = lambda C: SVC(
    C=C,
    kernel="linear",
    probability=False
)


results_svc = sweep_model_C(
    model_factory=svc_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
)

#This sweep shows how the linear SVC behaves as C changes:

#Accuracy gently fluctuates around 0.85 and peaks for moderate C (around 0.25–0.35); larger C values do not bring clear gains.
#Precision also reaches its highest point in the same C range, then slightly decreases as C increases, meaning more false positives for high C.
#Recall is quite stable, with a small improvement at medium C and then a plateau, so sensitivity to jailbreaks doesn’t change much.
#Specificity slowly decreases with larger C, so the model becomes a bit worse at correctly recognising benign prompts when C is high.
#F1 follows the same pattern as accuracy and precision, suggesting that moderate regularisation (C ≈ 0.25–0.35) gives the best overall balance.
#Training time is low but somewhat erratic across C, with a few spikes; there is no strong reason to choose a very large C from a computational point of view either.



How did the model performed on the training data?
We can compute the predictions on the training set too.


In [ ]:
y_pred = clf.predict(X_train_vector)

Then we compute the metrics on the new predictions using the target labels of the training set.


In [ ]:
y_pred_train = clf.predict(X_train_vector)

print(f"Accuracy (train):  {accuracy_score(y_train, y_pred_train):.5f}")
print(f"Precision (train): {precision_score(y_train, y_pred_train, pos_label='jailbreak'):.5f}")
print(f"Recall (train):    {recall_score(y_train, y_pred_train, pos_label='jailbreak'):.5f}")


# Specificity = recall of the negative class ('benign')
specificity = recall_score(y_train, y_pred, pos_label='benign')
print(f"Specificity: {specificity:.5f}")

print(f"F1-score:  {f1_score(y_train, y_pred, pos_label='jailbreak'):.5f}")


- if the performance is poor both on the training set and the test set, it is a case of **UNDERFITTING**.
- if the performance is poor on the test set but good on the training set, it is a case of **OVERFITTING**. (Basically, the model is not able to generalize).

In [ ]:
from sklearn.svm import SVC

svc_models_default = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 SVC (parametri default) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # 1) standard SVC
    clf = SVC(
        C=0.5,
        kernel="rbf",
        probability=True,
        random_state=42
    )
    clf.fit(X_train_vec, y_train)

    svc_models_default[emb_name] = clf

    # 2) evaluation
    evaluate_classifier(
        clf,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )

    #For SVC with w2v embeddings, performance is reasonably strong: around 0.81 accuracy and F1≈0.77. Recall for jailbreaks is slightly
#higher than precision, so the model catches most jailbreaks but still mislabels some benign prompts as malicious.

#With glove_twitter, metrics are very similar but a bit worse than w2v: slightly lower accuracy, precision, recall, and F1.
#It still behaves in a balanced way, but w2v has a small edge.

#With glove_wiki, accuracy and F1 drop further. Recall on jailbreaks falls to 0.72, even if specificity stays good,
#meaning the model is more conservative and misses more jailbreaks than the previous two.

#FastText improves over all the GloVe variants and w2v: accuracy rises to about 0.83 and F1 to ~0.79. Recall on jailbreaks (0.83)
#is high while keeping similar specificity, so it is better at detecting attacks without sacrificing too many benign examples.

#SBERT again is the best representation: accuracy reaches about 0.88 with the highest precision, recall, and F1 (~0.85). The confusion matrix shows
#few mistakes on both classes, with high recall for jailbreaks and very strong specificity for benign prompts.


In [ ]:
# C_values viene da list_c, come prima
C_values = list_c

results_svc_sweep = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Sweep C per SVC su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # Factory for SVC
    svc_factory = lambda C: SVC(
        C=C,
        kernel="rbf",
        probability=True,
        random_state=42
    )

    results_svc = sweep_model_C(
        model_factory=svc_factory,
        X_train_vector=X_train_vec,
        y_train=y_train,
        X_val_vector=X_val_vec,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        C_values=C_values,
        model_name=f"SVC_{emb_name}"
    )

    results_svc_sweep[emb_name] = results_svc

    #For the SVC sweeps, each block explores how the regularization parameter C affects performance for a different embedding (w2v, GloVe Twitter, GloVe Wiki, FastText, SBERT).

#Across embeddings, the curves show that changing C slightly shifts the trade-off between precision, recall, and specificity, but there is no dramatic instability:
#performance usually stays in a fairly narrow band. Medium C values tend to give a good balance, while very small C underfits and very large C bring only marginal gains or small drops.
#As with the default runs, the SBERT and FastText sweeps reach the highest overall metric levels,
#confirming that these semantic embeddings are the most effective feature representations for the SVC classifier in this task.


# Clustering

Let's try to see if we can discover different meaninful structures of the data by using clustering alogorithms.
We will:
- Apply different clustering algorithms.
- Visualize clusters after dimensionality reduction (PCA, UMAP, t-SNE).
- Evaluate the quality of the clusters with internal and external metrics.
- Explore how the choice of **number of clusters** or **eps** might affects the metrics.


### Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import random
from scipy.stats import uniform, randint

Visualisation

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns

Clustering

In [ ]:
from sklearn.cluster import (
    KMeans,
    AgglomerativeClustering,
    DBSCAN
)

Clustering metrics

In [ ]:
from sklearn.metrics import (
    homogeneity_score,
    completeness_score,
    v_measure_score,
    silhouette_score,
    calinski_harabasz_score,
)

### k-means Clustering

Now we apply the actual clustering and observe the results

#### K-Means




Fit the clustering algorithm

In [ ]:
# Register start time
t_start = time.time()

kmeans = KMeans(n_clusters=2, random_state=random_seed)

kmeans.fit(X_train_vector)
labels_kmeans = kmeans.labels_


# Register end time
t_stop = time.time()

In [ ]:
print(f"Elapsed time: {t_stop - t_start:.5f}")

We can print the centers of the clusters (which are, in this case, two high-dimensional points)

In [ ]:
kmeans.cluster_centers_

We can see the cluster labels.
These labels represent the ID of the clusters found with K-Means

In [ ]:
kmeans.labels_

In [ ]:
len(kmeans.labels_)

We can store these labels in a new column of the data frame

In [ ]:
import pandas as pd

text_train_strings = df.loc[idx_train, "text"]

train_df = pd.DataFrame({
    "text": text_train_strings,
    "true_label": y_train,
    "cluster": kmeans.labels_
})

train_df.head()


#### Clustering results visualisation

##### Scatter plot with PCA

In [ ]:
from sklearn.decomposition import PCA

# PCA requires a dense matrix
X_train_dense = X_train_vector.toarray()

pca = PCA(n_components=2, random_state=42)
X_train_2D = pca.fit_transform(X_train_dense)

print("Shape after PCA:", X_train_2D.shape)


In [ ]:
colors = ['red', 'blue']  #a different color for each cluster
cluster_colors = [colors[c] for c in train_df["cluster"]]


In [ ]:
#plot
markers = {'benign': 'o', 'jailbreak': 'x'}

plt.figure(figsize=(8, 8))
for t in ['benign', 'jailbreak']:
    idx = [i for i, lbl in enumerate(y_train) if lbl == t]
    plt.scatter(
        X_train_2D[idx, 0],
        X_train_2D[idx, 1],
        s=20,
        alpha=0.7,
        c=[cluster_colors[i] for i in idx],
        marker=markers[t],
        label=t
    )

plt.legend()
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("Clusters as Colors, True Labels as Markers")
plt.show()


the clusters don't look great with PCA, let's try a different technique to reduce dimensions: UMAP

In [ ]:
!pip install umap-learn

import umap.umap_ as umap


In [ ]:
import umap.umap_ as umap

umap_2d = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

X_train_2D = umap_2d.fit_transform(X_train_vector)


In [ ]:
#we will use the same graphical conventions
markers = {'benign': 'o', 'jailbreak': 'x'}

plt.figure(figsize=(8, 8))
for t in ['benign', 'jailbreak']:
    idx = [i for i, lbl in enumerate(y_train) if lbl == t]
    plt.scatter(
        X_train_2D[idx, 0],
        X_train_2D[idx, 1],
        s=20,
        alpha=0.7,
        c=[cluster_colors[i] for i in idx],
        marker=markers[t],
        label=t
    )

plt.legend()
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.title("Clusters as Colors, True Labels as Markers")
plt.show()


here the 2 clusters are more distinct, but they don't seem to be related to the "jailbreak" or "benign" label at all!

perhaps our data representation is just too high-dimensional to be understood with a simple graph in the Cartesian Plane

##### t-SNE

A possible solution to the high-dimensionality problem is **t-SNE** (t-distributed Stochastic Neighbor Embedding) and algorithm for dimensionality reduction.
We can use it to better visualize the clusters.

Import the `TSNE` object

In [ ]:
from sklearn.manifold import TSNE

Fit the embedding model on the data and transform them

In [ ]:
X_dense = X_train_vector.toarray()

X_embedded = TSNE(
    n_components=2,
    perplexity=20,
    random_state=42
).fit_transform(X_dense)

print(X_embedded.shape)  # (n_samples, 2)


Now look at the shape of the transformed data

In [ ]:
X_embedded.shape

Now we can try visualising the results of the clustering process using the downprojected data

In [ ]:
import matplotlib.pyplot as plt

# colors for K-Means clusters
colors = ['red', 'blue']

fig, ax = plt.subplots(1, 2, figsize=(18, 9))

# ---- Plot 1: K-Means clustering ----
ax[0].set_title("K-Means clustering (k = 2)")
ax[0].scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    s=5,
    alpha=0.7,
    c=[colors[idx] for idx in train_df['cluster']]
)
ax[0].set_xlabel("t-SNE dim 1")
ax[0].set_ylabel("t-SNE dim 2")

# ---- Plot 2: True labels (benign vs jailbreak) ----
ax[1].set_title("Benign vs. jailbreak samples")
ax[1].scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    s=5,
    alpha=0.7,
    c=['g' if lbl == 'benign' else 'r' for lbl in train_df['true_label']]
)
ax[1].set_xlabel("t-SNE dim 1")
ax[1].set_ylabel("t-SNE dim 2")

plt.show()


let's look at some of the points in the red cluster:

In [ ]:
# Filter only the samples that belong to cluster 0 (red cluster in the t-SNE plot)
cluster_red_df = train_df[train_df["cluster"] == 0]

print(f"Number of samples in the red cluster (cluster 0): {len(cluster_red_df)}\n")

# Select up to 20 examples from this cluster
# You can use .head(20) to take the first ones, or .sample(...) to pick them randomly
subset = cluster_red_df.head(20)   # or: cluster_red_df.sample(n=min(20, len(cluster_red_df)), random_state=42)

# Loop over the selected samples and print some useful information
for i, row in subset.iterrows():
    print("=" * 80)
    print(f"Index:      {i}")                 # original index in the training set
    print(f"Cluster:    {row['cluster']}")    # should be 0 (red cluster)
    print(f"True label: {row['true_label']}") # 'benign' or 'jailbreak'
    print("Text:")
    print(row["text"])                        # the original text associated with this sample
    print()


**very interesting!** it seems that using a "multiple choice quiz" approach is a good way of jailbreaking

#### Evaluation

now we will evaluate the clustering using some commonly used metrics

In [ ]:
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score

# Metrics
homogeneity = homogeneity_score(train_df['true_label'], train_df['cluster'])
completeness = completeness_score(train_df['true_label'], train_df['cluster'])
vmeasure = v_measure_score(train_df['true_label'], train_df['cluster'])

print("Homogeneity:", homogeneity)
print("Completeness:", completeness)
print("V-measure:", vmeasure)

In [ ]:
from sklearn.metrics import (
    homogeneity_score,
    completeness_score,
    v_measure_score,
    silhouette_score,
    calinski_harabasz_score
)

# Metrics: homogeneity, completeness, v-measure
homogeneity = homogeneity_score(train_df['true_label'], train_df['cluster'])
completeness = completeness_score(train_df['true_label'], train_df['cluster'])
vmeasure = v_measure_score(train_df['true_label'], train_df['cluster'])

print("Homogeneity:", homogeneity)
print("Completeness:", completeness)
print("V-measure:", vmeasure)

# -------------------------------------------------------------------
# Extra metrics: Silhouette & Calinski-Harabasz
# -------------------------------------------------------------------

print(f"Silhouette score      = {silhouette_score(X_train_vector, train_df['cluster']):.4f}")
print(f"Calinski-Harabasz     = {calinski_harabasz_score(X_dense, train_df['cluster']):.4f}")


**Elbow method** with the **homogeneity score**

We iterate over different values of the `n_clusters` parameter to collect the corresponding homogeneity score values, we plot these value pairs and we look for the elbow.

In [ ]:
n_clusters = list(range(2, 50))

Accumulator for the scores

In [ ]:
homogeneity_values = []

Iteratively compute the scores for the different `n_clusters` values

In [ ]:
for k in n_clusters:
    # Register start time
    t_start = time.time()

    # Fit K-Means on the vectorized data
    kmeans = KMeans(
        n_clusters=k,
        random_state=random_seed
    ).fit(X_train_vector)          # <-- usa X_train_vector, non df

    # Compute homogeneity between true labels and clusters
    homogeneity_values.append(
        homogeneity_score(y_train, kmeans.labels_)
    )

    # Register end time
    t_stop = time.time()

    # Print elapsed time for this k
    print(f"Elapsed time: {t_stop - t_start:.5f} seconds (k = {k})")

Plot the collected scores as function of the `n_clusters` hyperparameter

In [ ]:
plt.figure(figsize=(6,4))
plt.plot(n_clusters, homogeneity_values, marker='o')
plt.xlabel("Number of clusters (k)")
plt.ylabel("Homogeneity")
plt.title("Homogeneity vs k for K-Means on text vectors")
plt.grid(True)
plt.show()


**Elbow method** with the **Calinski-Harabasz index score**

We iterate over different values of the `n_clusters` parameter to collect the corresponding C-H index score values, we plot these value pairs and we look for the elbow.

In [ ]:
n_clusters = list(range(2, 10))

Accumulator for the scores

In [ ]:
ch_values = []

Iteratively compute the scores for the different `n_clusters` values

In [ ]:
from sklearn.metrics import calinski_harabasz_score
from sklearn.cluster import KMeans
import time

# Convert sparse → dense
#X_dense = X_train_vector.toarray()



for k in n_clusters:
    t_start = time.time()

    # Fit KMeans su vettori densi
    kmeans = KMeans(
        n_clusters=k,
        random_state=random_seed
    ).fit(X_dense)

    # Calinski-Harabasz score
    ch_values.append(calinski_harabasz_score(X_dense, kmeans.labels_))

    t_stop = time.time()
    print(f"Elapsed time: {t_stop - t_start:.5f} seconds (k = {k})")


Plot the collected scores as function of the `n_clusters` hyperparameter

In [ ]:
plt.plot(n_clusters, ch_values, marker='o')
plt.xlabel("Number of clusters (k)")
plt.ylabel("Calinski-Harabasz score")
plt.title("CH score vs k for K-Means")
plt.grid(True)
plt.show()


In [ ]:
# Best k according to CH
best_k = n_clusters[int(np.argmax(ch_values))]
print("➡️ Best k according to CH:", best_k)

In [ ]:
# ============================
# 2) Final KMeans with best_k
# ============================
kmeans_best = KMeans(
    n_clusters=best_k,
    random_state=random_seed
).fit(X_dense)

df_plot = train_df.copy()
df_plot["cluster"] = kmeans_best.labels_

In [ ]:
# ============================
# 3) PCA 2D visualization
# ============================
pca = PCA(n_components=2, random_state=42)
X_train_2D_pca = pca.fit_transform(X_dense)

print("Shape after PCA:", X_train_2D_pca.shape)

# One color per cluster
# (make sure length of 'colors' is >= best_k)
colors = ['red', 'blue', 'green', 'purple', 'orange']
cluster_colors = [colors[c] for c in df_plot["cluster"]]

# Markers for true labels
markers = {'benign': 'o', 'jailbreak': 'x'}

plt.figure(figsize=(8, 8))
for t in ['benign', 'jailbreak']:
    # indices where true label == t
    idx = [i for i, lbl in enumerate(y_train) if lbl == t]
    plt.scatter(
        X_train_2D_pca[idx, 0],
        X_train_2D_pca[idx, 1],
        s=20,
        alpha=0.7,
        c=[cluster_colors[i] for i in idx],
        marker=markers[t],
        label=t
    )

plt.legend()
plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title(f"PCA – K-Means (k={best_k}), Clusters as Colors, True Labels as Markers")
plt.show()

In [ ]:
# ============================
# 4) UMAP 2D visualization
# ============================
umap_2d = umap.UMAP(
    n_components=2,
    n_neighbors=15,
    min_dist=0.1,
    metric='cosine',
    random_state=42
)

# UMAP can work directly with the sparse matrix
X_train_2D_umap = umap_2d.fit_transform(X_train_vector)

markers = {'benign': 'o', 'jailbreak': 'x'}
# Reuse same color mapping
cluster_colors = [colors[c] for c in df_plot["cluster"]]

plt.figure(figsize=(8, 8))
for t in ['benign', 'jailbreak']:
    idx = [i for i, lbl in enumerate(y_train) if lbl == t]
    plt.scatter(
        X_train_2D_umap[idx, 0],
        X_train_2D_umap[idx, 1],
        s=20,
        alpha=0.7,
        c=[cluster_colors[i] for i in idx],
        marker=markers[t],
        label=t
    )

plt.legend()
plt.xlabel("UMAP Component 1")
plt.ylabel("UMAP Component 2")
plt.title(f"UMAP – K-Means (k={best_k}), Clusters as Colors, True Labels as Markers")
plt.show()


In [ ]:
# ============================
# 5) t-SNE 2D visualization
# ============================
tsne = TSNE(
    n_components=2,
    perplexity=20,
    random_state=42
)

X_embedded = tsne.fit_transform(X_dense)
print("Shape after t-SNE:", X_embedded.shape)

colors_tsne = ['red', 'blue', 'green', 'purple', 'orange']

fig, ax = plt.subplots(1, 2, figsize=(18, 9))

# ---- Plot 1: K-Means clustering in t-SNE space ----
ax[0].set_title(f"K-Means clustering (k = {best_k}) – t-SNE")
ax[0].scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    s=5,
    alpha=0.7,
    c=[colors_tsne[idx] for idx in df_plot['cluster']]
)
ax[0].set_xlabel("t-SNE dim 1")
ax[0].set_ylabel("t-SNE dim 2")

# ---- Plot 2: True labels in t-SNE space ----
ax[1].set_title("Benign vs. jailbreak samples – t-SNE")
ax[1].scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    s=5,
    alpha=0.7,
    c=['g' if lbl == 'benign' else 'r' for lbl in df_plot['true_label']]
)
ax[1].set_xlabel("t-SNE dim 1")
ax[1].set_ylabel("t-SNE dim 2")

plt.show()

<div class="alert alert-block alert-danger">
<b>Q: Repeat the analysis with the elbow method using the silhouette score</b>
</div>

We iterate over different values of the `n_clusters` parameter to collect the corresponding silhouette score values, we plot these value pairs and we look for the elbow.

In [ ]:
n_clusters = list(range(2, 50))

Accumulator for the scores

In [ ]:
silhouette_values = []

Iteratively compute the scores for the different `n_clusters` values

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time




for k in n_clusters:
    # Register start time
    t_start = time.time()

    # Fit clustering algorithm on your vectorized text data
    kmeans = KMeans(
        n_clusters=k,
        random_state=random_seed
    ).fit(X_train_vector)

    # Compute Silhouette score for the current number of clusters
    score = silhouette_score(X_train_vector, kmeans.labels_)
    silhouette_values.append(score)

    # Register end time
    t_stop = time.time()

    # Print elapsed time
    print(f"Elapsed time: {t_stop - t_start:.5f} seconds (k: {k:2d}), Silhouette={score:.4f}")


Plot the collected scores as function of the `n_clusters` hyperparameter

In [ ]:
plt.figure()

plt.plot(n_clusters, silhouette_values, marker='o')  # ← QUI

plt.title('K-Means Silhouette score for varying number of clusters')
plt.xlabel('No. of clusters')
plt.ylabel('Silhouette score')
plt.grid()
plt.show()


let's plot with 24 and 25 clusters since the graph changes its trend there dramatically

In [ ]:
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import numpy as np

# Make sure X_dense and df_plot (with column 'true_label') already exist

# ============================
# 0) t-SNE 2D visualization (computed once)
# ============================
tsne = TSNE(
    n_components=2,
    perplexity=20,
    random_state=42
)

X_embedded = tsne.fit_transform(X_dense)
print("Shape after t-SNE:", X_embedded.shape)

# ============================
# 1) Loop over k = 24 and k = 25
# ============================
for k_clusters in [24, 25]:
    print(f"\n=== K-Means with k = {k_clusters} ===")

    kmeans_k = KMeans(
        n_clusters=k_clusters,
        random_state=42,
        n_init=10
    )
    cluster_labels_k = kmeans_k.fit_predict(X_dense)

    # Save (overwrite) cluster assignment for plotting
    df_plot["cluster"] = cluster_labels_k

    print(f"Cluster counts (k={k_clusters}):", np.bincount(cluster_labels_k))

    # Build a colormap with k distinct colors
    cmap = plt.cm.get_cmap("tab20", k_clusters)
    colors_tsne = [cmap(i) for i in range(k_clusters)]

    fig, ax = plt.subplots(1, 2, figsize=(18, 9))

    # ---- Plot 1: K-Means clustering in t-SNE space ----
    ax[0].set_title(f"K-Means clustering (k = {k_clusters}) – t-SNE")
    ax[0].scatter(
        X_embedded[:, 0],
        X_embedded[:, 1],
        s=5,
        alpha=0.7,
        c=[colors_tsne[idx] for idx in df_plot["cluster"]]
    )
    ax[0].set_xlabel("t-SNE dim 1")
    ax[0].set_ylabel("t-SNE dim 2")

    # ---- Plot 2: True labels in t-SNE space ----
    ax[1].set_title("Benign vs. jailbreak samples – t-SNE")
    ax[1].scatter(
        X_embedded[:, 0],
        X_embedded[:, 1],
        s=5,
        alpha=0.7,
        c=["g" if lbl == "benign" else "r" for lbl in df_plot["true_label"]]
    )
    ax[1].set_xlabel("t-SNE dim 1")
    ax[1].set_ylabel("t-SNE dim 2")

    plt.tight_layout()
    plt.show()


Unfortunately we didn't manage to identify any other meaningful clusters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    homogeneity_score,
    completeness_score,
    v_measure_score,
    silhouette_score,
    calinski_harabasz_score
)

def evaluate_clustering(
    X,
    labels_pred,
    y_true=None,
    compute_ch=True,
    name="Clustering Evaluation"
):
    """
    General evaluation function for clustering algorithms.

    Parameters
    ----------
    X : array-like or sparse matrix
        Feature matrix (vectorized text in your case).
    labels_pred : array-like
        Cluster labels predicted by the clustering algorithm.
    y_true : array-like or None
        Ground truth labels. If None, external metrics are skipped.
    compute_ch : bool
        Whether to compute Calinski-Harabasz score (requires dense array).
    name : str
        Title for printing results.
    """

    print(f"\n===== {name} =====")

    # -----------------------------------------------------------
    # External metrics (require ground truth)
    # -----------------------------------------------------------
    if y_true is not None:
        h = homogeneity_score(y_true, labels_pred)
        c = completeness_score(y_true, labels_pred)
        v = v_measure_score(y_true, labels_pred)

        print(f"Homogeneity:        {h:.5f}")
        print(f"Completeness:       {c:.5f}")
        print(f"V-measure:          {v:.5f}")
    else:
        print("Ground truth not provided: skipping homogeneity/completeness/v-measure.\n")


    # -----------------------------------------------------------
    # Internal metrics
    # -----------------------------------------------------------
    try:
        sil = silhouette_score(X, labels_pred)
        print(f"Silhouette score:   {sil:.5f}")
    except Exception as e:
        print(f"Silhouette score failed: {e}")

    if compute_ch:
        try:
            # convert to dense
            X_dense = X.toarray() if hasattr(X, "toarray") else X
            ch = calinski_harabasz_score(X_dense, labels_pred)
            print(f"Calinski-Harabasz: {ch:.5f}")
        except Exception as e:
            print(f"CH score failed: {e}")


let's define the function elbow_method, a general-purpose tool designed to evaluate clustering quality across a range of possible numbers of clusters

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    homogeneity_score,
    completeness_score,
    v_measure_score,
    silhouette_score,
    calinski_harabasz_score
)

def elbow_method(
    X,
    clusterer_factory,
    k_values,
    metric="silhouette",
    y_true=None,
    title=None,
    to_dense_for_ch=True,
    x_label="Number of clusters (k)"
):
    """
    Generic elbow-method helper for clustering.
    """

    scores = []

    # Precompute dense if necessary for CH
    if metric == "calinski_harabasz":
        if to_dense_for_ch and hasattr(X, "toarray"):
            X_for_ch = X.toarray()
        else:
            X_for_ch = X
    else:
        X_for_ch = None  # not used

    print(f"\n=== Elbow method using metric: {metric} ===")

    for k in k_values:
        t_start = time.time()

        # create and fit the clusterer
        clusterer = clusterer_factory(k)
        clusterer.fit(X)

        labels_pred = clusterer.labels_
        t_stop = time.time()

        # choose metric
        if metric == "silhouette":
            try:
                score = silhouette_score(X, labels_pred)
            except Exception as e:
                print(f"k={k}: silhouette failed ({e}), setting score=NaN")
                score = np.nan

        elif metric == "homogeneity":
            if y_true is None:
                raise ValueError("y_true is required for homogeneity.")
            score = homogeneity_score(y_true, labels_pred)

        elif metric == "completeness":
            if y_true is None:
                raise ValueError("y_true is required for completeness.")
            score = completeness_score(y_true, labels_pred)

        elif metric == "v_measure":
            if y_true is None:
                raise ValueError("y_true is required for v_measure.")
            score = v_measure_score(y_true, labels_pred)

        elif metric == "calinski_harabasz":
            try:
                score = calinski_harabasz_score(X_for_ch, labels_pred)
            except Exception as e:
                print(f"k={k}: CH failed ({e}), setting score=NaN")
                score = np.nan

        else:
            raise ValueError(f"Unknown metric: {metric}")

        scores.append(score)

        print(f"k = {k} | {metric} = {score:.5f} | elapsed = {t_stop - t_start:.4f} s")

    # ---- Plot ----
    if title is None:
        title = f"Elbow method ({metric})"

    plt.figure(figsize=(6,4))
    plt.plot(k_values, scores, marker='o')
    plt.xlabel(x_label)
    plt.ylabel(metric)
    plt.title(title)
    plt.grid(True)
    plt.show()

    return {
        "k_values": k_values,
        "scores": scores,
    }


let's evaluate the algorithm with 2 clusters

In [ ]:
kmeans = KMeans(n_clusters=2, random_state=42).fit(X_train_vector)

evaluate_clustering(
    X=X_train_vector,
    labels_pred=kmeans.labels_,
    y_true=y_train,
    name="K-Means (k=2)"
)


In [ ]:
k_values = list(range(2, 10))

kmeans_factory = lambda k: KMeans(
    n_clusters=k,
    random_state=random_seed
)

In [ ]:
results_hom = elbow_method(
    X=X_train_vector,
    clusterer_factory=kmeans_factory,
    k_values=k_values,
    metric="homogeneity",
    y_true=y_train,
    title="K-Means Homogeneity vs k"
)


In [ ]:


results_sil = elbow_method(
    X=X_train_vector,
    clusterer_factory=kmeans_factory,
    k_values=k_values,
    metric="silhouette",
    y_true=None,
    title="K-Means Silhouette vs k"
)

In [ ]:
results_ch = elbow_method(
    X=X_train_vector,
    clusterer_factory=kmeans_factory,
    k_values=k_values,
    metric="calinski_harabasz",
    y_true=None,
    title="K-Means Calinski–Harabasz vs k",
    to_dense_for_ch=True  # fa .toarray() dentro
)


2-3 seems to be a good number of clusters

###Agglomerative clustering

now we will evaluate a different algorithm using the same techniques

In [ ]:
from sklearn.cluster import AgglomerativeClustering

# PCA / Agglomerative / silhouette ecc. require dense array
X_train_dense = X_train_vector.toarray()


In [ ]:
agg = AgglomerativeClustering(
    n_clusters=2,
    linkage="ward"
).fit(X_train_dense)

evaluate_clustering(
    X=X_train_dense,
    labels_pred=agg.labels_,
    y_true=y_train,
    name="Agglomerative Clustering (k=2, ward)"
)


let's plot the evaluation metrics with different numbers of clusters

In [ ]:
k_values = list(range(2, 8))

agg_factory = lambda k: AgglomerativeClustering(
    n_clusters=k,
    linkage="ward"
)


In [ ]:
results_hom_agg = elbow_method(
    X=X_train_dense,               # DENSO
    clusterer_factory=agg_factory,
    k_values=k_values,
    metric="homogeneity",
    y_true=y_train,
    title="Agglomerative Homogeneity vs k"
)


In [ ]:
results_sil_agg = elbow_method(
    X=X_train_dense,
    clusterer_factory=agg_factory,
    k_values=k_values,
    metric="silhouette",
    y_true=None,
    title="Agglomerative Silhouette vs k"
)


In [ ]:
results_ch_agg = elbow_method(
    X=X_train_dense,
    clusterer_factory=agg_factory,
    k_values=k_values,
    metric="calinski_harabasz",
    y_true=None,
    title="Agglomerative Calinski–Harabasz vs k",
    to_dense_for_ch=False   # opzionale, è già denso
)


2-3 seems to be a good number of clusters

###DBScan

let's do the same with DBScan

In [ ]:
from sklearn.cluster import DBSCAN

#eps: 0.1, 0.2, ..., 2.0
eps_values = [(e + 1) / 10 for e in range(20)]

In [ ]:
# Factory for DBSCAN: k = eps
dbscan_factory = lambda eps: DBSCAN(
    eps=eps,
    min_samples=5,
    metric="cosine",
    n_jobs=-1
)

In [ ]:
dbscan_clust = dbscan_factory(0.5)
dbscan_clust.fit(X_train_vector)

evaluate_clustering(
    X=X_train_vector,
    labels_pred=dbscan_clust.labels_,
    y_true=y_train,
    name="DBSCAN (eps=0.5)"
)

let's see how the evaluation metrics change when eps changes

In [ ]:
results_dbscan_sil = elbow_method(
    X=X_train_vector,
    clusterer_factory=dbscan_factory,
    k_values=eps_values,
    metric="silhouette",
    y_true=None,
    title="DBSCAN Silhouette vs eps",
    x_label="eps"
)

In [ ]:
results_dbscan_hom = elbow_method(
    X=X_train_vector,
    clusterer_factory=dbscan_factory,
    k_values=eps_values,
    metric="homogeneity",
    y_true=y_train,
    title="DBSCAN Homogeneity vs eps",
    x_label="eps"
)

In [ ]:
results_dbscan_ch = elbow_method(
    X=X_train_vector,
    clusterer_factory=dbscan_factory,
    k_values=eps_values,              # qui eps_values = [0.1, 0.2, ..., 2.0]
    metric="calinski_harabasz",
    y_true=None,
    title="DBSCAN Calinski–Harabasz vs eps",
    to_dense_for_ch=True,             # fa .toarray() dentro
    x_label="eps"
)


lets try to fit DBScan with eps=0.3 and count how many clusters it generated

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

eps=0.3

# Fit DBSCAN
dbscan = DBSCAN(
    eps,
    min_samples=5,
    metric="cosine",
    n_jobs=-1
)

dbscan.fit(X_train_vector)

labels = dbscan.labels_

# Count clusters (ignore label -1 which means noise)
unique_labels = set(labels)
clusters = [lbl for lbl in unique_labels if lbl != -1]
n_clusters_generated = len(clusters)
n_noise = list(labels).count(-1)

print("Cluster labels:", unique_labels)
print(f"Number of clusters found (excluding noise): {n_clusters_generated}")
print(f"Number of noise points: {n_noise}")


let's plot

In [ ]:
# ============================
# 2) t-SNE 2D visualization
# ============================

# If X_train_vector is sparse, convert to dense
try:
    X_dense = X_train_vector.toarray()
except AttributeError:
    X_dense = X_train_vector  # already dense

tsne = TSNE(
    n_components=2,
    perplexity=20,
    random_state=42
)

X_embedded = tsne.fit_transform(X_dense)
print("Shape after t-SNE:", X_embedded.shape)

# ----------------------------
# Color mapping for DBSCAN:
# - each cluster gets a color
# - noise points (label = -1) are light gray
# ----------------------------
unique_labels_sorted = sorted(unique_labels)
cluster_labels = [lbl for lbl in unique_labels_sorted if lbl != -1]

n_clusters = len(cluster_labels)

# Use a matplotlib colormap for clusters
cmap = plt.cm.get_cmap("tab20", n_clusters)

cluster_colors = []
for lbl in labels:
    if lbl == -1:
        # noise points
        cluster_colors.append("lightgray")
    else:
        idx = cluster_labels.index(lbl)
        cluster_colors.append(cmap(idx))

fig, ax = plt.subplots(1, 2, figsize=(18, 9))

# ---- Plot 1: DBSCAN clustering in t-SNE space ----
ax[0].set_title(f"DBSCAN clustering (eps={eps}) – t-SNE")
ax[0].scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    s=5,
    alpha=0.7,
    c=cluster_colors
)
ax[0].set_xlabel("t-SNE dim 1")
ax[0].set_ylabel("t-SNE dim 2")

# ---- Plot 2: True labels (benign vs jailbreak) in t-SNE space ----
ax[1].set_title("Benign vs. jailbreak samples – t-SNE")
ax[1].scatter(
    X_embedded[:, 0],
    X_embedded[:, 1],
    s=5,
    alpha=0.7,
    c=['g' if lbl == 'benign' else 'r' for lbl in y_train]
)
ax[1].set_xlabel("t-SNE dim 1")
ax[1].set_ylabel("t-SNE dim 2")

plt.show()

we didn't manage to locate any new meaningful clusters

### final comparison

let's compare clusters made by different clustering algorithms by plotting them

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import numpy as np

X_for_agg = X_train_dense

agg = AgglomerativeClustering(
    n_clusters=2,
    linkage="ward"
)
agg_labels = agg.fit_predict(X_for_agg)

print("Agglomerative - cluster unici:", np.unique(agg_labels))

from sklearn.cluster import DBSCAN

dbscan = DBSCAN(
    eps=0.3,
    min_samples=5,
    metric="cosine",
    n_jobs=-1
)
db_labels = dbscan.fit_predict(X_train_vector)

print("DBSCAN - labels:", np.unique(db_labels))
print("DBSCAN - n° cluster (escluding noise):", len([c for c in set(db_labels) if c != -1]))
print("DBSCAN - n° noise:", list(db_labels).count(-1))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

markers = {'benign': 'o', 'jailbreak': 'x'}

def plot_clusters_2d(X_2D, labels, y_true, title):

    unique_clusters = np.unique(labels)
    cmap = plt.cm.get_cmap("tab10", len(unique_clusters))
    cluster2color = {c: cmap(i) for i, c in enumerate(unique_clusters)}

    plt.figure(figsize=(8, 8))

    for t in ['benign', 'jailbreak']:
        idx = [i for i, lbl in enumerate(y_true) if lbl == t]

        plt.scatter(
            X_2D[idx, 0],
            X_2D[idx, 1],
            s=20,
            alpha=0.7,
            c=[cluster2color[labels[i]] for i in idx],
            marker=markers[t],
            label=t
        )

    plt.legend()
    plt.xlabel("Component 1")
    plt.ylabel("Component 2")
    plt.title(title)
    plt.grid(True)
    plt.show()


In [ ]:
# Hierarchical
plot_clusters_2d(
    X_2D=X_train_2D,
    labels=agg_labels,
    y_true=y_train,
    title="Agglomerative clustering – clusters as colors, true labels as markers"
)

# DBSCAN
plot_clusters_2d(
    X_2D=X_train_2D,
    labels=db_labels,
    y_true=y_train,
    title="DBSCAN clustering – clusters as colors, true labels as markers"
)


### other embeddings

let's repeat the previous evaluation on the other embeddings we extracted from our dataset

In [ ]:
from sklearn.cluster import KMeans

for emb_name in embedding_names:
    if emb_name not in other_embeddings["train"]:
        print(f"⚠️ Embedding '{emb_name}' not present in other_embeddings['train'], skip.")
        continue

    X_emb = other_embeddings["train"][emb_name]

    print("\n" + "="*70)
    print(f"🔹 K-Means (k=2) on embedding: {emb_name}")
    print("="*70)

    kmeans = KMeans(n_clusters=2, random_state=random_seed).fit(X_emb)
    labels_pred = kmeans.labels_

    evaluate_clustering(
        X=X_emb,
        labels_pred=labels_pred,
        y_true=y_train,
        name=f"K-Means (k=2) on {emb_name}"
    )


In [ ]:
k_values = list(range(2, 10))

from sklearn.cluster import KMeans

def kmeans_factory(k):
    return KMeans(
        n_clusters=k,
        random_state=random_seed
    )

for emb_name in embedding_names:
    if emb_name not in other_embeddings["train"]:
        print(f"⚠️ Embedding '{emb_name}' not present, skip.")
        continue

    X_emb = other_embeddings["train"][emb_name]

    print("\n" + "#"*80)
    print(f"📈 Elbow K-Means su embedding: {emb_name}")
    print("#"*80)

    # Homogeneity
    _ = elbow_method(
        X=X_emb,
        clusterer_factory=kmeans_factory,
        k_values=k_values,
        metric="homogeneity",
        y_true=y_train,
        title=f"K-Means Homogeneity vs k ({emb_name})"
    )

    # Silhouette
    _ = elbow_method(
        X=X_emb,
        clusterer_factory=kmeans_factory,
        k_values=k_values,
        metric="silhouette",
        y_true=None,
        title=f"K-Means Silhouette vs k ({emb_name})"
    )

    # Calinski–Harabasz
    _ = elbow_method(
        X=X_emb,
        clusterer_factory=kmeans_factory,
        k_values=k_values,
        metric="calinski_harabasz",
        y_true=None,
        title=f"K-Means Calinski–Harabasz vs k ({emb_name})",
        to_dense_for_ch=True
    )


recap on the clustering algorithms metrics:

In [ ]:
from sklearn.cluster import AgglomerativeClustering

for emb_name in embedding_names:
    if emb_name not in other_embeddings["train"]:
        continue

    X_emb = other_embeddings["train"][emb_name]  # già denso (np.array)

    print("\n" + "="*70)
    print(f"🔹 Agglomerative (k=2, ward) on embedding: {emb_name}")
    print("="*70)

    agg = AgglomerativeClustering(
        n_clusters=2,
        linkage="ward"
    ).fit(X_emb)

    evaluate_clustering(
        X=X_emb,
        labels_pred=agg.labels_,
        y_true=y_train,
        name=f"Agglomerative (k=2, ward) su {emb_name}"
    )


In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np

eps = 0.3

for emb_name in embedding_names:
    if emb_name not in other_embeddings["train"]:
        continue

    X_emb = other_embeddings["train"][emb_name]

    print("\n" + "="*70)
    print(f"🔹 DBSCAN (eps={eps}) su embedding: {emb_name}")
    print("="*70)

    dbscan = DBSCAN(
        eps=eps,
        min_samples=5,
        metric="cosine",
        n_jobs=-1
    ).fit(X_emb)

    labels = dbscan.labels_
    unique_labels = set(labels)
    clusters = [lbl for lbl in unique_labels if lbl != -1]
    n_clusters_generated = len(clusters)
    n_noise = list(labels).count(-1)

    print("Cluster labels:", unique_labels)
    print(f"Number of clusters found (excluding noise): {n_clusters_generated}")
    print(f"Number of noise points: {n_noise}")

    evaluate_clustering(
        X=X_emb,
        labels_pred=labels,
        y_true=y_train,
        name=f"DBSCAN (eps={eps}) su {emb_name}"
    )


In [ ]:
from sklearn.decomposition import PCA
import umap.umap_ as umap

for emb_name in embedding_names:
    if emb_name not in other_embeddings["train"]:
        continue

    X_emb = other_embeddings["train"][emb_name]

    print("\n" + "="*70)
    print(f"🎨 PCA + UMAP + KMeans per embedding: {emb_name}")
    print("="*70)

    # KMeans
    kmeans = KMeans(n_clusters=2, random_state=random_seed).fit(X_emb)
    km_labels = kmeans.labels_

    # PCA 2D
    pca = PCA(n_components=2, random_state=random_seed)
    X_pca_2D = pca.fit_transform(X_emb)

    plot_clusters_2d(
        X_2D=X_pca_2D,
        labels=km_labels,
        y_true=y_train,
        title=f"PCA – KMeans clusters vs true labels ({emb_name})"
    )

    # UMAP 2D
    umap_2d = umap.UMAP(
        n_components=2,
        n_neighbors=15,
        min_dist=0.1,
        metric='cosine',
        random_state=random_seed
    )
    X_umap_2D = umap_2d.fit_transform(X_emb)

    plot_clusters_2d(
        X_2D=X_umap_2D,
        labels=km_labels,
        y_true=y_train,
        title=f"UMAP – KMeans clusters vs true labels ({emb_name})"
    )


# Anomaly detection

In [ ]:
# Many anomaly detection algorithms require dense matrices (NumPy arrays),
# so this helper converts sparse matrices (like from CountVectorizer) to dense.
def to_dense(X):
    return X.toarray() if hasattr(X, "toarray") else X


In [ ]:
import time
import numpy as np

def run_model(name, fn):
  """
    Utility to:
      - measure runtime of an anomaly detection model
      - run the model via a function `fn()`
      - print how long it took and how many points were labeled as each class

    `fn` must return an array of predictions (usually +1 for normal, -1 for anomaly).
    """
  t_start = time.time()
  y_pred = fn()
  t_stop = time.time()

  vals, cnts = np.unique(y_pred, return_counts=True)
  print(f"[{name:<16}] time={t_stop - t_start:6.2f}s  labels={dict(zip(vals, cnts))}")
  return y_pred


In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor

def anomaly_detection(
    X,
    rep_name,
    outlier_fraction=0.01,
    random_seed=42,
    do_tsne=False,
    light_mode=False
):
    print("\n" + "="*80)
    print(f"🔍 ANOMALY DETECTION on representation: {rep_name}")
    print("="*80)

    def to_dense_local(X_):
        return X_.toarray() if hasattr(X_, "toarray") else X_

    X_dense = to_dense_local(X)
    X_scaled = X_dense  # here you could add scaling/normalization if needed

    results = {}

    # ---------------------------------------------------------
    # 0) One-Class SVM
    # ---------------------------------------------------------
    results["svm"] = run_model(
        "OneClassSVM",
        lambda: OneClassSVM(
            kernel="rbf",
            gamma="scale",
            nu=outlier_fraction
        ).fit(X_scaled).predict(X_scaled)
    )

    # ---------------------------------------------------------
    # 1) Isolation Forest
    # ---------------------------------------------------------
    results["isolation"] = run_model(
        "IsolationForest",
        lambda: IsolationForest(
            contamination=outlier_fraction,
            random_state=random_seed,
            n_jobs=-1
        ).fit(X_scaled).predict(X_scaled)
    )

    # ---------------------------------------------------------
    # 2) Local Outlier Factor (LOF)
    # ---------------------------------------------------------
    results["lof"] = run_model(
        "LocalOutlierFactor",
        lambda: LocalOutlierFactor(
            n_neighbors=35,
            contamination=outlier_fraction,
            n_jobs=-1
        ).fit_predict(X_scaled)
    )

    # ---------------------------------------------------------
    # 3) t-SNE visualization
    # ---------------------------------------------------------
    if do_tsne:
        plot_tsne(X_scaled, results, rep_name)

    return results


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

colors = np.array(["r", "g"])  # -1 = red, +1 = green

def plot_tsne(X, preds, rep_name):
    print("Calcolo t-SNE...")
    X_2d = TSNE(n_components=2, perplexity=20, random_state=42).fit_transform(X)

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))
    methods = ["svm", "isolation", "lof"]
    titles  = ["One-Class SVM", "Isolation Forest", "Local Outlier Factor"]

    for i, (m, t) in enumerate(zip(methods, titles)):
        y = preds[m]
        ax[i].scatter(X_2d[:, 0], X_2d[:, 1],
                      s=5, alpha=0.5,
                      c=colors[(y + 1) // 2])
        ax[i].set_title(f"{rep_name} – {t}")
        ax[i].set_xticks([])
        ax[i].set_yticks([])

    plt.suptitle(f"t-SNE – {rep_name}")
    plt.show()


In [ ]:
results_bow = anomaly_detection(
    X_train_vector,
    rep_name="bow",
    outlier_fraction=0.01,
    random_seed=random_seed,
    do_tsne=False,
    #light_mode=True
)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1) Isolation Forest predictions
y_pred_is = results_bow["isolation"]

X_bow_dense = X_train_vector.toarray()


max_features = 50
X_bow_dense_small = X_bow_dense[:, :max_features]

fig, ax = plt.subplots(1, 2, figsize=(24, 8), sharey=True)

# Non-anomalies (label = 1)
ax[0].boxplot(X_bow_dense_small[y_pred_is == 1])
ax[0].set_title("predicted non-anomalies (Isolation Forest)")

# anomalies (label = -1)
ax[1].boxplot(X_bow_dense_small[y_pred_is == -1])
ax[1].set_title("predicted anomalies (Isolation Forest)")

feature_labels = [f"f{i}" for i in range(max_features)]

for idx in [0, 1]:
    ax[idx].set_xticklabels(feature_labels, rotation=90)
    ax[idx].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
y_pred_lof = results_bow["lof"]

fig, ax = plt.subplots(1, 2, figsize=(24, 8), sharey=True)

ax[0].boxplot(X_bow_dense_small[y_pred_lof == 1])
ax[0].set_title("predicted non-anomalies (LOF)")

ax[1].boxplot(X_bow_dense_small[y_pred_lof == -1])
ax[1].set_title("predicted anomalies (LOF)")

for idx in [0, 1]:
    ax[idx].set_xticklabels(feature_labels, rotation=90)
    ax[idx].grid(True)

plt.tight_layout()
plt.show()


In [ ]:
anomaly_results = {}

for emb_name in embedding_names:
    X_emb = other_embeddings["train"][emb_name]
    anomaly_results[emb_name] = anomaly_detection(
        X_emb,
        rep_name=emb_name,
        outlier_fraction=0.01,
        do_tsne=True
    )


no matter the word embedding, the anomalies don't seem to be describing useful additional patterns in the data

let's see if they can be used to improve performance on other models:

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# 1) Logistic Regression on FULL BoW
logreg_bow_full = LogisticRegression(
    max_iter=1000,
    solver="liblinear"
)

logreg_bow_full.fit(X_train_vector, y_train)

print("=== LogReg BoW – full training data (VALIDATION) ===")
results_logreg_full_val = evaluate_classifier(
    logreg_bow_full,
    X_val_vector,
    y_val
)


In [ ]:
# 2) Logistic Regression on CLEAN BoW (anomalies removed)

# Isolation Forest predictions on BoW features
y_pred_is = results_bow["isolation"]   # +1 = normal, -1 = anomaly

# Align y_train with X_train_vector indices
y_train_arr = y_train.reset_index(drop=True).to_numpy()

# Keep only inliers
mask_inliers = (y_pred_is == 1)

X_train_bow_clean = X_train_vector[mask_inliers]
y_train_clean     = y_train_arr[mask_inliers]

print(f"Train size (full):  {X_train_vector.shape[0]}")
print(f"Train size (clean): {X_train_bow_clean.shape[0]}  "
      f"(removed {X_train_vector.shape[0] - X_train_bow_clean.shape[0]} anomalies)")

logreg_bow_clean = LogisticRegression(
    max_iter=1000,
    solver="liblinear"
)

logreg_bow_clean.fit(X_train_bow_clean, y_train_clean)

print("=== LogReg BoW – anomalies removed (VALIDATION) ===")
results_logreg_clean_val = evaluate_classifier(
    logreg_bow_clean,
    X_val_vector,
    y_val
)

we can notice that removing the anomalies didn't affect much the performance of the classifier.

# Non linear classifiers

In the following section, we train several supervised machine learning models—including k-Nearest Neighbors (KNN), Decision Trees, and a Multi-Layer Perceptron (MLP) neural network—directly on these vectorized features. For each model, we evaluate performance using standard classification metrics such as accuracy, precision, recall, F1-score, and the confusion matrix. The goal is to compare how different algorithms behave on the same representation of the data and to identify which models are most effective at distinguishing jailbreak prompts from benign ones, without applying any hyperparameter search procedure.

### Imports

In [ ]:
import numpy as np
import pandas as pd
import os
import time
import random
from scipy.stats import uniform, randint


Visualisation

In [ ]:
%matplotlib inline
# Show plots inline in the notebook
# Plotting libraries
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns



Data preparation

In [ ]:
# Feature scaling (for models that are sensitive to feature magnitude)
from sklearn.preprocessing import StandardScaler

Classifiers

In [ ]:
from sklearn.neighbors import KNeighborsClassifier # k-Nearest Neighbors
from sklearn.tree import DecisionTreeClassifier, plot_tree # Decision Trees
from sklearn.neural_network import MLPClassifier # Multi-Layer Perceptron (Neural Network)

Evaluation metrics

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
    precision_recall_curve,
    average_precision_score,
    roc_curve,
    roc_auc_score
)
from sklearn.calibration import CalibrationDisplay

Model selection and cross validation

In [ ]:
# Hyperparameter search utilities
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

### Non-linear classifiers

#### k-Nearest Neighbors



We create our list of values to try and we fit and evaluate the model iteratively.

In [ ]:
list_h_param_values = [1, 2, 3, 4, 5]
list_h_param_values

In [ ]:
list_h_param_values = [1, 2, 3, 4, 5]

knn_factory = lambda C: KNeighborsClassifier(n_neighbors=int(C))

results_knn = sweep_model_C(
    model_factory=knn_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
    C_values=list_h_param_values,
    model_name="k-Nearest Neighbors"
)


Across all metrics, the hyperparameter sweep shows that KNN does not significantly benefit from changing k.
Some values (like k = 2 and k=4) provide better precision and specificity, meaning the classifier becomes more selective and avoids false alarms.
However, recall remains critically low for every k, preventing the model from detecting jailbreak prompts reliably.

The combined trends indicate that:

the model is systematically biased toward the benign class,

increasing or decreasing k does not correct this imbalance,

KNN cannot form meaningful neighborhoods for the jailbreak samples in this embedding space,

and no value of k produces a balanced or robust classifier.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_models_default = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 k-Nearest Neighbors (default, k=5) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # KNN with fixed k
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train_vec, y_train)

    knn_models_default[emb_name] = clf

    # evalaution on validation set
    evaluate_classifier(
        clf,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )


The experiment clearly shows that the embedding quality dominates KNN performance.
With classical embeddings (w2v, GloVe), KNN shows moderate accuracy and high recall but poor specificity. FastText improves recall dramatically but at the cost of excessive false alarms.

SBERT stands out by a large margin, offering:

- high recall similar to FastText,

- much higher precision and specificity,

- the best accuracy (0.833) and F1-score (0.814),

- and a more balanced confusion matrix.

Despite this, even with SBERT, KNN remains inferior to more advanced models like SVM and MLP in overall robustness and precision-recall balance.

In [ ]:
# possibile values of k
list_h_param_values = [1, 2, 3, 4, 5]

results_knn_sweep = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Sweep k per k-Nearest Neighbors su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # Factory for KNN, C used as "k"
    knn_factory = lambda C: KNeighborsClassifier(n_neighbors=int(C))

    results_knn = sweep_model_C(
        model_factory=knn_factory,
        X_train_vector=X_train_vec,
        y_train=y_train,
        X_val_vector=X_val_vec,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        C_values=list_h_param_values,
        model_name=f"k-Nearest Neighbors_{emb_name}"
    )

    results_knn_sweep[emb_name] = results_knn


Across all embeddings, the k-sweep shows that changing k slightly alters the balance between precision and recall, but does not dramatically change performance.
The dominant factor is the embedding representation, not the neighborhood size.

w2v, GloVe Twitter, and GloVe Wiki produce similar and limited performance.

FastText shows meaningful improvements, especially in recall.

SBERT is clearly the best embedding, providing consistent, stable, and significantly better results for every metric.

Across all experiments, KNN shows clear limitations as a non-linear classifier for jailbreak detection.
The model’s performance is dominated by the embedding space, and varying k does not fundamentally alter its behaviour. While FastText improves sensitivity and SBERT dramatically improves the overall balance between classes, KNN remains:

- unstable,

- heavily biased toward the benign class,

- inconsistent across embeddings,

- and incapable of achieving strong F1-scores except with SBERT.

Therefore, although SBERT + KNN yields acceptable performance, KNN in general is not a reliable classifier for this task. More advanced models (Decision Tree, SVM, MLP) are significantly more robust and accurate.

#### Decision Tree

You can find the documentation at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

We create our list of values to try and we fit and evaluate the model iteratively.

In [ ]:
list_h_param_values = [x + 1 for x in range(10)]
list_h_param_values

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_factory = lambda C: DecisionTreeClassifier(
    max_depth=int(C),
    random_state=42
)


results_tree = sweep_model_C(
    model_factory=tree_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
    C_values=list_h_param_values,
    model_name="Decision Tree"
)


The Decision Tree improves noticeably as depth increases from 1 to around 5–7, where it achieves its best balance between sensitivity and precision. Beyond that point, deeper trees degrade the ability to correctly classify benign prompts and begin to overfit without offering clear advantages.

Thus, intermediate-depth trees represent the optimal configuration, while very shallow and very deep trees are suboptimal.
This analysis confirms the classic behaviour of decision trees: too simple → underfit; too complex → overfit.

Visualise the tree (example)

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# 1) create model
tree_clf = DecisionTreeClassifier(
    max_depth=5,
    random_state=42
)

# 2) Fit the model
tree_clf.fit(X_train_vector, y_train)


In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# 1. train witn max_depth=5
clf_tree = DecisionTreeClassifier(max_depth=5, random_state=42)
clf_tree.fit(X_train_vector, y_train)

# 2. Visualizza tree
plt.figure(figsize=(20, 12))
plot_tree(
    clf_tree,
    filled=True,
    feature_names=vectorizer.get_feature_names_out(),   # nomi delle features
    class_names=['benign', 'jailbreak'],
    max_depth=5,  # limita la profondità del plot (opzionale ma utile)
    fontsize=8
)
plt.show()


now we repeat for the other embeddings

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dt_models_default = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 DecisionTreeClassifier (default, max_depth=5) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    clf = DecisionTreeClassifier(
        max_depth=5,
        random_state=42
    )
    clf.fit(X_train_vec, y_train)

    dt_models_default[emb_name] = clf

    # evaluation on validation set
    evaluate_classifier(
        clf,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )


Across embeddings, the tree with max_depth = 5 shows:

- moderate performance for classical embeddings (w2v, GloVe),

- high sensitivity but lower precision with FastText,

- the best and most stable performance with SBERT.

This confirms that embedding quality strongly influences Decision Tree performance, and that more semantically expressive embeddings — like SBERT — allow even simple models to achieve significantly better classification results.

In [ ]:
#max_depth possible values
list_max_depth = [2, 3, 4, 5, 6, 7, 8]

results_dt_sweep = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Sweep max_depth per DecisionTreeClassifier su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # factory for DecisionTree: C as max_depth
    dt_factory = lambda C: DecisionTreeClassifier(
        max_depth=int(C),
        random_state=42
    )

    results_dt = sweep_model_C(
        model_factory=dt_factory,
        X_train_vector=X_train_vec,
        y_train=y_train,
        X_val_vector=X_val_vec,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        C_values=list_max_depth,
        model_name=f"DecisionTree_{emb_name}"
    )

    results_dt_sweep[emb_name] = results_dt


Across all embeddings, the Decision Tree shows:

- moderate performance with w2v and both GloVe variants,

- higher sensitivity but reduced precision with FastText,

- the strongest and most stable results with SBERT.

These observations reinforce the notion that embedding quality strongly influences tree-based models.
SBERT remains the most effective representation, enabling the Decision Tree to form clearer, more discriminative partitions of the data, even under depth constraints.

Putting all results together, the Decision Tree:

- performs best at intermediate depths (5–7),

- suffers from underfitting when shallow,

- suffers from overfitting when too deep,

- reaches only moderate performance with classic embeddings (w2v/GloVe),

- becomes aggressive but imprecise with FastText,

- and achieves the strongest and most stable results with SBERT.

The Decision Tree is highly dependent on embedding quality. While tuning max_depth improves performance, only semantically expressive embeddings—especially SBERT—enable the model to reach its full potential. Without high-quality embeddings, the tree remains limited regardless of depth.

#### Kernel SVM
we repeat our analysis on this other algorithm

In [ ]:
from sklearn.svm import SVC

svm_models_default = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Kernel SVM (SVC RBF, default) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # modello SVM with kernel RBF
    clf = SVC(
        kernel="rbf",
        C=1.0,

    )
    clf.fit(X_train_vec, y_train)

    svm_models_default[emb_name] = clf

    # evaluation on validation set
    evaluate_classifier(
        clf,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )


Across all embeddings, several patterns emerge:

Embedding quality is the dominant factor.

Even with the same hyperparameters, performance varies dramatically depending on the embedding.

SBERT > FastText > (w2v ≈ GloVe)

- SBERT consistently provides the highest accuracy, precision, recall, and F1.

- FastText is a strong second, improving recall and overall robustness.

- w2v and GloVe provide similar but weaker results.


In [ ]:
#same list as before
C_values = list_c

results_svm_sweep = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Sweep C per Kernel SVM (RBF) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # factory for SVM RBF (parameter called C)
    svm_factory = lambda C: SVC(
        kernel="rbf",
        C=C
    )

    results_svm = sweep_model_C(
        model_factory=svm_factory,
        X_train_vector=X_train_vec,
        y_train=y_train,
        X_val_vector=X_val_vec,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        C_values=C_values,
        model_name=f"KernelSVM_RBF_{emb_name}"
    )

    results_svm_sweep[emb_name] = results_svm


Across all embeddings, the C-sweep shows that:

- The RBF SVM is relatively robust: performance does not collapse for “wrong” C; it mostly gently improves then saturates.

- For simpler embeddings (w2v, GloVe), tuning C only brings small incremental gains.

- For FastText and especially SBERT, increasing C leads to consistent, meaningful improvements in accuracy, F1, and specificity, with recall staying high.

- SBERT + RBF SVM with moderately large C is clearly the best configuration, combining high accuracy, high recall, and high precision, at the cost of slightly higher training time.

In other words:

Tuning C refines an already strong model; the real performance leap comes from using richer embeddings (FastText, SBERT), and C mainly helps these embeddings reach their full potential.

We create our list of values to try and we fit and evaluate the model iteratively.

In [ ]:
list_h_param_values = np.linspace(0.1, 3, 30)
list_h_param_values

In [ ]:
svc_factory = lambda C: SVC(C=C, kernel="linear")

import numpy as np
from sklearn.svm import SVC

list_h_param_values = np.linspace(0.1, 3, 30)

svc_factory = lambda C: SVC(
    C=C,
    kernel="linear",
)

results_svc = sweep_model_C(
    model_factory=svc_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
    C_values=list_h_param_values,
    model_name="Linear SVC"
)


The most important conclusion from the C-sweep is that the Linear SVM is extremely insensitive to the choice of C. All metrics remain nearly constant as C changes:

- No significant overfitting at high C

- No strong underfitting at low C

- Very small oscillations in performance

- Decision boundary is largely stable

This strongly indicates that:

The dataset is almost linearly separable
and the linear SVM finds a good separating hyperplane regardless of regularization strength

Kernel SVM emerges as a highly effective classifier for jailbreak detection, but its true performance is realized only when using expressive, semantically rich embeddings. SBERT stands out decisively, enabling the SVM to form clearer and more reliable decision boundaries than any other embedding. FastText serves as a strong intermediate option, outperforming classical embeddings but still falling short of SBERT in terms of precision–recall balance. Traditional embeddings like w2v and GloVe consistently yield moderate and quickly saturating performance, confirming that embedding quality—not C tuning—is the primary driver of model effectiveness

#### (Deep) Neural Network

we try to do the same with a neural network. We didn't choose the best parameters to make sure the model wouldn't be too hefty to train

We create our list of values to try and we fit and evaluate the model iteratively.

In [ ]:
list_h_param_values = np.linspace(1e-4, 1, 10)
list_h_param_values

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_factory = lambda C: MLPClassifier(
    hidden_layer_sizes=(50,),
    alpha=C,
    max_iter=200,
    random_state=42,
)

results_mlp = sweep_model_C(
    model_factory=mlp_factory,
    X_train_vector=X_train_vector,
    y_train=y_train,
    X_val_vector=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
    C_values=list_h_param_values,
    model_name="MLPClassifier (alpha sweep)"
)


The α-sweep shows that the MLPClassifier is a highly robust and consistently well-performing classifier for jailbreak detection.
Regularization has only a moderate impact on its metrics, confirming that the network forms a strong internal representation early and maintains it regardless of α.

- Accuracy, precision, recall, and F1 all remain high and stable.

- Specificity stays consistently strong, indicating reliable handling of benign prompts.

- Improvements or degradations due to α are minor compared to the impact of embeddings seen in other models.

In [ ]:
from sklearn.neural_network import MLPClassifier

mlp_models_default = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 MLPClassifier (default) su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # modello MLP "base"
    clf = MLPClassifier(
        hidden_layer_sizes=(50,),
        alpha=1e-3,
        max_iter=200,
        random_state=42
    )
    clf.fit(X_train_vec, y_train)

    mlp_models_default[emb_name] = clf

    # evaluation on validation set
    evaluate_classifier(
        clf,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )


Across all embeddings, the default MLPClassifier demonstrates robust and stable performance, but its effectiveness grows dramatically with embedding quality.

- With w2v and GloVe, the model performs moderately well but remains limited by the embeddings’ lack of semantic richness.

- FastText significantly improves both recall and precision, allowing the neural network to build more informative decision boundaries.

- SBERT achieves the best results across all metrics, confirming that deep semantic representations help even a shallow neural network reach high accuracy, strong class separation, and excellent F1-scores.
In conclusion, the deep neural network is a strong model whose performance is primarily driven by embedding quality. SBERT allows the MLP to operate at its full potential, making SBERT + MLP one of the best-performing combinations in the entire classifier suite—second only to SBERT + SVM

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier

#list of values to try
list_h_param_values = np.linspace(1e-4, 1, 10)

results_mlp_sweep = {}

for emb_name in embedding_names:
    print("\n" + "="*60)
    print(f"🔹 Sweep alpha per MLPClassifier su embedding: {emb_name}")
    print("="*60)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    # factory for MLP: C as alpha
    mlp_factory = lambda C: MLPClassifier(
        hidden_layer_sizes=(50,),
        alpha=C,
        max_iter=200,
        random_state=42
    )

    results_mlp = sweep_model_C(
        model_factory=mlp_factory,
        X_train_vector=X_train_vec,
        y_train=y_train,
        X_val_vector=X_val_vec,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        C_values=list_h_param_values,
        model_name=f"MLPClassifier_alpha_{emb_name}"
    )

    results_mlp_sweep[emb_name] = results_mlp


Putting all embeddings together:

- SBERT still delivers the highest absolute performance for almost every α; its best results are obtained with low regularization (small α).

- FastText is the second-best option and actually benefits from relatively large α, combining high recall with improved specificity and lower training time.

- GloVe Wiki is clearly better than GloVe Twitter, and both gain something from moderate α but never reach SBERT/FastText levels.

- w2v is the least sensitive to α: its metrics change very little, confirming that regularization cannot compensate for its limited representational power

The deep neural network is a strong and stable classifier, but its true effectiveness is determined by the richness of the embedding.
SBERT enables the MLP to function at its full potential, producing results that are competitive with the best-performing models in the entire study—second only to the combination SBERT + RBF SVM.
FastText provides a solid alternative, while w2v and GloVe embeddings limit the model by constraining the expressive power of its learned representations.

In summary, the MLPClassifier excels when paired with semantically meaningful embeddings, confirming that embedding quality is the key driver of neural network performance in jailbreak detection.

Across all models and embeddings tested, SBERT consistently delivers the highest semantic quality, enabling every classifier to perform at significantly higher levels.
When coupled with SBERT, both Kernel SVM (RBF) and Linear SVM achieve near-optimal performance, making them the best choices for jailbreak detection.

The MLPClassifier provides a strong alternative with similarly robust results and minimal sensitivity to hyperparameters.
Decision Trees provide interpretability but fall behind in accuracy and F1.
KNN, finally, shows clear limitations and is not competitive for this task

# XGBoost and Random Forest

Imports

In [ ]:
import os
import time
import random
from scipy.stats import uniform, randint

Visualisation

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
from sklearn.metrics import PrecisionRecallDisplay, RocCurveDisplay, ConfusionMatrixDisplay

Data preparation

In [ ]:
# from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Classifiers

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

Evaluation metrics

In [ ]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay,
    precision_recall_curve,
    average_precision_score,
    roc_curve,
    roc_auc_score,
    classification_report
)
from sklearn.calibration import CalibrationDisplay

Model selection and cross validation

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

XGBoost requires numerical values for labels

embeddings

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score

le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_val_enc   = le.transform(y_val)

print("Classi:", le.classes_)

In [ ]:
# Dictionary to store results for each embedding
# For each embedding we will save the trained model and its F1-score
xgb_results = {}

# Loop over all available embedding names
for emb_name in embedding_names:

    # Skip embeddings that are not available in the dictionary
    if emb_name not in other_embeddings["train"]:
        print(f"Embedding {emb_name} not available, skipping.")
        continue

    print("\n" + "="*70)
    print(f"🚀 XGBoost with embedding: {emb_name}")
    print("="*70)

    # Retrieve training and validation embeddings
    X_train_emb = other_embeddings["train"][emb_name]
    X_val_emb   = other_embeddings["val"][emb_name]

    # Print shapes for sanity check
    print("Shapes:", X_train_emb.shape, X_val_emb.shape)

    # Initialize XGBoost classifier with fixed hyperparameters
    xgb_emb = XGBClassifier(
        n_estimators=100,
        max_depth=5,
        n_jobs=-1,
        eval_metric="logloss",
        random_state=42
    )

    # Train the model on the training embeddings
    xgb_emb.fit(X_train_emb, y_train_enc)

    # Predict labels on the validation set
    y_val_pred = xgb_emb.predict(X_val_emb)

    # Compute F1-score on the validation set
    f1 = f1_score(y_val_enc, y_val_pred, pos_label=1)

    print(f"F1-score (validation): {f1:.4f}")

    # Store model and performance for later comparison
    xgb_results[emb_name] = {
        "model": xgb_emb,
        "f1": f1
    }

The results show that Word2Vec embeddings achieve the best performance with XGBoost,
obtaining the highest F1-score on the validation set. GloVe Twitter and GloVe Wikipedia
perform slightly worse, while FastText, despite its higher dimensionality, does not
provide an improvement in F1-score. This suggests that simpler, domain-aligned
embeddings (Word2Vec) are more effective for this classification task than larger or
more general-purpose embeddings.

In [ ]:
# Select the embedding that achieved the highest F1-score on the validation set
best_emb_name = max(
    xgb_results,
    key=lambda k: xgb_results[k]["f1"]
)

# Retrieve the corresponding trained XGBoost model
best_xgb_emb = xgb_results[best_emb_name]["model"]

# Retrieve the corresponding F1-score
best_f1 = xgb_results[best_emb_name]["f1"]

# Print summary of the best embedding configuration
print("\nBEST XGBOOST EMBEDDING")
print("Embedding:", best_emb_name)
print("F1-score:", best_f1)

FastText outperforms the other embeddings, likely due to its ability to model subword information and rare tokens, which are common in jailbreak prompts.

In [ ]:
def evaluate_classifier(
    clf,
    X_val,
    y_val,
    pos_label=1,
    neg_label=0,
    show_plots=True
):
    """
    Evaluate a trained classifier on a validation set.

    Parameters
    ----------
    clf : trained classifier
        Model implementing predict() and optionally predict_proba() or decision_function()
    X_val : array-like
        Validation features
    y_val : array-like
        True labels (numeric: 0/1)
    pos_label : int
        Label of the positive class (default: 1)
    neg_label : int
        Label of the negative class (default: 0)
    show_plots : bool
        Whether to show confusion matrix, ROC and Precision-Recall curves
    """

    from sklearn.metrics import (
        accuracy_score,
        precision_score,
        recall_score,
        f1_score,
        ConfusionMatrixDisplay,
        RocCurveDisplay,
        PrecisionRecallDisplay
    )
    import matplotlib.pyplot as plt

    # Predictions
    y_pred = clf.predict(X_val)

    # Scores for ROC / PR curves
    y_score = None

    if hasattr(clf, "predict_proba"):
        proba = clf.predict_proba(X_val)
        pos_idx = list(clf.classes_).index(pos_label)
        y_score = proba[:, pos_idx]

    elif hasattr(clf, "decision_function"):
        y_score = clf.decision_function(X_val)

    # Metrics
    print("Accuracy :", accuracy_score(y_val, y_pred))
    print("Precision:", precision_score(y_val, y_pred, pos_label=pos_label, zero_division=0))
    print("Recall   :", recall_score(y_val, y_pred, pos_label=pos_label, zero_division=0))
    print("F1-score :", f1_score(y_val, y_pred, pos_label=pos_label, zero_division=0))

    # Plots
    if show_plots:
        fig, axes = plt.subplots(1, 3, figsize=(18, 4))

        # Confusion Matrix
        ConfusionMatrixDisplay.from_predictions(
            y_val,
            y_pred,
            display_labels=[neg_label, pos_label],
            ax=axes[0]
        )
        axes[0].set_title("Confusion Matrix")

        # ROC + Precision-Recall (only if scores are available)
        if y_score is not None:
            RocCurveDisplay.from_predictions(
                y_val,
                y_score,
                pos_label=pos_label,
                ax=axes[1]
            )
            axes[1].set_title("ROC Curve")

            PrecisionRecallDisplay.from_predictions(
                y_val,
                y_score,
                pos_label=pos_label,
                ax=axes[2]
            )
            axes[2].set_title("Precision-Recall Curve")
        else:
            axes[1].set_axis_off()
            axes[2].set_axis_off()

        plt.tight_layout()
        plt.show()

In [ ]:
X_val_best = other_embeddings["val"][best_emb_name]

evaluate_classifier(
    clf=best_xgb_emb,
    X_val=X_val_best,
    y_val=y_val_enc,
    pos_label=1,
    neg_label=0,
    show_plots=True,
)

The XGBoost model with FastText embeddings achieves strong performance on the validation set, with an F1-score of 0.80. The high ROC-AUC (0.91) and Average Precision (0.87) indicate good discriminative ability, while the confusion matrix shows a balanced trade-off between false positives and false negatives, making this model suitable for jailbreak detection.

now let's try some different parameters

In [ ]:
# List of values to test for the number of trees (n_estimators)
list_h_param_values = [10, 50, 100, 200]

# Factory function that creates an XGBoost classifier
# using the provided value as number of estimators
xgb_factory = lambda C: XGBClassifier(
    n_estimators=int(C),     # number of boosting trees
    max_depth=10,            # maximum depth of each tree
    n_jobs=6,                # parallel threads
    tree_method="hist",      # histogram-based algorithm (faster)
    eval_metric="logloss"    # evaluation metric for binary classification
)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def sweep_model_C(
    model_factory,
    X_train_vector,
    y_train,
    X_val_vector,
    y_val,
    pos_label,
    C_values,
    model_name="model sweep"
):
    results = {}

    for C in C_values:
        clf = model_factory(C)
        clf.fit(X_train_vector, y_train)

        y_pred = clf.predict(X_val_vector)

        results[C] = {
            "accuracy": accuracy_score(y_val, y_pred),
            "precision": precision_score(y_val, y_pred, pos_label=pos_label, zero_division=0),
            "recall": recall_score(y_val, y_pred, pos_label=pos_label, zero_division=0),
            "f1": f1_score(y_val, y_pred, pos_label=pos_label, zero_division=0),
        }

        print(f"\n[{model_name}] C={C}")
        print(" Accuracy :", results[C]['accuracy'])
        print(" Precision:", results[C]['precision'])
        print(" Recall   :", results[C]['recall'])
        print(" F1       :", results[C]['f1'])

    return results


In [ ]:
# This block performs a hyperparameter sweep over different data representations
# (Bag-of-Words and all available embeddings) using XGBoost.
# For each representation, we vary the number of estimators and evaluate
#performance on the validation set using F1-score for the positive class.
results_xgb_sweep = {}

for rep_name in all_representations:

    print("\n" + "="*70)
    print(f"🔹 sweep_model_C per XGBoost su rappresentazione: {rep_name}")
    print("="*70)

    # --- Recupero i vettori corretti ---
    if rep_name == "bow":
        X_tr = X_train_vector
        X_valr = X_val_vector
    else:
        X_tr = other_embeddings["train"][rep_name]
        X_valr = other_embeddings["val"][rep_name]

    # --- Sweep ---
    res = sweep_model_C(
        model_factory=xgb_factory,
        X_train_vector=X_tr,
        y_train=y_train_enc,
        X_val_vector=X_valr,
        y_val=y_val_enc,
        pos_label=1,
        C_values=list_h_param_values,
        model_name=f"XGBoost sweep ({rep_name})"
    )

    results_xgb_sweep[rep_name] = res


Bag-of-Words yields the best overall performance for XGBoost, suggesting that explicit lexical cues are more informative than dense semantic embeddings for jailbreak detection.

In [ ]:
all_representations = {}

In [ ]:
# Candidate values for the hyperparameter we want to sweep.
# Here we vary n_estimators (number of trees) to see how model capacity affects validation performance.
list_h_param_values = [10, 50, 100, 200]

# XGBoost expects numeric labels, so we convert string labels to binary:
#   "jailbreak" -> 1 (positive class), everything else -> 0 (benign)
y_train_xgb = (y_train == "jailbreak").astype(int)
y_test_xgb  = (y_test  == "jailbreak").astype(int)
y_val_xgb   = (y_val   == "jailbreak").astype(int)

# Sanity check: should print [0 1] if both classes are present in the training split
print(np.unique(y_train_xgb))


from xgboost import XGBClassifier

# Factory function: given a value (C), build an XGBoost model with that number of trees.
# NOTE: the variable name "C" is just a placeholder here; it actually controls n_estimators.
xgb_factory = lambda C: XGBClassifier(
    n_estimators=int(C),   # number of boosting rounds / trees
    max_depth=10,          # fixed depth for all sweeps (controls tree complexity)
    n_jobs=6,              # parallelism
    tree_method="hist",    # faster histogram-based algorithm (CPU-friendly)
    eval_metric="logloss"  # required by xgboost; we still evaluate with F1 in sweep_model_C
)

# Run a 1D hyperparameter sweep on the original BoW/CountVectorizer representation.
# For each n_estimators value, we train on X_train_vector and compute metrics on X_val_vector.
results_xgb = sweep_model_C(
    model_factory=xgb_factory,
    X_train_vector=X_train_vector,
    y_train=y_train_xgb,
    X_val_vector=X_val_vector,
    y_val=y_val_xgb,
    pos_label=1,                     # positivo = jailbreak
    C_values=list_h_param_values,
    model_name="XGBoost (n_estimators sweep)"
)


The sweep over the number of estimators shows a consistent performance across all configurations. Increasing the number of trees slightly improves the F1-score, with the best result achieved at n_estimators = 200. The improvements are marginal, indicating that the model is already stable with fewer trees and does not strongly benefit from further increases in complexity. This suggests a good bias–variance trade-off and no evident overfitting as the number of estimators grows.

In [ ]:
from sklearn.metrics import make_scorer, f1_score

f1_pos = make_scorer(f1_score, pos_label=1)

A way to measure the "importance" of each feature is given by some models. For instance:
- the `feature_importances_` attribute of XGBoost classifiers
- the `plot_importance` method of XGBoost
- the `feature_importances_` attribute of Random Forests
Let's try to work on the `plot_importance` from xgb



In [ ]:
import xgboost as xgb
import matplotlib.pyplot as plt
from xgboost import plot_importance

In [ ]:
import matplotlib.pyplot as plt
from xgboost import XGBClassifier, plot_importance

# 1) Train an XGBoost model directly on the BoW features
xgb_clf = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    n_jobs=6,
    tree_method="hist",
    eval_metric="logloss",
    random_state=42
)
xgb_clf.fit(X_train_vector, y_train_enc)

# 2) Retrieve feature names from the CountVectorizer
feature_names = vectorizer.get_feature_names_out().tolist()

# 3) Attach feature names to the booster for proper visualization
booster = xgb_clf.get_booster()
booster.feature_names = feature_names

# 4) Plot the top 20 most important features
fig, ax = plt.subplots(figsize=(12, 12))

plot_importance(
    booster,
    ax=ax,
    height=0.8,
    importance_type="weight",   # how often a feature is used for splits
    xlabel="Feature importance",
    show_values=False,
    max_num_features=20
)

plt.title("XGBoost Feature Importance (Top 20)")
plt.tight_layout()
plt.show()


In [ ]:
# Extract feature importance scores from the trained XGBoost model.
# The resulting dictionary maps each feature name to its importance
# based on how often it is used to split the data across all trees.
feature_importances_dict = xgb_clf.get_booster().get_fscore()
feature_importances_dict

Plot features importance

In [ ]:
def select_features(feature_importances, fraction=0.1, top=True):
    if not 0 < fraction <= 1:
        raise ValueError("fraction must be between 0 (exclusive) and 1 (inclusive)")

    values = np.array(list(feature_importances.values()))
    # For top x%, threshold should be (1 - fraction) quantile
    threshold = np.quantile(values, 1 - fraction if top else fraction)

    if top:
        return {k: v for k, v in feature_importances.items() if v >= threshold}
    else:
        return {k: v for k, v in feature_importances.items() if v <= threshold}


In [ ]:
# Select the bottom 5% least important features according to XGBoost
# These features contribute the least to the model decisions
irrelevant_columns = select_features(feature_importances_dict, 0.05, top=False)

# Display the least important features
irrelevant_columns

In [ ]:
list(irrelevant_columns.items())[:20]


In [ ]:
len(irrelevant_columns)


Analysis of the Least Important Features in XGBoost

The analysis of the bottom 5% least important features shows that they are mainly common, semantically neutral English words such as “about”, “always”, “answer”, “both”, and “business”. These terms appear frequently across both classes (benign and jailbreak) and do not carry meaningful discriminative information. As a result, XGBoost correctly assigns them minimal importance, since they do not contribute to separating normal user inputs from jailbreak attempts.

The fact that only a relatively small portion of features falls into this “irrelevant” category suggests that the vectorization process produces a reasonably clean and informative feature space. Overall, no anomalies are observed among the low-importance features: the model is effectively filtering out linguistic noise and focusing on the terms that provide real predictive value for jailbreak detection.

In [ ]:
relevant_columns = select_features(feature_importances_dict, 0.05, top=True)
relevant_columns

In [ ]:
relevant_columns = list(relevant_columns.keys())
relevant_columns# let's take a look at the distribution of values

let's try to remove all irrelevant features

In [ ]:
import numpy as np
from scipy.sparse import csr_matrix

# 1. Map irrelevant feature names → indices
feature_names = vectorizer.get_feature_names_out()
irrelevant_feature_indices = [
    np.where(feature_names == feat)[0][0]
    for feat in irrelevant_columns.keys()
]

# 2. Function to drop columns from CSR matrix
def drop_columns_csr(mat, cols_to_drop):
    mask = np.ones(mat.shape[1], dtype=bool)
    mask[cols_to_drop] = False
    return mat[:, mask]

# 3. Apply reduction
X_train_reduced = drop_columns_csr(X_train_vector, irrelevant_feature_indices)
X_test_reduced  = drop_columns_csr(X_test_vector, irrelevant_feature_indices)
X_val_reduced  = drop_columns_csr(X_val_vector, irrelevant_feature_indices)


Fit the model using the subset of features

In [ ]:
import time
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Encode labels to 0/1 for XGBoost (benign=0, jailbreak=1)
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)

In [ ]:
clf = XGBClassifier(
    n_estimators=200,
    max_depth=10,
    n_jobs=6,
    tree_method="hist",
    eval_metric="logloss",
    random_state=42
)

# Register start time
t_start = time.time()

# Train the classifier on the reduced feature set
clf.fit(X_train_reduced, y_train_enc)

# Register end time
t_stop = time.time()

print(f"Training time: {t_stop - t_start:.2f} seconds")

In [ ]:
y_val_enc = le.transform(y_val)

evaluate_classifier(
    clf=clf,
    X_val=X_val_reduced,
    y_val=y_val_enc,
    pos_label=1,
    neg_label=0,
    show_plots=True
)

The XGBoost model trained on the reduced feature set shows strong discriminative performance on the validation set. The high ROC–AUC (0.92) and Average Precision (0.89) indicate good class separability, while the confusion matrix highlights a balanced trade-off between precision (0.81) and recall (0.79) for the jailbreak class. Overall, the model effectively detects jailbreak prompts while maintaining a low false positive rate.

In [ ]:
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

# Encode string labels ("benign"/"jailbreak") into numeric labels (0/1) required by XGBoost
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)   # e.g., ['benign','jailbreak'] -> [0,1]
y_test_enc  = le.transform(y_test)        # same mapping applied to test labels

# Sanity check: prints the class order used by the encoder (defines which class is 0 and which is 1)
print(le.classes_)   # expected: ['benign' 'jailbreak']

# Define an XGBoost classifier (baseline configuration) to train on the reduced feature space
xgb_clf = XGBClassifier(
    n_estimators=10,        # number of trees (kept small here for a fast baseline)
    max_depth=10,           # maximum tree depth (controls model complexity)
    n_jobs=6,               # CPU parallelism
    tree_method="hist",     # faster histogram-based training
    eval_metric="logloss"   # evaluation metric for training
)

# Fit XGBoost on the reduced training matrix (features selected/reduced earlier)
xgb_clf.fit(X_train_reduced, y_train_enc)

# Evaluate the model on the reduced validation matrix using the numeric labels (pos_label=1 => "jailbreak")
evaluate_classifier(
    clf=xgb_clf,
    X_val=X_val_reduced,    # reduced validation features (must match X_train_reduced columns)
    y_val=y_val_enc,        # numeric validation labels produced by the same LabelEncoder
    pos_label=1,            # positive class index (jailbreak)
    neg_label=0,            # negative class index (benign)
    show_plots=True,        # show confusion matrix, ROC, and PR curves (if scores are available)
)

Model performance on Reduced Features:
the model trained on reduced features performs better and requires fewer trees to peak.

Peak: The best result is achieved at C=50 with an F1 score of 0.809 (Accuracy ~84.5%), which is higher than the best result of the full feature set.

Behavior: At C=10, it underperforms (F1 0.77), suggesting it needs a minimum capacity to grasp the patterns in the reduced space. However, after C=50, performance drops slightly, indicating potential overfitting or noise capture.

In [ ]:
# Define an F1 scorer focused on the positive class (jailbreak = 1)
# This is important because the task is asymmetric and jailbreak detection is the priority
f1_pos = make_scorer(f1_score, pos_label=1)

# Initialize a base XGBoost classifier
# No hyperparameters related to model capacity are fixed here,
# since they will be explored via GridSearch
xgb = XGBClassifier(
    tree_method="hist",     # histogram-based training for efficiency
    eval_metric="logloss",  # standard loss for binary classification
    n_jobs=6,               # parallel computation
    random_state=42         # reproducibility
)

# Define the hyperparameter grid to explore
# - n_estimators controls the number of trees
# - max_depth controls tree complexity
param_grid = {
    "n_estimators": [10, 50],
    "max_depth": [5, 10],
}

# Set up GridSearchCV to perform cross-validated hyperparameter tuning
# The model is selected based on F1-score on the positive class
grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring=f1_pos,   # optimize for jailbreak detection
    cv=3,             # 3-fold cross-validation
    n_jobs=-1         # use all available cores
)

# Fit the grid search on the reduced training feature space
# Labels are numeric (0 = benign, 1 = jailbreak)
grid.fit(X_train_reduced, y_train_enc)

# Retrieve the best-performing model according to cross-validation
best_xgb_reduced = grid.best_estimator_

# Print the best hyperparameter configuration found
print("Best params (reduced):", grid.best_params_)

In [ ]:
evaluate_classifier(
    clf=best_xgb_reduced,
    X_val=X_val_reduced,
    y_val=y_val_enc,
    pos_label=1,
    neg_label=0,
    show_plots=True,
)

In [ ]:
import numpy as np
from xgboost import XGBClassifier

# Hyperparameter values to sweep (number of trees)
list_h_param_values = [10, 50, 100, 200]

# Convert string labels to numeric labels (0/1)
# Required by XGBoost
#   1 = jailbreak
#   0 = benign
y_train_xgb = (y_train == "jailbreak").astype(int)
y_val_xgb   = (y_val == "jailbreak").astype(int)

# Sanity check: should print [0 1]
print(np.unique(y_train_xgb))

# Factory function that creates an XGBoost model
# using C as the number of estimators (trees)
xgb_factory = lambda C: XGBClassifier(
    n_estimators=int(C),   # number of trees
    max_depth=10,          # fixed tree depth
    n_jobs=6,              # parallel computation
    tree_method="hist",    # fast histogram-based training
    eval_metric="logloss"  # training loss
)

# Hyperparameter sweep on the reduced feature space
results_xgb = sweep_model_C(
    model_factory=xgb_factory,
    X_train_vector=X_train_reduced,  # reduced training features
    y_train=y_train_xgb,             # numeric training labels
    X_val_vector=X_val_reduced,      # reduced validation features
    y_val=y_val_xgb,                 # numeric validation labels
    pos_label=1,                     # positive class = jailbreak
    C_values=list_h_param_values,    # values to sweep
    model_name="XGBoost (n_estimators sweep)"
)


#### Random Forest

You can find the documentation at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

##### Model fitting and evaluation

Create classifier instance and fit it on the training data

In [ ]:
clf = RandomForestClassifier()

Fit classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# 1) Define a Random Forest classifier
# - n_estimators: number of trees in the forest
# - max_depth: maximum depth of each tree (controls model complexity)
# - n_jobs: use all available CPU cores
# - random_state: ensures reproducibility
rf_clf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    n_jobs=-1,
    random_state=42
)

# 2) Train the Random Forest on the vectorized training data (BoW / TF-IDF)
rf_clf.fit(X_train_vector, y_train)

# 3) Evaluate the model on the validation set
# Labels are kept as strings ("benign" / "jailbreak"),
# since RandomForest supports non-numeric labels directly
evaluate_classifier(
    clf=rf_clf,
    X_val=X_val_vector,
    y_val=y_val,
    pos_label="jailbreak",   # positive class
    neg_label="benign",      # negative class
    show_plots=True          # show confusion matrix, ROC, and PR curves
)

The Random Forest model achieves high precision but low recall on the jailbreak class. This indicates a conservative behavior: when the model predicts jailbreak it is usually correct, but it fails to identify a large portion of actual jailbreak prompts. Overall performance is therefore limited by the high number of false negatives, making the model less suitable when recall on jailbreaks is critical.

In [ ]:
from sklearn.metrics import make_scorer, f1_score

f1_jb = make_scorer(f1_score, pos_label="jailbreak")

##### Features importance

Similarly to XGBoost we can also visualise features importance of random forests.
You can access this information from the same `feature_importances_` attribute, that returns the importance of each feature. You can find the documentation at this [link](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier.feature_importances_).

**However, it is important not to trust blindly the values returned by such attribute**, as it only represents "the (normalized) total reduction of the criterion brought by that feature".

 for now fit with random parameters

In [ ]:
clf.fit(X_train_vector, y_train)

<div class="alert alert-block alert-danger">
<b>
    Q:
    Use the following line to create a dict that maps each column to its importance (as computed by the XGBoost model).
</b>
</div>

In [ ]:
feature_names = vectorizer.get_feature_names_out()
feature_importances_dict = dict(zip(feature_names, clf.feature_importances_))

feature_importances_dict


Plot features importance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sort features by importance score in descending order
sorted_features = sorted(
    feature_importances_dict.items(),
    key=lambda x: x[1],
    reverse=True
)

# Select the top 20 most important features
top_features = sorted_features[:20]

# Separate feature names and their importance values
names = [x[0] for x in top_features]
values = [x[1] for x in top_features]

# Create a horizontal bar plot
plt.figure(figsize=(10, 8))
plt.barh(names, values)

# Use a logarithmic scale to better visualize differences
plt.xscale("log")

# Invert y-axis so the most important feature appears at the top
plt.gca().invert_yaxis()


let's look at the irrelevant features

In [ ]:
irrelevant_columns = select_features(feature_importances_dict, fraction=0.05, top=False)
irrelevant_columns

let's look at the relevant ones

In [ ]:
relevant_columns = select_features(feature_importances_dict, 0.05, top=True)
relevant_columns


##### Explaining predictions

In addition to making predictions with your models, is very important to explain them, to make you models more intepretable.
Today, we are going to see LIME a tool for explainability that works with any model.

LIME stands for Local Interpretable Model-agnostic Explanations. LIME focuses on training local surrogate linear models to explain individual predictions. Local surrogate models are interpretable models that are used to explain individual predictions of black box machine learning models. Surrogate models are trained to approximate the predictions of the underlying black box model. Instead of training a global surrogate model, LIME focuses on training local surrogate models.

LIME is model-agnostic, meaning that it can be applied to any machine learning model. The technique attempts to understand the model by perturbing the input of data samples and understanding how the predictions change.

Why Lime?

The intuition behind LIME is very simple. First, forget the training data and imagine we have only the black box model where we supply the input data. The black box model generates the predictions for the model. We can enquire the box as many times as we like. Our objective is to understand why the machine learning model made a certain prediction.

Now, LIME comes into play. LIME tests what happens to the predictions when we provide variations in the data which is being fed into the machine learning model.

LIME generates a new dataset consisting of permuted samples and the corresponding predictions of the black box model. On this new dataset LIME then trains an interpretable model. It is weighted by the proximity of the sampled instances to the instance of interest. The learned model should be a good approximation of the machine learning model predictions locally, but it does not have to be a good global approximation. This kind of accuracy is also called local fidelity.

The user has to determine the complexity, e.g. by selecting the maximum number of features that the linear regression model may use.

The following example is based on the material avaialble at this [link](https://www.kaggle.com/code/prashant111/explain-your-model-predictions-with-lime)

Let's start by installing the library

In [ ]:
!pip install lime

An then let's import the tool

In [ ]:
import lime
import lime.lime_tabular

####LIME has one explainer for all the models

Select an instance from the test set and use it to explain the model:

In [ ]:
j = 42

In [ ]:
clf.predict(X_val_vector[j])

In [ ]:
y_val.iloc[j]

In [ ]:
from lime.lime_text import LimeTextExplainer

# Initialize a LIME text explainer for binary classification
# The class_names order must match the model's label encoding
explainer = LimeTextExplainer(
    class_names=["benign", "jailbreak"]
)

In [ ]:
j = 42  # index of the validation instance to explain

# Original text instance (raw text, not vectorized)
text_instance = df.loc[idx_val[j], "text"]

# Print the ground-truth label for this instance
print("True label:", y_val.iloc[j])

# Print a short snippet of the text for inspection
print("Text snippet:", text_instance[:200], "...\n")


In [ ]:
def rf_predict_proba_text(text_list):
    """
    text_list: list of raw text strings
    returns: numpy array of shape (n_samples, n_classes) with class probabilities
    """

    # 1. Vectorize the input texts using the SAME vectorizer used during training
    X_vec = vectorizer.transform(text_list)

    # 2. Use the already trained RandomForest model to predict class probabilities
    return clf.predict_proba(X_vec)


In [ ]:
j = 10  # index of the instance to explain

# Retrieve the raw text corresponding to the selected validation sample
text_j = df.loc[idx_val[j], "text"]

# Display true label and text preview
print("True label:", y_val.iloc[j])
print("Text snippet:", text_j[:200], "...\n")

# Generate a LIME explanation for the selected instance
exp = explainer.explain_instance(
    text_j,                 # raw text input
    rf_predict_proba_text,  # prediction function (with internal vectorization)
    num_features=6,         # number of most influential words to display
    labels=[1]              # class to explain (1 = jailbreak)
)

# Visualize the explanation directly in the notebook
exp.show_in_notebook()


Show the predictions

In [ ]:
exp.show_in_notebook()

In [ ]:
y_val.values[j]

In [ ]:
true_label = y_val.iloc[j]   # "benign" o "jailbreak"


lime_explanation_list = exp.as_list(label=1)
lime_explanation_list


We can also get a prediction on all the features

In [ ]:
from lime.lime_text import LimeTextExplainer

# Initialize the LIME text explainer with the class names
explainer = LimeTextExplainer(class_names=["benign", "jailbreak"])

# Wrapper function: converts raw text into vectors and returns class probabilities
def rf_predict_proba_text(text_list):
    # Vectorize the input texts using the same vectorizer used for training
    X_vec = vectorizer.transform(text_list)
    # Return probability predictions from the trained Random Forest
    return clf.predict_proba(X_vec)

# Index of the validation example to explain
j = 30

# IMPORTANT: use the original raw text, not vectorized features
text_j = df.loc[idx_val[j], "text"]

# Retrieve the index corresponding to the "jailbreak" class
class_index = list(clf.classes_).index("jailbreak")

# Generate a LIME explanation for the selected instance
exp = explainer.explain_instance(
    text_j,                 # raw input text
    rf_predict_proba_text,  # prediction function (model-agnostic)
    num_features=100,       # number of features shown in the explanation
    labels=[class_index]    # class to explain (jailbreak)
)

# Display the explanation inside the notebook
exp.show_in_notebook()


In [ ]:
lime_explanation_list = exp.as_list(label=class_index)
lime_explanation_list




How can we interpret the result?

# Large Language Models

we will try to do zero-shot and few-shot evaluation with a reasonably small pretrained LLM on our dataset

In [ ]:
train_df = df.loc[idx_train].copy()
val_df   = df.loc[idx_val].copy()
test_df  = df.loc[idx_test].copy()

In [ ]:
!pip install -q -U transformers bitsandbytes accelerate langchain_community #xformers
# !pip install -U "transformers" "accelerate" "huggingface_hub" sentencepiece


Now load a few libraries from Huggingface transformers:

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tqdm import tqdm
import matplotlib.pyplot as plt
import warnings
import gc

# from langchain_community.llms import HuggingFacePipeline
# from langchain_core.prompts import PromptTemplate

In [ ]:
# torch.random.manual_seed(random_seed)

In [ ]:
#load the dataset
import torch
import gc
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

train_df = df.loc[idx_train]
val_df   = df.loc[idx_val]
test_df  = df.loc[idx_test]

print("Dataset sizes:")
print(f" Train: {len(train_df)}")
print(f" Val:   {len(val_df)}")
print(f" Test:  {len(test_df)}")
print("\nLabel distribution (train):")
print(train_df['label'].value_counts())


In [ ]:
##!must activate GPU on Colab to run this part

from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "microsoft/Phi-3-mini-4k-instruct"

print(f"Loading model {model_name} ...")

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cuda",
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Model loaded successfully (Phi-3 Mini).")


In [ ]:

def generate_response(prompt, max_new_tokens=4, temperature=0.0):
    """
    Genera a response dall'LLM.
    """
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=2048
    )
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            do_sample=(temperature > 0),
            pad_token_id=tokenizer.eos_token_id
        )

    full_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = full_output[len(prompt):].strip()
    return response


In [ ]:
# let's extract some prompts and labels to use
import numpy as np

N_FEWSHOT_EXAMPLES = 2
RANDOM_SEED = random_seed
np.random.seed(RANDOM_SEED)

benign_examples = train_df[train_df["label"] == "benign"].sample(
    min(N_FEWSHOT_EXAMPLES, (train_df["label"] == "benign").sum()),
    random_state=RANDOM_SEED,
)

jailbreak_examples = train_df[train_df["label"] == "jailbreak"].sample(
    min(N_FEWSHOT_EXAMPLES, (train_df["label"] == "jailbreak").sum()),
    random_state=RANDOM_SEED,
)

print("Few-shot examples extracted:")
print(f" benign:    {len(benign_examples)}")
print(f" jailbreak: {len(jailbreak_examples)}")


In [ ]:
#zero_shot

def classify_prompt_zero_shot(prompt_text: str) -> str:
    """
    Classifica prompt come benign o jailbreak usando zero-shot prompting.
    """
    prompt = f"""
You are a security classifier. Classify the following prompt as either:
- benign = safe, harmless
- jailbreak = attempts to bypass safety or elicit harmful instructions.

Respond with exactly one word: benign or jailbreak.

Prompt:
\"\"\"{prompt_text[:800]}\"\"\"

Classification:
"""
    resp = generate_response(prompt).lower()

    if "jailbreak" in resp:
        return "jailbreak"
    if "benign" in resp:
        return "benign"
    return "benign"   # fallback


In [ ]:
#few-shot

def classify_prompt_few_shot(prompt_text: str, n_examples: int = 2) -> str:
    """
    Few-shot classification usando esempi reali del tuo training set.
    """

    n_ben = min(n_examples, len(benign_examples))
    n_jb  = min(n_examples, len(jailbreak_examples))

    prompt = "You are an expert security classifier.\n"
    prompt += "Classify each prompt as 'benign' or 'jailbreak'.\n\n"

    # jailbreak examples
    for i in range(n_jb):
        ex = str(jailbreak_examples.iloc[i]["text"])[:400]
        prompt += f"Prompt: {ex}\nClassification: jailbreak\n\n"

    # benign examples
    for i in range(n_ben):
        ex = str(benign_examples.iloc[i]["text"])[:400]
        prompt += f"Prompt: {ex}\nClassification: benign\n\n"

    # target prompt
    prompt += f"Prompt: {prompt_text[:800]}\nClassification: "

    resp = generate_response(prompt).lower()

    if "jailbreak" in resp:
        return "jailbreak"
    if "benign" in resp:
        return "benign"
    return "benign"


In [ ]:
#evaluation
def evaluate_llm_classifier(classifier_func, df_split, desc="Eval"):
    y_true = df_split["label"].tolist()
    y_pred = []

    for _, row in tqdm(df_split.iterrows(), total=len(df_split), desc=desc):
        y_pred.append(classifier_func(row["text"]))

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, pos_label="jailbreak")
    rec = recall_score(y_true, y_pred, pos_label="jailbreak")
    f1  = f1_score(y_true, y_pred, pos_label="jailbreak")

    print(f"\nResults: {desc}")
    print(f" Accuracy : {acc:.4f}")
    print(f" Precision: {prec:.4f}")
    print(f" Recall   : {rec:.4f}")
    print(f" F1-score : {f1:.4f}")

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "y_true": y_true,
        "y_pred": y_pred
    }


In [ ]:
print("=== ZERO-SHOT on VAL ===")
results_zero_val = evaluate_llm_classifier(
    classify_prompt_zero_shot,
    val_df,
    desc="LLM Zero-Shot (Validation)"
)


it isn't as good as the best models we have trained so far, but it's still decent, especially considering that it wasn't trained specifically for our classification task

In [ ]:
print("=== FEW-SHOT on VAL (2 examples per class) ===")
results_few_val = evaluate_llm_classifier(
    lambda txt: classify_prompt_few_shot(txt, n_examples=2),
    val_df,
    desc="LLM Few-Shot (Validation)"
)


Interestingly, the few-shot setting underperforms compared to zero-shot in this experiment.  
While few-shot prompting usually helps small LLMs by providing concrete examples, in our case it may actually hurt performance due to several factors:

1. **Small number of examples:**  
   We only provide two examples per class. If these examples are not fully representative, the model may overfit to their style or wording, reducing its ability to generalize.

2. **Phi-3 Mini’s limited context reasoning:**  
   Although Phi-3 Mini is strong for its size, its reasoning ability with many in-context examples is limited.  
   Adding extra demonstrations increases prompt complexity, which may confuse the model rather than guide it.

3. **Prompt length and structure:**  
   The few-shot prompt becomes longer and more verbose. Small LLMs sometimes degrade with long instructions or mixed-format demonstrations.

4. **Distribution shift between examples and test prompts:**  
   If the few-shot examples differ too much from the inputs in the validation/test set, the model might be biased toward incorrect patterns.

Because of these reasons, the zero-shot classifier ends up being cleaner and more stable, while the few-shot prompt introduces noise and biases that degrade performance.


In [ ]:
print("=== ZERO-SHOT on TEST ===")
results_zero_test = evaluate_llm_classifier(
    classify_prompt_zero_shot,
    test_df,
    desc="LLM Zero-Shot (Test)"
)

print("\n=== FEW-SHOT on TEST ===")
results_few_test = evaluate_llm_classifier(
    lambda txt: classify_prompt_few_shot(txt, n_examples=2),
    test_df,
    desc="LLM Few-Shot (Test)"
)


# Grid search

## Model and hyperparameters selection:

Now we will use grid search to optimize some of the parameters of our models automatically.
we will save our "best" models in the aformentioned dictionary

##SVC

### Cross-validation based hyperparameter selection using Grid Search

Grid Search is an exhaustive search over specified parameter values for an estimator.
Scikit-Learn implements it in `GridSearchCV`, [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) you can find the documentation.

In [ ]:
from sklearn.model_selection import GridSearchCV


We can try for example looking for the best regularisation parameter for a SVM on our data set.

Define the model

In [ ]:
from sklearn.svm import SVC
clf = SVC()

Define the search space of the hyper parameters of the selected model.

In [ ]:
param_grid = {'C': np.linspace(0.1, 5.0, 50), 'max_iter': [20000]}

In [ ]:
param_grid

Create the `GridSearchCV` object.
We are going to do a 3-fold cross-validation

In [ ]:
search = GridSearchCV(clf, param_grid, cv=3, verbose=True)

Fit the search estimator on the validation data to search for the best parameters

In [ ]:
search.fit(X_train_vector, y_train)

Display the results

In [ ]:
search.cv_results_

The grid search shows that increasing the regularization parameter C leads to a clear improvement in performance up to a plateau around C \approx 3.0. Beyond this point, the mean test score stabilizes around 0.848–0.849, indicating diminishing returns from weaker regularization. The standard deviation across folds remains low throughout, suggesting stable and robust performance. Training and scoring times increase moderately with higher C, but without critical overhead. Overall, the results indicate a good bias–variance trade-off around C \in [2.8, 3.2], which represents the optimal region for this model.

In [ ]:
results_df = pd.DataFrame(search.cv_results_)
results_df

The table reports the full cross-validation results of the grid search over the hyperparameter C for Logistic Regression.
The mean test score shows a gradual improvement as C increases, followed by a plateau, indicating limited sensitivity to further regularization changes.
The relatively low standard deviation across folds suggests stable performance and good generalization.


Display the best parameters

In [ ]:
search.best_params_

Retrieve the best estimator.

Once the search is complete, `GridSearchCV` fits the estimator on all available data

In [ ]:
best_clf = search.best_estimator_

Finally evaluate the best estimator in the validation data.

In [ ]:
y_pred = search.predict(X_val_vector)

In [ ]:


evaluate_classifier(best_clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")


The classifier shows solid overall performance (accuracy ≈ 86%), but it is clearly biased toward the benign class. While specificity is high (≈90%), recall on the jailbreak class is lower (≈81%), meaning that nearly 1 out of 5 jailbreak instances is missed. This trade-off may be problematic in safety-critical settings, where false negatives are more costly than false positives. The confusion matrix confirms this imbalance, suggesting that threshold tuning or recall-oriented optimization would be necessary if jailbreak detection is the primary goal.

#### other embeddings

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {
    'C': np.linspace(0.1, 5.0, 50),
    'max_iter': [20000],
}

for emb_name in embedding_names:
    print("\n" + "="*70)
    print(f"🔹 GridSearch SVC su embedding: {emb_name}")
    print("="*70)

    X_train_vec = other_embeddings["train"][emb_name]

    # Modello base
    base_clf = SVC()

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid,
        cv=3,
        verbose=True,
        n_jobs=-1
    )

    # Fit della grid search
    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params per {emb_name}: {best_params}")

    # Nome descrittivo della variante
    variant_name = "grid_C_0.1_5.0"

    # Salvataggio nel registry
    register_model(
        models_registry,
        model_name="SVC",
        embedding_name=emb_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


The GridSearch results show that the optimal regularization parameter C varies significantly across embeddings, confirming that the representation space strongly influences the margin–complexity trade-off of the SVM.
Lower-dimensional embeddings such as glove_wiki favor smaller C values, suggesting that stronger regularization helps prevent overfitting, while richer representations like fastText and GloVe Twitter benefit from higher C, allowing the model to exploit their higher expressive power.

Overall, the fact that all best C values fall well within the explored range [0.1, 5.0] indicates that the search space was appropriately chosen. However, the relatively close optimal values across embeddings also suggest limited sensitivity to fine-grained tuning of C, implying that embedding choice has a larger impact on performance than minor hyperparameter adjustments.

### Random Search

Differently from a grid search, a randomised one does not try all parameter values; instead a fixed number of parameter settings is sampled from the specified distributions.
we will also briefly experiment with this technique

In [ ]:
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV


We can look again for the best regularisation parameter for a SVM on our data set.

Define the model

In [ ]:
clf = SVC()

Define the search space of the hyper parameters of the selected model.
Here we have the real difference w.r.t. a Grid search, the parametrs are not provided as a list of values, but as a distribution to sample from. In this case we try a uniform distribution.

In [ ]:
param_grid = {'C': uniform(0, scale=5), 'max_iter': [20000]}

In [ ]:
uniform(0, scale=5)

Create the `RandomizedSearchCV` object.
We are going to do a 3-fold cross-validation

In [ ]:
search = RandomizedSearchCV(clf, param_grid, n_iter=50, cv=3, verbose=True)

Fit the search estimator on the train data to search for the best parameters

In [ ]:
search.fit(X_train_vector, y_train)

Display the results

In [ ]:
search.cv_results_

The GridSearch shows that performance is relatively stable across a wide range of C values, with only marginal differences in mean validation score. This suggests that the SVC model is not highly sensitive to the exact regularization strength in this range, and that further fine-grained tuning of C is unlikely to yield substantial performance gains.

In [ ]:
results_df = pd.DataFrame(search.cv_results_)
results_df

Display the best parameters

In [ ]:
search.best_params_

Randomized search yielded a very similar result to grid search

Retrieve the best estimator.

Once the search is complete, `RandomizedSearchCV` fits the estimator on all available data (unless differently instructed).

In [ ]:
best_clf = search.best_estimator_

Finally evaluate the best estimator in the validation data


In [ ]:
y_pred = best_clf.predict(X_val_vector)

In [ ]:


evaluate_classifier(best_clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")


The model shows solid overall performance (Accuracy ≈ 0.86, F1 ≈ 0.82) with good balance between precision and recall for the jailbreak class. However, around 20% of jailbreak samples are still missed (false negatives), which is critical in a safety-oriented setting. While specificity is high (~0.90), the remaining false negatives suggest that further optimization should prioritize recall for the positive class, even at the cost of a slight increase in false positives.

##Logistic Regression

###Cross-validation based hyperparameter selection using Grid Search

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression()

In [ ]:
param_grid = {'C': np.linspace(0.1, 5.0, 50), 'max_iter': [20000]}

In [ ]:
param_grid

In [ ]:
search = GridSearchCV(clf, param_grid, cv=3, verbose=True)

In [ ]:
search.fit(X_train_vector, y_train)

In [ ]:
search.cv_results_

The GridSearch shows that model performance is relatively stable across a wide range of C values, with only marginal differences in mean test score. This suggests that the SVM is not highly sensitive to the regularization strength in this interval, and that gains from further fine-grained tuning of C are limited. Additionally, higher C values do not consistently improve performance and sometimes increase training time, indicating diminishing returns and a potential risk of overfitting.

In [ ]:
results_df = pd.DataFrame(search.cv_results_)
results_df

In [ ]:
search.best_params_

In [ ]:
best_clf = search.best_estimator_

In [ ]:
# # Assumendo che il tuo models_registry esista già

# model_name = "LogisticRegression"
# embedding_name = "bow"   # cambialo se vuoi un nome diverso
# variant_name = f"gridsearch_C_{search.best_params_['C']:.3f}"
# model = best_clf
# params = search.best_params_

# register_model(
#     models_registry,
#     model_name=model_name,
#     embedding_name=embedding_name,
#     variant_name=variant_name,
#     model=model,
#     params=params
# )


###Random Search

In [ ]:
clf = LogisticRegression(max_iter=20000)

In [ ]:
param_grid = {'C': uniform(0, scale=5)}
uniform(0, scale=5)

In [ ]:
search = RandomizedSearchCV(
    clf,
    param_grid,
    n_iter=50,
    cv=3,
    verbose=True
)

In [ ]:
search.fit(X_train_vector, y_train)

In [ ]:
search.cv_results_

The randomized hyperparameter search shows that performance is relatively stable across a wide range of C values, with mean cross-validation accuracy consistently around 0.85. The best configurations differ only marginally from the others, suggesting that the model is not highly sensitive to C in this range. While RandomizedSearch explores the space more efficiently than GridSearch, the resulting gain over simpler settings is limited, indicating that further improvements are more likely to come from better features or embeddings rather than additional tuning of C alone.

In [ ]:
results_df = pd.DataFrame(search.cv_results_)
results_df

In [ ]:
search.best_params_

In [ ]:

best_clf = search.best_estimator_

In [ ]:
y_pred = best_clf.predict(X_val_vector)

In [ ]:
evaluate_classifier(best_clf, X_val_vector, y_val,
                    pos_label="jailbreak",
                    neg_label="benign")

The model achieves good overall performance (Accuracy ≈ 0.86, F1 ≈ 0.83) with a balanced trade-off between precision and recall for the jailbreak class. However, around 19% of jailbreak instances are still misclassified as benign, which is a relevant limitation in a safety-critical setting. While specificity remains high (~0.90), the remaining false negatives indicate that the classifier, although robust, is not fully reliable for strict jailbreak detection without further tuning or threshold adjustment.

### other embeddings

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import numpy as np

# 🔧 griglia iperparametri per Logistic Regression
param_grid_lr = {
    "C": np.linspace(0.1, 5.0, 50),
    "max_iter": [20000],
}

# includi bow più tutti gli embeddings
all_representations = ["bow"] + embedding_names
# embedding_names = ["w2v", "glove_twitter", "glove_wiki", "fasttext", "sbert"]

for rep_name in all_representations:
    print("\n" + "="*70)
    print(f"🔹 GridSearch LogisticRegression su rappresentazione: {rep_name}")
    print("="*70)

    # --- scegli la matrice giusta ---
    if rep_name == "bow":
        X_train_vec = X_train_vector
        X_val_vec   = X_val_vector
    else:
        X_train_vec = other_embeddings["train"][rep_name]
        X_val_vec   = other_embeddings["val"][rep_name]

    # modello base
    base_clf = LogisticRegression()

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_lr,
        cv=3,
        verbose=True,
        n_jobs=-1,
    )

    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params LogisticRegression per {rep_name}: {best_params}")

    # Valutazione sul validation set
    evaluate_classifier(
        best_model,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )

    # nome variante parlante
    C_val = best_params["C"]
    variant_name = f"grid_lr_C_{C_val:.3f}_maxiter_20000"

    # 🔐 registra nel registry IL MODELLO GIUSTO
    register_model(
        registry=models_registry,
        model_name="LogisticRegression",  # <-- CORRETTO
        embedding_name=rep_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


##Multinomial Bayes

make negative values positive in other embeddings to use the model

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
import numpy as np

param_grid_nb = {
    "alpha": np.linspace(0.1, 5.0, 50),
    "fit_prior": [True, False],
}

all_representations = ["bow"] + embedding_names   # bow, w2v, glove_twitter, ...

for rep_name in all_representations:
    print("\n" + "="*70)
    print(f"🔹 GridSearch MultinomialNB su rappresentazione: {rep_name}")
    print("="*70)

    if rep_name == "bow":
        # BoW: already positive
        X_train_vec = X_train_vector
        X_val_vec   = X_val_vector
    else:
        # other embeddigs could contain negative values
        X_train_raw = other_embeddings["train"][rep_name]
        X_val_raw   = other_embeddings["val"][rep_name]

        # make the features non-negative
        scaler = MinMaxScaler(feature_range=(0.0, 1.0))
        X_train_vec = scaler.fit_transform(X_train_raw)
        X_val_vec   = scaler.transform(X_val_raw)

    # base model NB
    base_clf = MultinomialNB()

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_nb,
        cv=3,
        verbose=True,
        n_jobs=-1,
    )

    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params NB per {rep_name}: {best_params}")

    # evaluation on validation set
    evaluate_classifier(
        best_model,
        X_val_vec,
        y_val,
        pos_label="jailbreak",
        neg_label="benign",
    )

    # variant name
    variant_name = "grid_nb_alpha_0.1_5.0"

    register_model(
        registry=models_registry,
        model_name="MultinomialNB",
        embedding_name=rep_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


The MultinomialNB model performs very well on BoW, achieving balanced metrics with strong recall (0.86) and a solid F1-score (0.79). This confirms that NB is well-suited for sparse count features. In contrast, all dense embeddings (Word2Vec, GloVe, FastText) lead to substantially worse performance: although recall remains high, specificity drops sharply, meaning many benign samples are misclassified as jailbreaks. This makes these representations unreliable for a security task where false positives are costly. SBERT is the only exception, delivering performance close to BoW, but still slightly less stable. Overall, BoW remains the best representation for Naive Bayes in this jailbreak-detection setting.


##K-Nearest Neighbors

In [ ]:
# === GRID SEARCH KNN SU BOW (CountVectorizer) ===

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

print("\n" + "="*70)
print("🔹 GridSearch KNN su rappresentazione: bow (CountVectorizer)")
print("="*70)

X_train_vec_bow = X_train_vector
X_val_vec_bow   = X_val_vector

param_grid_knn = {
    "n_neighbors": [1, 3, 5, 7, 9, 11],
    "weights": ["uniform", "distance"],
}

base_clf = KNeighborsClassifier()

search = GridSearchCV(
    estimator=base_clf,
    param_grid=param_grid_knn,
    cv=3,
    verbose=True,
    n_jobs=-1,
)

search.fit(X_train_vec_bow, y_train)

best_model  = search.best_estimator_
best_params = search.best_params_

print(f"\n✅ Best params KNN per bow: {best_params}")

# Valutazione sul validation set
evaluate_classifier(
    best_model,
    X_val_vec_bow,
    y_val,
    pos_label="jailbreak",
    neg_label="benign",
)

# Variante per il registry
variant_name = "grid_knn_bow_k_1_11"

# Registrazione nel models_registry
register_model(
    models_registry,
    model_name="KNN",
    embedding_name="bow",
    variant_name=variant_name,
    model=best_model,
    params=best_params,
)


The KNN model on BoW performs poorly for the target task despite hyperparameter tuning. While specificity is very high (0.96), the recall for the positive jailbreak class is extremely low (0.22), meaning that most jailbreak instances are missed. The seemingly acceptable accuracy (0.66) is misleading and mainly driven by the majority benign class. Overall, this model is unsuitable for jailbreak detection, where false negatives are particularly costly.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid_knn = {
    "n_neighbors": [1, 3, 5, 7, 9, 11],
    "weights": ["uniform", "distance"],
}

for emb_name in embedding_names:
    print("\n" + "="*70)
    print(f"🔹 GridSearch KNN su embedding: {emb_name}")
    print("="*70)

    X_train_vec = other_embeddings["train"][emb_name]

    base_clf = KNeighborsClassifier()

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_knn,
        cv=3,
        verbose=True,
        n_jobs=-1,
    )

    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params KNN per {emb_name}: {best_params}")

    variant_name = "grid_knn_k_1_11"

    register_model(
        models_registry,
        model_name="KNN",
        embedding_name=emb_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


In [ ]:
register_model(
    models_registry,
    model_name="KNN",
    embedding_name=emb_name,
    variant_name=variant_name,
    model=best_model,
    params=best_params,
)


##Decision Tree

In [ ]:
# === GRID SEARCH DECISION TREE SU BOW (CountVectorizer) ===

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

print("\n" + "="*70)
print("🔹 GridSearch DecisionTree su rappresentazione: bow (CountVectorizer)")
print("="*70)

X_train_vec_bow = X_train_vector
X_val_vec_bow   = X_val_vector

param_grid_dt = {
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 10, 50],
    "min_samples_leaf": [1, 5, 10],
}

base_clf = DecisionTreeClassifier(random_state=42)

search = GridSearchCV(
    estimator=base_clf,
    param_grid=param_grid_dt,
    cv=3,
    verbose=True,
    n_jobs=-1,
)

search.fit(X_train_vec_bow, y_train)

best_model  = search.best_estimator_
best_params = search.best_params_

print(f"\n✅ Best params DecisionTree per bow: {best_params}")

# Valutazione sul validation set
evaluate_classifier(
    best_model,
    X_val_vec_bow,
    y_val,
    pos_label="jailbreak",
    neg_label="benign",
)

# Nome variante per il registry
variant_name = "grid_dt_bow_depth_split_leaf"

# Registrazione nel models_registry
register_model(
    models_registry,
    model_name="DecisionTree",
    embedding_name="bow",
    variant_name=variant_name,
    model=best_model,
    params=best_params,
)


This Decision Tree, even with the best hyperparameters (max_depth=10, min_samples_leaf=5, min_samples_split=50), shows clearly weaker performance compared to linear models. Accuracy is moderate (≈0.77), but the main issue is the low recall on the jailbreak class (≈0.58): the model misses a large portion of malicious samples, as confirmed by the 125 false negatives. The confusion matrix highlights a conservative behavior, with good specificity (≈0.89) but poor sensitivity, meaning the tree prefers predicting benign and fails to capture many jailbreaks. This is a typical limitation of Decision Trees on high-dimensional sparse representations like BoW, where they struggle to generalize and tend to underfit when regularized to avoid overfitting. Overall, this model is not suitable if detecting jailbreaks is a priority.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

param_grid_dt = {
    "max_depth": [None, 5, 10, 20],
    "min_samples_split": [2, 10, 50],
    "min_samples_leaf": [1, 5, 10],
}

for emb_name in embedding_names:
    print("\n" + "="*70)
    print(f"🔹 GridSearch DecisionTree su embedding: {emb_name}")
    print("="*70)

    X_train_vec = other_embeddings["train"][emb_name]

    base_clf = DecisionTreeClassifier(random_state=42)

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_dt,
        cv=3,
        verbose=True,
        n_jobs=-1,
    )

    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params DecisionTree per {emb_name}: {best_params}")

    variant_name = "grid_dt_depth_split_leaf"

    register_model(
        models_registry,
        model_name="DecisionTree",
        embedding_name=emb_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


##Kernel SVM

This RBF Kernel SVM on BoW achieves solid but not outstanding performance. Accuracy is around 0.85, with a good balance between precision (0.84) and specificity (0.90), meaning the model is quite conservative and reliable when predicting the benign class. However, recall on the jailbreak class is lower (≈0.78), so a non-negligible fraction of jailbreak samples is still missed. The confusion matrix confirms this behavior: false negatives (67 jailbreaks predicted as benign) are noticeably higher than false positives (45 benign predicted as jailbreak). Overall, the model improves over simpler methods like KNN and Decision Trees, but it does not reach the performance of linear models or SVMs with stronger representations, indicating that the non-linear kernel adds limited value on sparse BoW features.

In [ ]:
# === GRID SEARCH KERNEL SVM (RBF) SU BOW (CountVectorizer) ===

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np

print("\n" + "="*70)
print("🔹 GridSearch Kernel SVM (RBF) su rappresentazione: bow (CountVectorizer)")
print("="*70)

X_train_vec_bow = X_train_vector
X_val_vec_bow   = X_val_vector

param_grid_ksvm = {
    "C": np.logspace(-2, 2, 7),        # 0.01 → 100
    "gamma": np.logspace(-3, 1, 5),    # 0.001 → 10
}

base_clf = SVC(kernel="rbf")

search = GridSearchCV(
    estimator=base_clf,
    param_grid=param_grid_ksvm,
    cv=3,
    verbose=True,
    n_jobs=-1,
)

search.fit(X_train_vec_bow, y_train)

best_model  = search.best_estimator_
best_params = search.best_params_

print(f"\n✅ Best params Kernel SVM per bow: {best_params}")

# Valutazione sul validation set
evaluate_classifier(
    best_model,
    X_val_vec_bow,
    y_val,
    pos_label="jailbreak",
    neg_label="benign",
)

# Variante per il registry
variant_name = "grid_rbf_C_gamma_bow"

register_model(
    models_registry,
    model_name="KernelSVM",
    embedding_name="bow",
    variant_name=variant_name,
    model=best_model,
    params=best_params,
)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid_ksvm = {
    "C": np.logspace(-2, 2, 7),        # da 0.01 a 100
    "gamma": np.logspace(-3, 1, 5),    # da 0.001 a 10
}

for emb_name in embedding_names:
    print("\n" + "="*70)
    print(f"🔹 GridSearch Kernel SVM (RBF) su embedding: {emb_name}")
    print("="*70)

    X_train_vec = other_embeddings["train"][emb_name]

    base_clf = SVC(kernel="rbf")

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_ksvm,
        cv=3,
        verbose=True,
        n_jobs=-1,
    )

    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params Kernel SVM per {emb_name}: {best_params}")

    variant_name = "grid_rbf_C_gamma"

    register_model(
        models_registry,
        model_name="KernelSVM",
        embedding_name=emb_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


##(Deep) Neural Network

In [ ]:
# === GRID SEARCH MLPClassifier SU BOW (CountVectorizer) ===

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

print("\n" + "="*70)
print("🔹 GridSearch MLPClassifier su rappresentazione: bow (CountVectorizer)")
print("="*70)

X_train_vec_bow = X_train_vector
X_val_vec_bow   = X_val_vector

param_grid_mlp = {
    "hidden_layer_sizes": [(50,), (100,)],
    "alpha": np.linspace(1e-4, 1.0, 10),
}

base_clf = MLPClassifier(
    max_iter=200,
    random_state=42,
)

search = GridSearchCV(
    estimator=base_clf,
    param_grid=param_grid_mlp,
    cv=3,
    verbose=True,
    n_jobs=-1,
)

search.fit(X_train_vec_bow, y_train)

best_model  = search.best_estimator_
best_params = search.best_params_

print(f"\n✅ Best params MLP per bow: {best_params}")

# Valutazione sul validation set
evaluate_classifier(
    best_model,
    X_val_vec_bow,
    y_val,
    pos_label="jailbreak",
    neg_label="benign",
)

variant_name = "grid_mlp_alpha_hidden_bow"

register_model(
    models_registry,
    model_name="MLP",
    embedding_name="bow",
    variant_name=variant_name,
    model=best_model,
    params=best_params,
)


This MLP trained on BoW achieves solid but not outstanding performance. Accuracy is reasonably high (0.85) and the F1-score (~0.81) indicates a fairly balanced trade-off between precision and recall. The model detects jailbreak prompts with good recall (0.81), meaning most malicious cases are caught, but it still misses a non-negligible fraction. At the same time, specificity (0.88) shows it is fairly reliable on benign inputs, although false positives are not trivial. The confusion matrix confirms this behavior: the model is slightly biased toward predicting jailbreak, which helps recall but increases benign misclassification. Overall, the MLP performs comparably to linear models on BoW, but it does not clearly outperform simpler approaches, suggesting that the added non-linearity brings limited benefit with this representation.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid_mlp = {
    "hidden_layer_sizes": [(50,), (100,)],
    "alpha": np.linspace(1e-4, 1.0, 10),   # valori tipo quelli che usavi prima
}

for emb_name in embedding_names:
    print("\n" + "="*70)
    print(f"🔹 GridSearch MLPClassifier su embedding: {emb_name}")
    print("="*70)

    X_train_vec = other_embeddings["train"][emb_name]

    base_clf = MLPClassifier(
        max_iter=200,
        random_state=42,
    )

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_mlp,
        cv=3,
        verbose=True,
        n_jobs=-1,
    )

    search.fit(X_train_vec, y_train)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params MLP per {emb_name}: {best_params}")

    variant_name = "grid_mlp_alpha_hidden"

    register_model(
        models_registry,
        model_name="MLP",
        embedding_name=emb_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("\n" + "="*70)
print("🌿 RandomForest su rappresentazione: bow (CountVectorizer)")
print("="*70)

X_train_vec_bow = X_train_vector
X_val_vec_bow   = X_val_vector

rf_bow = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    n_jobs=-1,
    random_state=42
)

rf_bow.fit(X_train_vec_bow, y_train)

# Valutazione sul validation set
evaluate_classifier(
    clf=rf_bow,
    X_val=X_val_vec_bow,
    y_val=y_val,
    pos_label="jailbreak",
    neg_label="benign",
    show_plots=True,
)

# Registrazione nel registry
register_model(
    registry=models_registry,
    model_name="RandomForest",
    embedding_name="bow",
    variant_name="rf_fixed_200_10",
    model=rf_bow,
    params={
        "n_estimators": 200,
        "max_depth": 10,
        "random_state": 42,
    },
)


This RandomForest trained on BoW shows a very conservative behavior. Accuracy is moderate (≈0.75), but it is driven almost entirely by the benign class. The model achieves very high specificity (≈0.95), meaning it almost never flags benign samples as jailbreak, and precision is high because predicted jailbreaks are usually correct. However, recall on the jailbreak class is poor (≈0.44): more than half of the true jailbreak samples are missed and classified as benign, as clearly visible from the confusion matrix (167 false negatives vs 133 true positives). As a consequence, the F1-score is low. In practice, this model is unsuitable if the goal is to detect jailbreaks reliably, because it prioritizes avoiding false alarms over catching malicious cases.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

print("Embedding disponibili:", list(other_embeddings["train"].keys()))

for emb_name in other_embeddings["train"].keys():

    print("\n" + "="*80)
    print(f"🌿 RandomForest su embedding: {emb_name}")
    print("="*80)

    X_train_emb = other_embeddings["train"][emb_name]
    X_val_emb   = other_embeddings["val"][emb_name]

    print("Shapes:", X_train_emb.shape, X_val_emb.shape)

    rf_emb = RandomForestClassifier(
        n_estimators=200,
        max_depth=10,
        n_jobs=-1,
        random_state=42
    )

    rf_emb.fit(X_train_emb, y_train)

    # Predizioni su validation (per loggare l'F1)
    y_val_pred = rf_emb.predict(X_val_emb)
    f1 = f1_score(y_val, y_val_pred, pos_label="jailbreak")
    print(f"F1-score (validation) per {emb_name}: {f1:.4f}")

    # Valutazione completa con la tua funzione
    evaluate_classifier(
        clf=rf_emb,
        X_val=X_val_emb,
        y_val=y_val,
        pos_label="jailbreak",
        neg_label="benign",
        show_plots=False,   # metti True se vuoi tutti i grafici per ogni embedding
    )

    # Registrazione nel registry PER QUESTO EMBEDDING
    register_model(
        registry=models_registry,
        model_name="RandomForest",
        embedding_name=emb_name,
        variant_name="rf_fixed_200_10",
        model=rf_emb,
        params={
            "n_estimators": 200,
            "max_depth": 10,
            "random_state": 42
        },
    )


## XGBoost

In [ ]:
# === GRID SEARCH XGBOOST SU BOW (CountVectorizer) ===

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

print("\n" + "="*70)
print("🔹 GridSearch XGBoost su rappresentazione: bow (CountVectorizer)")
print("="*70)

X_train_vec_bow = X_train_vector
X_val_vec_bow   = X_val_vector

# 1) Etichette numeriche per XGBoost: 0 = benign, 1 = jailbreak
y_train_bin = y_train.map(to_binary_label)
y_val_bin   = y_val.map(to_binary_label)

# 2) Scorer che massimizza F1 sulla classe positiva (1 = jailbreak)
f1_jb = make_scorer(f1_score, pos_label=1)

param_grid_xgb = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [5, 10, 15],
}

base_clf = XGBClassifier(
    n_jobs=-1,
    tree_method="hist",
    eval_metric="logloss",
    random_state=42,
    objective="binary:logistic",   # esplicito
)

search = GridSearchCV(
    estimator=base_clf,
    param_grid=param_grid_xgb,
    cv=3,
    scoring=f1_jb,
    verbose=True,
    n_jobs=-1,
)

# ⬅️ usa y_train_bin (numerico)
search.fit(X_train_vec_bow, y_train_bin)

best_model  = search.best_estimator_
best_params = search.best_params_

print(f"\n✅ Best params XGBoost per bow: {best_params}")

# 3) Valutazione sul validation set (versione rapida con sklearn)
from sklearn.metrics import classification_report, confusion_matrix

y_val_pred_bin = best_model.predict(X_val_vec_bow)

print("\nConfusion matrix (binaria, 0=benign, 1=jailbreak):")
print(confusion_matrix(y_val_bin, y_val_pred_bin))

print("\nClassification report:")
print(classification_report(
    y_val_bin,
    y_val_pred_bin,
    target_names=["benign", "jailbreak"]
))

# 4) Registrazione nel models_registry
variant_name = "grid_xgb_bow_n_est_depth"

register_model(
    models_registry,
    model_name="XGBoost",
    embedding_name="bow",
    variant_name=variant_name,
    model=best_model,
    params=best_params,
)


The XGBoost model trained on the BoW representation achieves a solid and well-balanced performance. With the best configuration found by grid search (max_depth = 15 and n_estimators = 100), the classifier reaches an overall accuracy of 0.84. Precision and recall are reasonably balanced for both classes: benign samples are correctly identified most of the time, while the model also maintains a good recall on the jailbreak class (0.81), which is critical for this task. The confusion matrix shows that errors are fairly symmetric, with a moderate number of false positives and false negatives, indicating no extreme bias toward either class. Overall, XGBoost performs competitively compared to other models on BoW, offering a good trade-off between precision and recall without being overly conservative or overly permissive.

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
import numpy as np

# ⬇️ etichette binarie per XGBoost: 0 = benign, 1 = jailbreak
y_train_bin = y_train.map(to_binary_label)
y_val_bin   = y_val.map(to_binary_label)

# scorer F1 sulla classe positiva (1 = jailbreak)
f1_jb = make_scorer(f1_score, pos_label=1)

param_grid_xgb = {
    "n_estimators": [10, 50, 100, 200],
    "max_depth": [5, 10, 15],
}

for emb_name in embedding_names:
    print("\n" + "="*70)
    print(f"🔹 GridSearch XGBoost su embedding: {emb_name}")
    print("="*70)

    X_train_vec = other_embeddings["train"][emb_name]
    X_val_vec   = other_embeddings["val"][emb_name]

    base_clf = XGBClassifier(
        n_jobs=-1,
        tree_method="hist",
        eval_metric="logloss",
        random_state=42,
        objective="binary:logistic",
    )

    search = GridSearchCV(
        estimator=base_clf,
        param_grid=param_grid_xgb,
        cv=3,
        scoring=f1_jb,
        verbose=True,
        n_jobs=-1,
    )

    # ⬅️ usa le etichette binarie
    search.fit(X_train_vec, y_train_bin)

    best_model  = search.best_estimator_
    best_params = search.best_params_

    print(f"\n✅ Best params XGBoost per {emb_name}: {best_params}")

    # 🔎 valutazione sul validation set con la TUA funzione
    evaluate_classifier(
        best_model,
        X_val_vec,
        y_val_bin,   # ⬅️ etichette 0/1
        pos_label=1,
        neg_label=0,
    )

    variant_name = "grid_xgb_n_est_depth"

    register_model(
        models_registry,
        model_name="XGBoost",
        embedding_name=emb_name,
        variant_name=variant_name,
        model=best_model,
        params=best_params,
    )


#saving all of the models on drive

In [ ]:
import pickle
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# folder on drive
DRIVE_BASE_PATH = "/content/drive/MyDrive/AI_for_security"
FILENAME        = "models_registry.pkl"

SAVE_PATH       = os.path.join(DRIVE_BASE_PATH, FILENAME)

print(f"Cartella di salvataggio:\n{DRIVE_BASE_PATH}")

os.makedirs(DRIVE_BASE_PATH, exist_ok=True)

print(f"📁 Sto salvando (e sovrascrivendo se necessario) su: {SAVE_PATH}")

# saving
try:
    with open(SAVE_PATH, "wb") as f:
        pickle.dump(models_registry, f)
    print("\n✅ Salvataggio completato! Il file è stato aggiornato con i nuovi modelli.")
except Exception as e:
    print(f"\n❌ Errore durante il salvataggio: {e}")

#Loading all of the models from drive

In [ ]:
import pickle
import os

LOAD_PATH = "/content/drive/MyDrive/AI_for_security/models_registry.pkl"

with open(LOAD_PATH, "rb") as f:
    models_registry = pickle.load(f)

print("✅ Modelli caricati!")


In [ ]:
for model_name, emb_dict in models_registry.items():
    print(f"\n📌 MODEL: {model_name}")
    print("-" * 60)

    for emb_name, variants in emb_dict.items():
        print(f"  🔹 Embedding: {emb_name}")

        for variant_name, info in variants.items():
            print(f"     ➝ Variant: {variant_name}")
            print(f"        Parameters: {info['params']}")
            print(f"        Type of model: {type(info['model']).__name__}")


# final evaluation

## all models, all embeddings

###precision-recall curves

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# function to generate colors
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n  # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# 1) labels
y_test_bin = y_test.map(to_binary_label).values

num_curves = sum(
    len(variants)
    for _, emb_dict in models_registry.items()
    for emb_name, variants in emb_dict.items()
    if emb_name in other_embeddings["test"]
)

colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(18, 12))

curve_idx = 0

# 3) Loop
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["test"]:
            continue

        X_test_emb = other_embeddings["test"][emb_name]

        for variant_name, info in variants.items():
            clf = info["model"]

            if hasattr(clf, "predict_proba"):
                scores = clf.predict_proba(X_test_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_test_emb)
            else:
                continue

            precision, recall, _ = precision_recall_curve(y_test_bin, scores)
            ap = average_precision_score(y_test_bin, scores)

            label = f"{model_name} | {emb_name} | {variant_name} (AP={ap:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(recall, precision, label=label, color=color, linewidth=2)

# 4) Styling
plt.xlabel("Recall", fontsize=13)
plt.ylabel("Precision", fontsize=13)
plt.title("Precision–Recall curves (Test Set)", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


Looking at the Precision–Recall curves and the Average Precision (AP) values, the three best models are all based on SBERT embeddings, which clearly dominate the ranking.

The SVC with SBERT is the top performer, achieving the highest AP (≈ 0.93). Its curve stays consistently high across almost the entire recall range, meaning it maintains strong precision even when recall increases. This indicates very reliable separation between benign and jailbreak samples.

Immediately behind it is the Kernel SVM (RBF) with SBERT, with an AP essentially identical (≈ 0.93). Its curve almost overlaps with the linear SVC, showing that the non-linear kernel does not add much beyond what SBERT already provides, but still delivers excellent and very stable performance.

The third best model is the MLP with SBERT (AP ≈ 0.93). While slightly below the two SVMs, it still shows a strong precision–recall trade-off and confirms that neural embeddings combined with a simple neural classifier work extremely well for this task.

Overall, the takeaway is clear: the representation (SBERT) matters more than the specific classifier, and SVM-based models extract the most consistent benefit from it.

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Funzione per generare N colori molto diversi (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Etichette binarie del VALIDATION SET
# -------------------------------------------
y_val_bin = y_val.map(to_binary_label).values

# -------------------------------------------
# 2) Conta quante curve verranno REALMENTE disegnate
#    (solo modelli che hanno predict_proba o decision_function,
#     e solo embedding per cui abbiamo il "val")
# -------------------------------------------
def count_real_curves_val():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["val"]:
                continue   # niente embedding val -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                    count += 1   # questa curva la disegniamo davvero
    return count

num_curves = count_real_curves_val()
print("Numero curve PR sul validation set:", num_curves)

# Palette colori
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(10, 8))
curve_idx = 0

# -------------------------------------------
# 3) Loop modelli/embedding sul VALIDATION
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["val"]:
            continue

        X_val_emb = other_embeddings["val"][emb_name]

        for variant_name, info in variants.items():
            clf = info["model"]

            # score continuo
            if hasattr(clf, "predict_proba"):
                scores = clf.predict_proba(X_val_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_val_emb)
            else:
                continue  # niente score -> niente curva

            precision, recall, _ = precision_recall_curve(y_val_bin, scores)
            ap = average_precision_score(y_val_bin, scores)

            label = f"{model_name} | {emb_name} | {variant_name} (AP={ap:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(recall, precision, label=label, color=color, linewidth=2)

# -------------------------------------------
# 4) Styling
# -------------------------------------------
plt.xlabel("Recall", fontsize=13)
plt.ylabel("Precision", fontsize=13)
plt.title("Precision–Recall curves (Validation Set)", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


The three best models on the validation set are all based on SBERT embeddings, which clearly dominate the Precision–Recall analysis.

The top performer is the linear SVC with SBERT (AP ≈ 0.939). Its PR curve stays consistently high across a wide recall range, indicating an excellent balance between precision and recall. This suggests that SBERT representations make the classes almost linearly separable, allowing a relatively simple classifier to perform extremely well.

Very close behind is the Kernel SVM (RBF) with SBERT (AP ≈ 0.935). The performance is nearly indistinguishable from the linear SVC, meaning that adding non-linearity brings little benefit here. This reinforces the idea that the embedding quality, rather than model complexity, is the key factor.

The MLP with SBERT ranks third (AP ≈ 0.934) and shows similarly strong behavior. However, its slight drop compared to SVC suggests that the neural network does not extract substantially more information than what is already encoded in the embeddings, while being more complex and potentially less stable.

Overall, the takeaway is clear: SBERT is the decisive component, and simpler classifiers like linear SVC already exploit its structure almost optimally.

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Funzione per generare N colori molto diversi (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Etichette binarie del TRAINING SET
# -------------------------------------------
y_train_bin = y_train.map(to_binary_label).values

# -------------------------------------------
# 2) Conta quante curve verranno REALMENTE disegnate sul TRAIN
# -------------------------------------------
def count_real_curves_train():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["train"]:
                continue   # niente embedding train -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                    count += 1   # questa curva la disegniamo davvero
    return count

num_curves = count_real_curves_train()
print("Numero curve PR sul training set:", num_curves)

# Palette colori
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(10, 8))
curve_idx = 0

# -------------------------------------------
# 3) Loop modelli/embedding sul TRAINING
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["train"]:
            continue

        X_train_emb = other_embeddings["train"][emb_name]

        for variant_name, info in variants.items():
            clf = info["model"]

            # score continuo
            if hasattr(clf, "predict_proba"):
                scores = clf.predict_proba(X_train_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_train_emb)
            else:
                continue  # niente score -> niente curva

            precision, recall, _ = precision_recall_curve(y_train_bin, scores)
            ap = average_precision_score(y_train_bin, scores)

            label = f"{model_name} | {emb_name} | {variant_name} (AP={ap:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(recall, precision, label=label, color=color, linewidth=2)

# -------------------------------------------
# 4) Styling
# -------------------------------------------
plt.xlabel("Recall", fontsize=13)
plt.ylabel("Precision", fontsize=13)
plt.title("Precision–Recall curves (Training Set)", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


Looking at the training-set Precision–Recall curves, the three best models all achieve an Average Precision (AP) of 1.00, which means they separate the two classes perfectly on the training data.

The top performers are SVC with SBERT embeddings, Kernel SVM (RBF) with SBERT embeddings, and MLP with SBERT embeddings. In all three cases, the PR curves stay at precision ≈ 1 across almost the entire recall range, indicating near-zero false positives and false negatives on the training set.

This result is extremely strong but also a red flag for overfitting: perfect AP on training suggests that these models may be memorizing patterns rather than learning generalizable decision boundaries. Their true value must therefore be judged by comparing these results with the validation and test PR curves, where any drop in AP will reveal how well this apparent performance transfers to unseen data.

### ROC

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Funzione per generare N colori molto diversi (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Etichette binarie del TEST SET
# -------------------------------------------
y_test_bin = y_test.map(to_binary_label).values

# -------------------------------------------
# 2) Conta quante ROC verranno REALMENTE disegnate sul TEST
# -------------------------------------------
def count_real_roc_curves_test():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["test"]:
                continue   # niente embedding test -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                    count += 1   # questa curva la disegniamo davvero
    return count

num_curves = count_real_roc_curves_test()
print("Numero curve ROC sul test set:", num_curves)

# Palette colori distinte
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(10, 8))
curve_idx = 0

# -------------------------------------------
# 3) Loop modelli/embedding sul TEST
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["test"]:
            continue

        X_test_emb = other_embeddings["test"][emb_name]

        for variant_name, info in variants.items():
            clf = info["model"]

            # score continuo
            if hasattr(clf, "predict_proba"):
                scores = clf.predict_proba(X_test_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_test_emb)
            else:
                continue  # niente score → niente curva

            fpr, tpr, _ = roc_curve(y_test_bin, scores)
            auc_val = roc_auc_score(y_test_bin, scores)

            label = f"{model_name} | {emb_name} | {variant_name} (AUC={auc_val:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(fpr, tpr, label=label, color=color, linewidth=2)

# -------------------------------------------
# 4) Diagonale random + styling
# -------------------------------------------
plt.plot([0, 1], [0, 1], linestyle="--", color="grey", linewidth=1, label="Random baseline")

plt.xlabel("False Positive Rate (1 - Specificity)", fontsize=13)
plt.ylabel("True Positive Rate (Recall)", fontsize=13)
plt.title("ROC curves (Test Set)", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


Looking at the ROC curves on the test set, the three best models are clearly those based on SBERT embeddings, which consistently dominate the upper-left region of the plot and achieve the highest AUC values.

The best overall model is linear SVC with SBERT, reaching an AUC of about 0.95. This indicates an excellent ranking ability: the model separates jailbreak and benign samples very well across almost all thresholds, with high true positive rates even at low false positive rates.

Very close behind is Kernel SVM (RBF) with SBERT, with an AUC just below 0.95. Its ROC curve almost overlaps with the linear SVC one, suggesting that the non-linear kernel does not add much on top of SBERT representations, which are already highly expressive.

The third best model is MLP with SBERT, with an AUC around 0.94. While slightly inferior to the SVM-based approaches, it still shows strong discriminative power and a smooth ROC curve, confirming that neural models also benefit significantly from high-quality sentence embeddings.

Overall, these results show that representation quality (SBERT) matters more than model complexity, and that SVM-based classifiers on SBERT provide the most reliable performance on the test set.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Funzione per generare N colori molto diversi (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Etichette binarie del VALIDATION SET
# -------------------------------------------
y_val_bin = y_val.map(to_binary_label).values

# -------------------------------------------
# 2) Conta quante ROC verranno REALMENTE disegnate sul VAL
# -------------------------------------------
def count_real_roc_curves_val():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["val"]:
                continue   # niente embedding val -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                    count += 1   # questa curva la disegniamo davvero
    return count

num_curves = count_real_roc_curves_val()
print("Numero curve ROC sul validation set:", num_curves)

# Palette colori
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(10, 8))
curve_idx = 0

# -------------------------------------------
# 3) Loop modelli/embedding sul VALIDATION
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["val"]:
            continue

        X_val_emb = other_embeddings["val"][emb_name]

        for variant_name, info in variants.items():
            clf = info["model"]

            # score continuo
            if hasattr(clf, "predict_proba"):
                scores = clf.predict_proba(X_val_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_val_emb)
            else:
                continue  # niente score -> niente curva

            fpr, tpr, _ = roc_curve(y_val_bin, scores)
            auc_val = roc_auc_score(y_val_bin, scores)

            label = f"{model_name} | {emb_name} | {variant_name} (AUC={auc_val:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(fpr, tpr, label=label, color=color, linewidth=2)

# -------------------------------------------
# 4) Diagonale random + styling
# -------------------------------------------
plt.plot([0, 1], [0, 1], linestyle="--", color="grey", linewidth=1, label="Random baseline")

plt.xlabel("False Positive Rate (1 - Specificity)", fontsize=13)
plt.ylabel("True Positive Rate (Recall)", fontsize=13)
plt.title("ROC curves (Validation Set)", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


Looking at the validation ROC curves, the three best models are all based on SBERT embeddings, which clearly dominate the ranking.

The best overall model is SVC with SBERT, reaching an AUC of about 0.954. Its ROC curve stays closest to the top-left corner across almost the entire range, meaning it achieves a very strong trade-off between true positive rate and false positive rate. This indicates excellent separability between benign and jailbreak samples.

Very close behind is Kernel SVM (RBF) with SBERT, with an AUC of roughly 0.952. Its curve is nearly indistinguishable from the linear SVC + SBERT, suggesting that once high-quality contextual embeddings are used, adding non-linearity brings only marginal gains.

The third best model is MLP with SBERT, with an AUC around 0.949. While slightly below the two SVM-based approaches, it still shows consistently high recall at low false positive rates, confirming that SBERT embeddings provide a very strong signal even for neural classifiers.

Overall, the takeaway is clear: representation matters more than the classifier. SBERT embeddings consistently outperform word-based embeddings, and SVM-based models exploit them slightly better than MLPs on the validation set.

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Funzione per generare N colori molto diversi (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Etichette binarie del TRAINING SET
# -------------------------------------------
y_train_bin = y_train.map(to_binary_label).values

# -------------------------------------------
# 2) Conta quante ROC verranno REALMENTE disegnate sul TRAIN
# -------------------------------------------
def count_real_roc_curves_train():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["train"]:
                continue   # niente embedding train -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                    count += 1   # questa curva la disegniamo davvero
    return count

num_curves = count_real_roc_curves_train()
print("Numero curve ROC sul training set:", num_curves)

# Palette colori
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(10, 8))
curve_idx = 0

# -------------------------------------------
# 3) Loop modelli/embedding sul TRAINING
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["train"]:
            continue

        X_train_emb = other_embeddings["train"][emb_name]

        for variant_name, info in variants.items():
            clf = info["model"]

            # score continuo
            if hasattr(clf, "predict_proba"):
                scores = clf.predict_proba(X_train_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_train_emb)
            else:
                continue  # niente score -> niente curva

            fpr, tpr, _ = roc_curve(y_train_bin, scores)
            auc_val = roc_auc_score(y_train_bin, scores)

            label = f"{model_name} | {emb_name} | {variant_name} (AUC={auc_val:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(fpr, tpr, label=label, color=color, linewidth=2)

# -------------------------------------------
# 4) Diagonale random + styling
# -------------------------------------------
# baseline classificatore random
plt.plot([0, 1], [0, 1], linestyle="--", color="grey", linewidth=1, label="Random baseline")

plt.xlabel("False Positive Rate (1 - Specificity)", fontsize=13)
plt.ylabel("True Positive Rate (Recall)", fontsize=13)
plt.title("ROC curves (Training Set)", fontsize=16)
plt.grid(True, linestyle="--", alpha=0.5)
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()


On the training set ROC curves, the three best-performing models all achieve near-perfect discrimination, with AUC values equal to 1.000, which clearly indicates overfitting rather than genuine generalization ability.

The top performer is XGBoost with SBERT embeddings, which reaches an AUC of 1.000 and shows a curve almost perfectly hugging the top-left corner. This means the model separates benign and jailbreak samples almost perfectly on the training data, a strong signal that it has memorized training patterns.

Very close to it is Kernel SVM with SBERT embeddings, also achieving an AUC of 1.000. Its ROC curve is indistinguishable from the ideal classifier on the training set, again suggesting extremely high capacity relative to the dataset size.

The third best is MLP with SBERT embeddings, which likewise reaches an AUC of 1.000. This confirms a consistent pattern: SBERT embeddings combined with high-capacity models dominate on training data, but such perfect performance is unrealistic and should be interpreted as a warning sign.

Overall, these results show that while SBERT-based models are extremely expressive, training-set ROC is not informative for model selection here. Validation and test ROC curves are essential to distinguish true performance from overfitting.

### calibration curves

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Funzione per generare N colori molto diversi (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equispaziati tra 0 e 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Etichette binarie del TEST SET
# -------------------------------------------
y_test_bin = y_test.map(to_binary_label).values

# -------------------------------------------
# 2) Conta quante calibration curves verranno disegnate sul TEST
# -------------------------------------------
def count_real_calib_curves_test():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["test"]:
                continue

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba"):
                    count += 1
    return count

num_curves = count_real_calib_curves_test()
print("Numero calibration curves sul test set:", num_curves)

# Palette colori distinte
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(16, 12))   # 🔥 immagine molto più grande
curve_idx = 0

# -------------------------------------------
# 3) Loop modelli/embedding sul TEST
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["test"]:
            continue

        X_test_emb = other_embeddings["test"][emb_name]

        for variant_name, info in variants.items():

            clf = info["model"]

            # SOLO modelli con predict_proba
            if not hasattr(clf, "predict_proba"):
                continue

            probs = clf.predict_proba(X_test_emb)[:, 1]

            # reliability diagram
            prob_true, prob_pred = calibration_curve(
                y_test_bin, probs, n_bins=10, strategy="quantile"
            )

            # Brier score
            brier = brier_score_loss(y_test_bin, probs)

            label = f"{model_name} | {emb_name} | {variant_name} (Brier={brier:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(prob_pred, prob_true, marker="o", linewidth=2,
                     label=label, color=color)

# -------------------------------------------
# 4) Perfect calibration line + styling
# -------------------------------------------
plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration")

plt.xlabel("Predicted probability", fontsize=16)   # 🔥 inglese
plt.ylabel("Fraction of positives", fontsize=16)   # 🔥 inglese
plt.title("Calibration curves (Test Set)", fontsize=20)
plt.grid(True, linestyle="--", alpha=0.4)

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=12)
plt.tight_layout()
plt.show()


Looking at the calibration curves on the test set and the corresponding Brier scores, the three best-calibrated models are all based on strong semantic embeddings and probabilistic outputs.

The best result is obtained by MLP with SBERT embeddings, which achieves the lowest Brier score (≈ 0.099). Its calibration curve stays very close to the diagonal, indicating that predicted probabilities closely match the true empirical frequencies. This means the model’s confidence estimates are highly reliable, not just its classifications.

The second best model is XGBoost with SBERT embeddings (Brier ≈ 0.121). Its curve is also well aligned with the perfect calibration line across most probability bins, showing only mild deviations at higher confidence levels. This suggests a good balance between discriminative power and probability calibration.

The third best performer is KNN with SBERT embeddings (Brier ≈ 0.126). While slightly less calibrated than MLP and XGBoost, it still provides reasonably accurate probability estimates compared to other models, especially in the mid-probability range.

Overall, these results confirm that SBERT embeddings consistently lead to better-calibrated models, and that neural or boosted models on top of them (MLP, XGBoost) are particularly effective when reliable probability estimates are required.

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Function to generate N very distinct colors (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equally spaced between 0 and 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Binary labels for the VALIDATION SET
# -------------------------------------------
y_val_bin = y_val.map(to_binary_label).values

# -------------------------------------------
# 2) Count how many calibration curves will actually be drawn on VAL
#    (only models with predict_proba and embeddings with "val")
# -------------------------------------------
def count_real_calib_curves_val():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["val"]:
                continue   # no val embedding -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba"):
                    count += 1
    return count

num_curves = count_real_calib_curves_val()
print("Number of calibration curves on validation set:", num_curves)

# Distinct color palette
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(16, 12))   # bigger figure
curve_idx = 0

# -------------------------------------------
# 3) Loop over models/embeddings on VALIDATION
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["val"]:
            continue

        X_val_emb = other_embeddings["val"][emb_name]

        for variant_name, info in variants.items():

            clf = info["model"]

            # ONLY models with predict_proba
            if not hasattr(clf, "predict_proba"):
                continue

            probs = clf.predict_proba(X_val_emb)[:, 1]

            # reliability diagram
            prob_true, prob_pred = calibration_curve(
                y_val_bin, probs, n_bins=10, strategy="quantile"
            )

            # Brier score
            brier = brier_score_loss(y_val_bin, probs)

            label = f"{model_name} | {emb_name} | {variant_name} (Brier={brier:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(prob_pred, prob_true, marker="o", linewidth=2,
                     label=label, color=color)

# -------------------------------------------
# 4) Perfect calibration line + styling
# -------------------------------------------
plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration")

plt.xlabel("Predicted probability", fontsize=16)
plt.ylabel("Fraction of positives", fontsize=16)
plt.title("Calibration curves (Validation Set)", fontsize=20)
plt.grid(True, linestyle="--", alpha=0.4)

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=12)
plt.tight_layout()
plt.show()


Looking at the validation-set calibration results, the three best-calibrated models are all based on SBERT embeddings, which clearly dominate in terms of probability reliability.

The MLP with SBERT is the best overall, achieving the lowest Brier score (~0.095) and a calibration curve that stays closest to the diagonal. This indicates that its predicted probabilities closely match the true empirical frequencies across bins.

The Logistic Regression with SBERT comes next (Brier ~0.107), showing very stable and well-aligned probabilities despite being a simpler linear model. This suggests that SBERT embeddings already encode the class separation in a way that is easy to calibrate.

The XGBoost with SBERT ranks third (Brier ~0.111), still providing good calibration, though with slightly more deviation at intermediate probability levels, likely due to the model’s higher flexibility.

Overall, these results indicate that SBERT embeddings consistently lead to well-calibrated models, and among them, MLP achieves the best balance between expressive power and probability reliability.

In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
import matplotlib.pyplot as plt
import numpy as np
import colorsys

# -------------------------------------------
# Function to generate N very distinct colors (HLS)
# -------------------------------------------
def generate_distinct_colors(n):
    colors = []
    for i in range(n):
        hue = i / n        # equally spaced between 0 and 1
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# -------------------------------------------
# 1) Binary labels for the TRAINING SET
# -------------------------------------------
y_train_bin = y_train.map(to_binary_label).values

# -------------------------------------------
# 2) Count how many calibration curves will actually be drawn on TRAIN
#    (only models with predict_proba and embeddings with "train")
# -------------------------------------------
def count_real_calib_curves_train():
    count = 0
    for model_name, emb_dict in models_registry.items():
        for emb_name, variants in emb_dict.items():

            if emb_name not in other_embeddings["train"]:
                continue   # no train embedding -> skip

            for variant_name, info in variants.items():
                clf = info["model"]
                if hasattr(clf, "predict_proba"):
                    count += 1
    return count

num_curves = count_real_calib_curves_train()
print("Number of calibration curves on training set:", num_curves)

# Distinct color palette
colors = generate_distinct_colors(num_curves)

plt.figure(figsize=(16, 12))   # bigger figure
curve_idx = 0

# -------------------------------------------
# 3) Loop over models/embeddings on TRAINING
# -------------------------------------------
for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        if emb_name not in other_embeddings["train"]:
            continue

        X_train_emb = other_embeddings["train"][emb_name]

        for variant_name, info in variants.items():

            clf = info["model"]

            # ONLY models with predict_proba
            if not hasattr(clf, "predict_proba"):
                continue

            probs = clf.predict_proba(X_train_emb)[:, 1]

            # reliability diagram
            prob_true, prob_pred = calibration_curve(
                y_train_bin, probs, n_bins=10, strategy="quantile"
            )

            # Brier score
            brier = brier_score_loss(y_train_bin, probs)

            label = f"{model_name} | {emb_name} | {variant_name} (Brier={brier:.3f})"
            color = colors[curve_idx]
            curve_idx += 1

            plt.plot(prob_pred, prob_true, marker="o", linewidth=2,
                     label=label, color=color)

# -------------------------------------------
# 4) Perfect calibration line + styling
# -------------------------------------------
plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration")

plt.xlabel("Predicted probability", fontsize=16)
plt.ylabel("Fraction of positives", fontsize=16)
plt.title("Calibration curves (Training Set)", fontsize=20)
plt.grid(True, linestyle="--", alpha=0.4)

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=12)
plt.tight_layout()
plt.show()


Looking at the training Precision–Recall curves, the three best-performing models all achieve perfect or near-perfect Average Precision (AP ≈ 1.00), which clearly separates them from the rest.

The SVC with SBERT embeddings shows a PR curve that stays essentially at precision ≈ 1 across almost the entire recall range, indicating that the model ranks positive (jailbreak) examples almost perfectly on the training data. This suggests extremely strong separability in the SBERT feature space.

The Kernel SVM (RBF) with SBERT embeddings exhibits an almost identical behavior, with AP equal to 1.00 and a curve tightly hugging the top-right corner. This confirms that the non-linear decision boundary can fully exploit the expressive power of SBERT on the training set.

The MLP with SBERT embeddings also reaches AP ≈ 1.00, maintaining very high precision even at high recall values. This indicates that the neural network easily fits the training distribution when using contextual sentence embeddings.

Overall, all three models demonstrate near-perfect training performance, which is a strong signal of representational power but also raises a clear overfitting warning. Their dominance on the training set makes validation and test performance crucial to assess real generalization.

## all models for each embedding

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import colorsys
from sklearn.metrics import (
    precision_recall_curve, average_precision_score,
    roc_curve, roc_auc_score, brier_score_loss
)
from sklearn.calibration import calibration_curve

# --------------------------------------------------------------
# 1) Funzione colori molto distinti (HLS)
# --------------------------------------------------------------
def generate_distinct_colors(n):
    colors = []
    if n == 0:
        return colors
    for i in range(n):
        hue = i / n
        lightness = 0.5
        saturation = 0.9
        rgb = colorsys.hls_to_rgb(hue, lightness, saturation)
        colors.append(rgb)
    return colors

# --------------------------------------------------------------
# 2) Etichette binarie
# --------------------------------------------------------------
y_train_bin = y_train.map(to_binary_label).values
y_val_bin   = y_val.map(to_binary_label).values
y_test_bin  = y_test.map(to_binary_label).values

# --------------------------------------------------------------
# 3) Split dictionary
# --------------------------------------------------------------
splits = {
    "train": {"y": y_train_bin, "X_dict": other_embeddings["train"]},
    "val":   {"y": y_val_bin,   "X_dict": other_embeddings["val"]},
    "test":  {"y": y_test_bin,  "X_dict": other_embeddings["test"]},
}

# --------------------------------------------------------------
# 4) Embedding list
# --------------------------------------------------------------
embedding_list = list(other_embeddings["train"].keys())
print("Embeddings trovati:", embedding_list)


In [ ]:
# ============================================================
# PRECISION–RECALL CURVES per embedding e split
# (ordine: TEST -> VAL -> TRAIN)
# ============================================================

for emb_name in embedding_list:
    print(f"\n=== Precision–Recall curves per embedding: {emb_name} ===")

    # ordine esplicito degli split
    for split_name in ["test", "val", "train"]:

        split_data = splits[split_name]

        if emb_name not in split_data["X_dict"]:
            print(f"  -> {emb_name} non presente in {split_name}, skip.")
            continue

        y_bin = split_data["y"]
        X_emb = split_data["X_dict"][emb_name]

        # Conta PR curve reali
        def count_real_pr_curves():
            c = 0
            for model_name, emb_dict in models_registry.items():
                if emb_name not in emb_dict:
                    continue
                for variant_name, info in emb_dict[emb_name].items():
                    clf = info["model"]
                    if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                        c += 1
            return c

        num_curves = count_real_pr_curves()
        print(f"  [{split_name}] num PR curves: {num_curves}")
        if num_curves == 0:
            continue

        colors = generate_distinct_colors(num_curves)
        curve_idx = 0

        plt.figure(figsize=(10, 8))

        for model_name, emb_dict in models_registry.items():
            if emb_name not in emb_dict:
                continue

            for variant_name, info in emb_dict[emb_name].items():
                clf = info["model"]

                if hasattr(clf, "predict_proba"):
                    scores = clf.predict_proba(X_emb)[:, 1]
                elif hasattr(clf, "decision_function"):
                    scores = clf.decision_function(X_emb)
                else:
                    continue

                precision, recall, _ = precision_recall_curve(y_bin, scores)
                ap = average_precision_score(y_bin, scores)

                label = f"{model_name} | {variant_name} (AP={ap:.3f})"
                plt.plot(recall, precision,
                         label=label,
                         color=colors[curve_idx],
                         linewidth=2)

                curve_idx += 1

        plt.xlabel("Recall", fontsize=14)
        plt.ylabel("Precision", fontsize=14)
        plt.title(f"Precision–Recall — {emb_name} — {split_name}", fontsize=16)
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)
        plt.tight_layout()
        plt.show()


In [ ]:
# ============================================================
# ROC CURVES per embedding e split
# (ordine: TEST -> VAL -> TRAIN)
# ============================================================

for emb_name in embedding_list:
    print(f"\n=== ROC curves per embedding: {emb_name} ===")

    # ordine esplicito degli split
    for split_name in ["test", "val", "train"]:

        split_data = splits[split_name]

        if emb_name not in split_data["X_dict"]:
            print(f"  -> {emb_name} non presente in {split_name}, skip.")
            continue

        y_bin = split_data["y"]
        X_emb = split_data["X_dict"][emb_name]

        def count_real_roc_curves():
            c = 0
            for model_name, emb_dict in models_registry.items():
                if emb_name not in emb_dict:
                    continue
                for variant_name, info in emb_dict[emb_name].items():
                    clf = info["model"]
                    if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                        c += 1
            return c

        num_curves = count_real_roc_curves()
        print(f"  [{split_name}] num ROC curves: {num_curves}")
        if num_curves == 0:
            continue

        colors = generate_distinct_colors(num_curves)
        curve_idx = 0

        plt.figure(figsize=(10, 8))

        for model_name, emb_dict in models_registry.items():
            if emb_name not in emb_dict:
                continue

            for variant_name, info in emb_dict[emb_name].items():
                clf = info["model"]

                if hasattr(clf, "predict_proba"):
                    scores = clf.predict_proba(X_emb)[:, 1]
                elif hasattr(clf, "decision_function"):
                    scores = clf.decision_function(X_emb)
                else:
                    continue

                fpr, tpr, _ = roc_curve(y_bin, scores)
                auc_val = roc_auc_score(y_bin, scores)

                label = f"{model_name} | {variant_name} (AUC={auc_val:.3f})"
                plt.plot(
                    fpr,
                    tpr,
                    label=label,
                    color=colors[curve_idx],
                    linewidth=2
                )
                curve_idx += 1

        # baseline random
        plt.plot([0, 1], [0, 1], "k--", label="Random baseline")

        plt.xlabel("False Positive Rate", fontsize=14)
        plt.ylabel("True Positive Rate", fontsize=14)
        plt.title(f"ROC — {emb_name} — {split_name}", fontsize=16)
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)
        plt.tight_layout()
        plt.show()


In [ ]:
# ============================================================
# CALIBRATION CURVES per embedding e split
# (ordine: TEST -> VAL -> TRAIN)
# ============================================================

for emb_name in embedding_list:
    print(f"\n=== Calibration curves per embedding: {emb_name} ===")

    # ordine esplicito degli split
    for split_name in ["test", "val", "train"]:

        split_data = splits[split_name]

        if emb_name not in split_data["X_dict"]:
            print(f"  -> {emb_name} non presente in {split_name}, skip.")
            continue

        y_bin = split_data["y"]
        X_emb = split_data["X_dict"][emb_name]

        # Conta curve reali
        def count_real_calib_curves():
            c = 0
            for model_name, emb_dict in models_registry.items():
                if emb_name not in emb_dict:
                    continue
                for variant_name, info in emb_dict[emb_name].items():
                    if hasattr(info["model"], "predict_proba"):
                        c += 1
            return c

        num_curves = count_real_calib_curves()
        print(f"  [{split_name}] num Calibration curves: {num_curves}")
        if num_curves == 0:
            continue

        colors = generate_distinct_colors(num_curves)
        curve_idx = 0

        plt.figure(figsize=(10, 8))

        # Disegno curve
        for model_name, emb_dict in models_registry.items():
            if emb_name not in emb_dict:
                continue

            for variant_name, info in emb_dict[emb_name].items():
                clf = info["model"]

                if not hasattr(clf, "predict_proba"):
                    continue

                probs = clf.predict_proba(X_emb)[:, 1]

                prob_true, prob_pred = calibration_curve(
                    y_bin,
                    probs,
                    n_bins=10,
                    strategy="quantile"
                )

                brier = brier_score_loss(y_bin, probs)

                label = f"{model_name} | {variant_name} (Brier={brier:.3f})"

                plt.plot(
                    prob_pred,
                    prob_true,
                    marker="o",
                    label=label,
                    color=colors[curve_idx],
                    linewidth=2
                )

                curve_idx += 1

        # Linea modello perfettamente calibrato
        plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration")

        plt.xlabel("Predicted probability", fontsize=14)
        plt.ylabel("Fraction of positives", fontsize=14)
        plt.title(f"Calibration — {emb_name} — {split_name}", fontsize=16)
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=10)
        plt.tight_layout()
        plt.show()


## all embeddings for each model

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

split_order = ["test", "val", "train"]

for model_name, emb_dict in models_registry.items():
    print(f"\n============================")
    print(f"=== PRECISION–RECALL: {model_name} ===")
    print(f"============================")

    for split_name in split_order:
        split_data = splits[split_name]
        y_bin = split_data["y"]
        X_dict = split_data["X_dict"]

        # Conta quante curve avremo (embedding+variante di QUESTO modello che esistono nello split)
        def count_curves_for_model_split():
            c = 0
            for emb_name, variants in emb_dict.items():
                if emb_name not in X_dict:
                    continue
                for variant_name, info in variants.items():
                    clf = info["model"]
                    if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                        c += 1
            return c

        num_curves = count_curves_for_model_split()
        print(f"  [{split_name}] num PR curves: {num_curves}")
        if num_curves == 0:
            continue

        colors = generate_distinct_colors(num_curves)
        curve_idx = 0

        plt.figure(figsize=(10, 8))

        for emb_name, variants in emb_dict.items():
            if emb_name not in X_dict:
                continue
            X_emb = X_dict[emb_name]

            for variant_name, info in variants.items():
                clf = info["model"]

                if hasattr(clf, "predict_proba"):
                    scores = clf.predict_proba(X_emb)[:, 1]
                elif hasattr(clf, "decision_function"):
                    scores = clf.decision_function(X_emb)
                else:
                    continue

                precision, recall, _ = precision_recall_curve(y_bin, scores)
                ap = average_precision_score(y_bin, scores)

                label = f"{emb_name} | {variant_name} (AP={ap:.3f})"
                plt.plot(
                    recall, precision,
                    label=label,
                    color=colors[curve_idx],
                    linewidth=2
                )
                curve_idx += 1

        plt.xlabel("Recall", fontsize=13)
        plt.ylabel("Precision", fontsize=13)
        plt.title(f"PR – {model_name} – {split_name}", fontsize=15)
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="lower left", fontsize=9)
        plt.tight_layout()
        plt.show()


In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

split_order = ["test", "val", "train"]

for model_name, emb_dict in models_registry.items():
    print(f"\n============================")
    print(f"=== ROC: {model_name} ===")
    print(f"============================")

    for split_name in split_order:
        split_data = splits[split_name]
        y_bin = split_data["y"]
        X_dict = split_data["X_dict"]

        def count_curves_for_model_split():
            c = 0
            for emb_name, variants in emb_dict.items():
                if emb_name not in X_dict:
                    continue
                for variant_name, info in variants.items():
                    clf = info["model"]
                    if hasattr(clf, "predict_proba") or hasattr(clf, "decision_function"):
                        c += 1
            return c

        num_curves = count_curves_for_model_split()
        print(f"  [{split_name}] num ROC curves: {num_curves}")
        if num_curves == 0:
            continue

        colors = generate_distinct_colors(num_curves)
        curve_idx = 0

        plt.figure(figsize=(10, 8))

        for emb_name, variants in emb_dict.items():
            if emb_name not in X_dict:
                continue
            X_emb = X_dict[emb_name]

            for variant_name, info in variants.items():
                clf = info["model"]

                if hasattr(clf, "predict_proba"):
                    scores = clf.predict_proba(X_emb)[:, 1]
                elif hasattr(clf, "decision_function"):
                    scores = clf.decision_function(X_emb)
                else:
                    continue

                fpr, tpr, _ = roc_curve(y_bin, scores)
                auc_val = roc_auc_score(y_bin, scores)

                label = f"{emb_name} | {variant_name} (AUC={auc_val:.3f})"
                plt.plot(
                    fpr, tpr,
                    label=label,
                    color=colors[curve_idx],
                    linewidth=2
                )
                curve_idx += 1

        # baseline random
        plt.plot([0, 1], [0, 1], "k--", label="Random baseline")

        plt.xlabel("False Positive Rate", fontsize=13)
        plt.ylabel("True Positive Rate", fontsize=13)
        plt.title(f"ROC – {model_name} – {split_name}", fontsize=15)
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="lower right", fontsize=9)
        plt.tight_layout()
        plt.show()


In [ ]:
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss

split_order = ["test", "val", "train"]

for model_name, emb_dict in models_registry.items():
    print(f"\n============================")
    print(f"=== CALIBRATION: {model_name} ===")
    print(f"============================")

    for split_name in split_order:
        split_data = splits[split_name]
        y_bin = split_data["y"]
        X_dict = split_data["X_dict"]

        def count_curves_for_model_split():
            c = 0
            for emb_name, variants in emb_dict.items():
                if emb_name not in X_dict:
                    continue
                for variant_name, info in variants.items():
                    clf = info["model"]
                    if hasattr(clf, "predict_proba"):
                        c += 1
            return c

        num_curves = count_curves_for_model_split()
        print(f"  [{split_name}] num Calibration curves: {num_curves}")
        if num_curves == 0:
            continue

        colors = generate_distinct_colors(num_curves)
        curve_idx = 0

        plt.figure(figsize=(10, 8))

        for emb_name, variants in emb_dict.items():
            if emb_name not in X_dict:
                continue
            X_emb = X_dict[emb_name]

            for variant_name, info in variants.items():
                clf = info["model"]

                if not hasattr(clf, "predict_proba"):
                    continue

                probs = clf.predict_proba(X_emb)[:, 1]

                prob_true, prob_pred = calibration_curve(
                    y_bin, probs, n_bins=10, strategy="quantile"
                )

                brier = brier_score_loss(y_bin, probs)

                label = f"{emb_name} | {variant_name} (Brier={brier:.3f})"
                plt.plot(
                    prob_pred,
                    prob_true,
                    marker="o",
                    label=label,
                    color=colors[curve_idx],
                    linewidth=2
                )
                curve_idx += 1

        plt.plot([0, 1], [0, 1], "k--", label="Perfect calibration")

        plt.xlabel("Predicted probability", fontsize=13)
        plt.ylabel("Fraction of positives", fontsize=13)
        plt.title(f"Calibration – {model_name} – {split_name}", fontsize=15)
        plt.grid(True, linestyle="--", alpha=0.5)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=9)
        plt.tight_layout()
        plt.show()


## Best models for each metric:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    roc_auc_score,
    average_precision_score,
)
from sklearn.calibration import calibration_curve

# 1️⃣ label → 0/1
def to_binary_labels(y_series):
    return y_series.map(lambda v: 1 if str(v).lower() == "jailbreak" else 0).values

# 2️⃣ get the respective data split
def get_X_for_split(emb_name: str, split_name: str):
    if emb_name == "bow":
        if split_name == "train":
            return X_train_vector
        elif split_name == "val":
            return X_val_vector
        elif split_name == "test":
            return X_test_vector
        else:
            raise ValueError(f"Split sconosciuto: {split_name}")
    else:
        return other_embeddings[split_name][emb_name]

# Label binarie per tutti gli split
y_train_bin = to_binary_labels(y_train)
y_val_bin   = to_binary_labels(y_val)
y_test_bin  = to_binary_labels(y_test)

print("Setup OK!")


In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import (accuracy_score, precision_recall_fscore_support,
                             roc_auc_score, average_precision_score)

rows = []

# Ensure we use the binary ground truth
y_true_test_bin = y_test_bin

for model_name, emb_dict in models_registry.items():
    for emb_name, variants in emb_dict.items():

        # Check if embedding is available
        try:
            X_test_emb = get_X_for_split(emb_name, "test")
        except KeyError:
            print(f"⚠️ No test embedding for {model_name}/{emb_name}, skipping.")
            continue

        for variant_name, info in variants.items():
            clf = info["model"]

            # 1. Discrete predictions
            try:
                y_pred = clf.predict(X_test_emb)
            except Exception as e:
                print(f"❌ Predict failed for {model_name}/{emb_name}/{variant_name}: {e}")
                continue

            # --- 🛠️ CRITICAL FIX HERE 🛠️ ---
            # If the model predicts strings (e.g., 'jailbreak'), convert them to integers (0/1).
            # This ensures types match y_true_test_bin.
            if y_pred.dtype.kind in {'U', 'S', 'O'}:  # Check for string/object types
                # Map 'jailbreak' to 1, everything else (e.g., 'benign') to 0
                y_pred = np.where(y_pred == 'jailbreak', 1, 0)

            # Now both y_true_test_bin and y_pred are integers.
            # --------------------------------

            acc = accuracy_score(y_true_test_bin, y_pred)

            # pos_label=1 because we are now working with 0s and 1s
            prec, rec, f1, _ = precision_recall_fscore_support(
                y_true_test_bin, y_pred,
                average="binary", pos_label=1
            )

            # 2. Continuous scores (probabilities)
            if hasattr(clf, "predict_proba"):
                try:
                    # Usually predict_proba returns [prob_class_0, prob_class_1]
                    # We need to check the order of classes in clf.classes_
                    class_1_index = np.where(clf.classes_ == 'jailbreak')[0][0]
                    scores = clf.predict_proba(X_test_emb)[:, class_1_index]
                except (IndexError, AttributeError):
                    # Fallback if classes are not strings or logic fails
                    scores = clf.predict_proba(X_test_emb)[:, 1]
            elif hasattr(clf, "decision_function"):
                scores = clf.decision_function(X_test_emb)
            else:
                scores = None

            if scores is not None:
                try:
                    auc_roc = roc_auc_score(y_true_test_bin, scores)
                    ap      = average_precision_score(y_true_test_bin, scores)
                except ValueError:
                    auc_roc = np.nan
                    ap      = np.nan
            else:
                auc_roc = np.nan
                ap      = np.nan

            rows.append({
                "model": model_name,
                "embedding": emb_name,
                "variant": variant_name,
                "accuracy": acc,
                "precision": prec,
                "recall": rec,
                "f1": f1,
                "auc_roc": auc_roc,
                "avg_precision": ap,
            })

# DataFrame sorted by descending F1
results_df = pd.DataFrame(rows)
results_df_sorted = results_df.sort_values(by="f1", ascending=False)

print("\n🎯 Top 10 models on Test (sorted by F1):")
results_df_sorted.head(10)

In [ ]:
N_BEST = 3

best_models = results_df_sorted.head(N_BEST).copy()
print("🏆 best models:")
best_models


In [ ]:
def evaluate_model_on_all_splits(model, emb_name: str, title: str = ""):
    print("\n" + "#" * 80)
    print(title)
    print("#" * 80)

    # Ordine desiderato: TEST → VAL → TRAIN
    for split_name, (X_mat, y_vec) in {
        "TEST":  (get_X_for_split(emb_name, "test"),  y_test),
        "VAL":   (get_X_for_split(emb_name, "val"),   y_val),
        "TRAIN": (get_X_for_split(emb_name, "train"), y_train),
    }.items():

        print(f"\n🔹 Split: {split_name}")
        evaluate_classifier(
            model,
            X_mat,
            y_vec,
            pos_label="jailbreak",
            neg_label="benign",
        )


In [ ]:
# 🔁 Metriche che vuoi considerare
metrics = ["accuracy", "precision", "recall", "f1", "auc_roc", "avg_precision"]

TOP_K = 3  # quanti modelli top per metrica

for metric in metrics:
    if metric not in results_df.columns:
        print(f"\n⚠️ Metrica {metric} non trovata nel DataFrame, skip.")
        continue

    print("\n" + "="*90)
    print(f"🏅 TOP {TOP_K} MODELS PER METRIC {metric} (evaluated on TEST SET)")
    print("="*90)

    # scarta eventuali NaN per quella metrica
    df_metric = results_df.dropna(subset=[metric])

    if df_metric.empty:
        print(f"⚠️ Nessun modello valido per metrica {metric}, skip.")
        continue

    # ordina decrescente per la metrica
    df_metric_sorted = df_metric.sort_values(by=metric, ascending=False)

    # prendi i top-K
    top_k = df_metric_sorted.head(TOP_K).copy()
    display(top_k)  # comodo in notebook per vedere la tabella

    # per ciascuno dei top-K: chiama evaluate_model_on_all_splits
    for rank, (_, row) in enumerate(top_k.iterrows(), start=1):
        model_name   = row["model"]
        emb_name     = row["embedding"]
        variant_name = row["variant"]

        clf = models_registry[model_name][emb_name][variant_name]["model"]

        title = (
            f"🏆 Metric: {metric} | Rank: {rank} | "
            f"MODEL: {model_name} | EMB: {emb_name} | VAR: {variant_name} | "
            f"{metric} (test) = {row[metric]:.4f}"
        )

        # Usa la tua funzione già definita
        evaluate_model_on_all_splits(clf, emb_name, title)
